In [ ]:
!git clone https://github.com/gibsonx/jlpt_simulator.git
%cd jlpt_simulator
!git checkout dev

!apt-get install python3-dev graphviz libgraphviz-dev pkg-config
!pip install -r requirements.txt 
!pip install pandas numpy==2.0.2

# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [1]:
import json
import random
import datetime
import os
import logging
import pickle
import re
import uuid
import threading
import asyncio
from typing import *
from tqdm import tqdm
import time
import yaml
import sys
import asyncio
import json
import random
import time
from tqdm.asyncio import tqdm_asyncio
from graphs.common.utils import collect_vocabulary
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from libs.LLMs import *
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [2]:
# Import N3 Vocabulary
file_path = '../../Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("../../Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

#### load Models

#### Exam Paper Outline
### A. overall thinking the structure of an exam
1. distribution of the difficulty 
2. topics
3. reasoning

In [3]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_chinese): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
# 
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
# 
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

instruction = """
Section 1: Vocabulary and Grammar
- Kanji reading (kanji_reading): 8 questions
- Write Chinese characters (write_chinese): 6 questions
- Word Meaning Selection (word_meaning): 11 questions
- Synonyms substitution (synonym_substitution): 5 questions
- word usage (word_usage): 5 questions
- Grammar fill in the blank (sentence_grammar): 13 questions
- Sentence sorting (sentence_sort): 5 questions
- Grammar structure selection (sentence_structure): 4-5 questions

Section 2: Reading Comprehension
- Short passages (short_passage_read): 4 articles
- Mid-size passages (midsize_passage_read): 2 articles
- Long passages (long_passage_read): 1 articles
- Information retrieval (info_retrieval): 1 articles

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 6 questions
- Key understanding (keypoint_understanding): 6 questions
- Summary understanding (summary_understanding): 3 questions
- Active expression (active_expression): 4 questions
- Immediate acknowledgment (immediate_ack): 9 questions
"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper. the complexity should be restricted to N3 level and respect japanese culture. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of easy, moderate, and difficult questions to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First,  randomly pick words in 'Vocabulary' for questions in Vocabulary and Grammar. At Section 2 and 3, randomly choose topics in 'TopicList' for Reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"  
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [4]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")
    
    
class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )
    
    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [5]:
# Read the topics from a file, sort them, and print the sorted list  
def process_topics(file_path):
    try:  
        # Read the file  
        with open(file_path, 'r', encoding='utf-8') as file:  
            topics = file.readlines()  
          
        # Remove any extra whitespace or newline characters  
        topics = [topic.strip() for topic in topics if topic.strip()]  
          
        # Shuffle the topics randomly  
        random.shuffle(topics)  
                
    except FileNotFoundError:  
        print("The file was not found. Please check the file path.")  
    except Exception as e:  
        print("An error occurred:", str(e)) 
      
    except FileNotFoundError:  
        print("The file was not found. Please check the file path.")  
    except Exception as e:  
        print("An error occurred:", str(e)) 

In [6]:
# Preload all topics from the file
with open("../../Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-05-14 21:35:47,357 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [7]:
from IPython.display import display, Markdown, Latex
display(Markdown(initial_outline.as_str))

# 日本語能力試験N3 模擬試験

## 第1部：語彙・文法

### 漢字読み (kanji_reading)

8問。与えられた漢字の読み仮名をひらがなで答える。

- **膳**
- **湿る**
- **現れ**
- **冗談**
- **栄養**
- **豊か**
- **筋肉**
- **答案**

### 漢字書き取り (write_chinese)

6問。ひらがなで書かれた単語を漢字で書く。

- **しゅつじょう（出場）**
- **しょくひん（食品）**
- **せつやく（節約）**
- **けんこう（健康）**
- **しゅうにゅう（収入）**
- **かんしゃ（感謝）**

### 語彙意味選択 (word_meaning)

11問。下線部の語の意味に最も近いものを選ぶ。

- **必死**
- **調子**
- **破産**
- **争う**
- **題名**
- **要素**
- **湿気**
- **努力**
- **礼儀**
- **資本**
- **特徴**

### 同義語置き換え (synonym_substitution)

5問。文中の語句を同じ意味の語に置き換える。

- **現代**
- **種類**
- **完全**
- **冷やす**
- **自然**

### 語彙使い方 (word_usage)

5問。語彙の使い方として正しい文を選ぶ。

- **保存**
- **拍手**
- **要点**
- **礼儀**
- **順調**

### 文法穴埋め (sentence_grammar)

13問。空欄に入る最も適切な語句や助詞を選ぶ。

- **句**
- **助詞**
- **順**
- **場合**
- **程度**
- **状態**
- **内容**
- **判断**
- **条件**
- **関する**
- **比較**
- **場合**
- **原因**

### 文の並べ替え (sentence_sort)

5問。バラバラになった文を正しい順番に並べる。

- **感謝**
- **決心**
- **説明**
- **提案**
- **結論**

### 文法構造選択 (sentence_structure)

4問。文の構造として正しいものを選ぶ。

- **原因・理由**
- **逆接**
- **条件**
- **目的**

## 第2部：読解

### 短文読解 (short_passage_read)

4題。短い文章を読み、内容に関する問いに答える。

- **店で価格を尋ねる**
- **レストランで食べ物を注文する**
- **道を尋ねる**
- **週末の予定について話す**

### 中文章読解 (midsize_passage_read)

2題。中程度の長さの文章を読み、主旨や詳細情報について答える。

- **趣味について話す**
- **家族について話す**

### 長文読解 (long_passage_read)

1題。やや長い文章を読み、全体の内容や筆者の意図を理解する。

- **旅行の計画について話す**

### 情報検索 (info_retrieval)

1題。案内や掲示などの情報から必要な情報を探し出す。

- **電車の切符を買う**

## 第3部：聴解

### 話題理解 (topic_understanding)

6問。流れる会話や説明から話題を把握する。

- **お気に入りのレストランについて話す**
- **健康とフィットネスについて話す**
- **最近の映画について話す**
- **支払い方法について話す**
- **家事の分担について話す**
- **公共施設の利用方法について話す**

### 要点理解 (keypoint_understanding)

6問。会話の中の重要な情報や要点を聞き取る。

- **仕事のプロジェクトについて話す**
- **最近のニュースについて意見を述べる**
- **交通状況について話す**
- **ショッピング体験を説明する**
- **ペットについて話す**
- **新しいスキルを学ぶ計画について話す**

### 概要理解 (summary_understanding)

3問。聴いた内容の大意やまとめを問う。

- **異文化交流について話す**
- **地域のイベントについて話す**
- **おすすめの旅行先について話す**

### 積極的表現 (active_expression)

4問。会話の中で自分の意見や希望を述べる場面を聞き取る。

- **キャリア目標について話す**
- **将来の抱負について話す**
- **友人へのプレゼント選びについて話す**
- **特別な日の計画について話す**

### 即時応答 (immediate_ack)

9問。短い会話や問いかけに対して即座に適切な返答を選ぶ。

- **バスの時刻表を尋ねる**
- **領収書を求める**
- **おすすめを尋ねる**
- **引っ越しの準備について話す**
- **学校や職場での一日の流れについて話す**
- **言語学習のコツについて話す**
- **好きな季節について話す**
- **地元の食べ物や特産品について話す**
- **健康診断や医者への訪問について話す**

In [8]:
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = subsection['subsection_title'].split(' (')[1].rstrip(')')
        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing 漢字読み (kanji_reading):   0%|          | 0/8 [00:00<?, ?it/s]

---WEB SEARCH---
---WEB SEARCH---
---Generator----


2025-05-14 21:36:00,547 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


---REVISOR---


2025-05-14 21:36:07,423 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


...

---REVISOR---


2025-05-14 22:45:26,473 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


--- AI Reviser feels Good Enough ---
--- Formatter ---


2025-05-14 22:45:29,163 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"

Processing 第3部：聴解: 100%|██████████| 5/5 [27:40<00:00, 332.03s/it]

Total execution time: 4181.70 seconds


In [9]:
yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# Display as formatted YAML
display(Markdown(yaml_str))

sections:
- section_title: 第1部：語彙・文法
  subsections:
  - subsection_title: 漢字読み (kanji_reading)
    description: 8問。与えられた漢字の読み仮名をひらがなで答える。
    question_topics:
    - topic: お気に入りのレストランについて話す
      result:
        question: 下線部の語の読み方として最も適当なものを、1・2・3・4の中から一つ選びなさい。私は休日によくお気に入りのレストランに行きます。
        choices:
        - おきにいり
        - おきにい
        - おきあいり
        - おきにいみ
        correct_answer: 1
        explanation: 「お気に入り」は「特に好きなもの」「愛着のあるもの」という意味で、日常会話や文章でよく使われる表現です。語全体の正しい読み方は「おきにいり」です。JLPT
          N3レベルの学習者にとっては、送り仮名を含む語の読み方を正確に覚えることが重要なポイントです。選択肢2、3、4は実際には使われない読み方なので、正しい音のつながりと送り仮名に注意して覚えましょう。
    - topic: 地元の観光名所について話す
      result:
        question: 地元の **観光名所** について話す。  1 かんこうみょうしょ  2 かんこうめいしょ  3 かんこうめいところ  4 かんこうなところ
        correct_answer: 2
        choices:
        - かんこうみょうしょ
        - かんこうめいしょ
        - かんこうめいところ
        - かんこうなところ
        explanation: 「観光名所」はN3レベルでよく出る語彙ですが、「名所（めいしょ）」の読みが難しいと感じる学習者が多いです。「名（めい）」は「有名（ゆうめい）」などで馴染みがあっても、「所（しょ）」の音読みとの組み合わせが分かりづらいからです。また、「名所」を「みょうしょ」と読んでしまうミスや、「名」と「所」を訓読みで「めいところ」などと読んでしまうこともよく見られます。選択肢2「かんこうめいしょ」が正しい読みです。
    - topic: 現れ
      result:
        question: 次の漢字の読み方として最もよいものを、1・2・3・4から一つ選びなさい。彼の努力がついに現れとなった。
        choices:
        - あらわれ
        - うつれ
        - ひかれ
        - みだれ
        correct_answer: 1
        explanation: 「現れ（あらわれ）」は、目に見える形で現れることや、何かが表面に出てくることを意味する名詞です。N3レベルで頻出する語彙であり、動詞「現れる（あらわれる）」の名詞形です。他の選択肢「うつれ」「ひかれ」「みだれ」は、それぞれ「移れ」「光れ」「乱れ」という意味や読みですが、「現れ」とは関係ありません。学習者が混同しやすいポイントは、「現」という漢字が「げん」「うつ（す）」など複数の読み方を持つこと、また「現れ」と「表れ（あらわれ）」の使い分けがあることです。関連語：現れる（あらわれる／to
          appear）、表れる（あらわれる／to become apparent）
    - topic: 冗談
      result:
        question: 次の漢字の読み方として正しいものを選びなさい。冗談は言わないでください。
        choices:
        - じょうたん
        - じょうだん
        - じょだん
        - じょうなん
        correct_answer: 2
        explanation: 「冗談」は日常会話でよく使われるN3レベルの語彙です。「冗」は「じょう」、「談」は「だん」と読むため、正しい組み合わせは「じょうだん」となります。選択肢には、発音が似ているものや誤読しやすいものが含まれており、漢字の音読みの知識が問われます。特に「じょうたん」や「じょだん」は一見正しそうに見えるため、注意が必要です。
    - topic: 栄養
      result:
        question: 次の漢字の正しい読み方を選びなさい。栄養
        choices:
        - えいよう
        - えいしょく
        - せいよう
        - せいしょく
        correct_answer: 1
        explanation: 「栄養（えいよう）」は、食事や健康に関する会話・文章で頻繁に使われるN3レベルの重要語彙です。「栄」は「えい」、「養」は「よう」と読むのが一般的ですが、「養」には「やしな（う）」や「いく」などの読みもあり、漢字の音読み・訓読みの区別が混乱しやすい点が学習者にとっての難しさです。また、「栄」の読みは「えい」と「さか（える）」があり、「養」は「よう」と「やしな（う）」があります。選択肢の「せいよう」は「西洋（せいよう）」の読みと混同しやすく、「せいしょく」は「生殖（せいしょく）」の読みと混同しやすい例です。また、「えいしょく」は「栄食」という熟語は一般的ではありませんが、似た音の組み合わせとして誤答例に挙げられます。このように、実際に存在する語彙や音の似た熟語と混同しやすい点が、漢字語彙問題の主な難しさとなります。
    - topic: 豊か
      result:
        question: 田中さんは都会での生活に疲れて、**自然が豊かな**町に引っ越すことにしました。次のうち、空欄に入る適切な読み方はどれですか？
        choices:
        - ほうふな
        - ゆたかな
        - とよかな
        - ほうかな
        correct_answer: 2
        explanation: 「豊か」は「ゆたか」と読み、な形容詞（形容動詞）です。「自然が豊かな町」は、自然が十分にあって恵まれている町という意味になります。選択肢1の「ほうふな」は「豊富な」で、「ほうふ」と読む別の単語です。選択肢3「とよかな」、4「ほうかな」は誤読です。「豊か」は「豊富（ほうふ）」と混同しやすいため、漢字と言葉の使い分けに注意しましょう。主なポイントと難しさ：・「豊か」はN3レベルの重要語で、意味・読み・文法（な形容詞）を正確に押さえる必要があります。・「豊富（ほうふ）」との違いが混乱しやすいので、選択肢に入れることで識別力を問うことができます。・やや長めの文脈にすることで、文脈理解力も同時に測れます。
    - topic: 筋肉
      result:
        question: 筋肉は身体を動かすために大切な組織です。この「筋肉」の正しい読み方を選びなさい。
        choices:
        - きんにく
        - きんしょく
        - すじにく
        - けんにく
        correct_answer: 1
        explanation: 「筋肉」はJLPT N3レベルでよく出る語彙です。漢字「筋（きん）」と「肉（にく）」を正しく組み合わせて読めるかがポイントです。「筋」は「すじ」とも読めますが、熟語では「きん」と読むことが多く、「筋肉」は「きんにく」となります。他の選択肢は間違った読み方や意味の異なる語です。漢字の訓読みと音読みの区別が難しい点が、この問題の主なチャレンジです。
    - topic: 答案
      result:
        question: この書類の規則をよく読んでください。次の中で「きそく」と読む漢字はどれですか？
        choices:
        - きそく
        - きぞく
        - きそん
        - きせつ
        correct_answer: 1
        explanation: 「規則（きそく）」は「ルール」という意味で、会社や学校などでよく使われるN3レベルの重要語彙です。「きぞく（貴族）」「きそん（既存）」「きせつ（季節）」は実在する日本語の読みで、音が似ているため間違えやすいですが、意味が異なります。漢字の意味と音を正確に結びつけることがポイントです。
  - subsection_title: 漢字書き取り (write_chinese)
    description: 6問。ひらがなで書かれた単語を漢字で書く。
    question_topics:
    - topic: しゅつじょう（出場）
      result:
        question: 彼は毎年、マラソン大会に しゅつじょう しています。今年も参加する予定です。
        choices:
        - 出場
        - 出張
        - 出演
        - 出席
        correct_answer: 1
        explanation: 「しゅつじょう」は「出場」と書き、スポーツ大会やコンクールなどに参加・出ることを意味します。選択肢2「出張」は仕事のために他の場所へ行くこと、選択肢3「出演」は舞台やテレビなどに出ること、選択肢4「出席」は会議や授業などに出ることを指し、いずれも「出場」とは意味が異なります。この問題では、似た読みや漢字を持つ語句が並んでいるため、文脈をよく読み取り、語彙の正確な使い分けができるかがポイントとなります。特に「出張」や「出演」など、日常でもよく使われる語と混同しやすいため、意味の違いをしっかり理解しているかを問う良問です。
    - topic: しょくひん（食品）
      result:
        question: 【指示】下線部のことばの正しい漢字表記を、1～4の中から一つ選びなさい。このスーパーでは新鮮な しょくひん がたくさん売られています。
        choices:
        - 食品
        - 食員
        - 食品物
        - 食員品
        correct_answer: 1
        explanation: 「食品」はN3レベルで頻出する語彙です。類似する漢字語が多いので注意が必要です。
    - topic: せつやく（節約）
      result:
        question: 家計のために せつやく を心がけています。正しい漢字表記を選んでください。
        choices:
        - 節約
        - 節約化
        - 節税
        - 節約費
        correct_answer: 1
        explanation: この問題のポイントは、「せつやく」という日常的な言葉に対して、正しい漢字表記を選ばせることです。 「節約（せつやく）」は「無駄遣いをしないで控えること、倹約すること」という意味で、N3の重要単語です。
          選択肢2「節約化」は実際には使われない表現で、語尾の「化」がついていることで迷わせます。 選択肢3「節税」は「税金を節約すること」で、意味が異なりますが、似た漢字なので混同しやすいです。
          選択肢4「節約費」も実際には使われない表現で、「費」がつくことで「費用」と混同しやすくなっています。 漢字の組み合わせや意味の違いに注意する必要があり、N3レベルの漢字力と語彙力を問う良問です。
    - topic: けんこう（健康）
      result:
        question: 次の文の（　　　）に入る正しい漢字を、1～4の中から一つ選びなさい。毎日運動をして、（けんこう）に気をつけています。
        choices:
        - 健考
        - 健康
        - 建康
        - 健体
        correct_answer: 2
        explanation: 「けんこう」に正しく対応する漢字は「健康」です。1の「健考」は実際には存在しない熟語ですが、読みが似ているため迷いやすい選択肢です。3の「建康」は中国の地名で、日本語では一般的に使われません。4の「健体」は「けんたい」と読み、「健康な体」という意味を持ちますが、「けんこう」とは読みません。このように、似た漢字や読み方に注意して、正しい語彙知識を身につけることが大切です。N3レベルでは、日常会話や文章で頻出する語彙の正しい漢字表記を問うことが重要です。似た漢字や意味が近い単語が多く、混乱しやすいので、文脈と漢字の意味・読み方の両方を確認しましょう。また、日本語学習者にとっては、日本語で一般的に使われる表現かどうかも判断のポイントとなります。
    - topic: しゅうにゅう（収入）
      result:
        question: 彼は毎月の しゅうにゅう が安定しています。
        choices:
        - 収入
        - 所得
        - 収納
        - 入収
        correct_answer: 1
        explanation: 「しゅうにゅう」の正しい漢字は「収入」です。他の選択肢も似た意味や読みを持つ熟語ですが、意味や使い方が異なります。「所得」は収入から経費などを差し引いた残りの金額、「収納」は物やお金をしまうこと、「入収」は実際には使われない言葉です。N3レベルでは、音読みの熟語で紛らわしいものが多いため、意味と正しい漢字の組み合わせを覚えているかが問われます。
    - topic: かんしゃ（感謝）
      result:
        question: 田中さんには、いつも助けてもらっているので、心から かんしゃ の気持ちを伝えました。
        choices:
        - 感情
        - 感謝
        - 感激
        - 感想
        correct_answer: 2
        explanation: この問題では、「かんしゃ」の正しい漢字を選ぶことが求められています。「感謝」は、誰かの親切や助けに対して「ありがたい」と思う気持ちを表す言葉です。選択肢には「感情（かんじょう）」「感激（かんげき）」「感想（かんそう）」など、意味が似ているように見える語が並んでいるため、文の内容をよく読んで正しい言葉を選ぶ必要があります。「感謝」は「ありがとう」と同じ気持ちを表す時に使います。
  - subsection_title: 語彙意味選択 (word_meaning)
    description: 11問。下線部の語の意味に最も近いものを選ぶ。
    question_topics:
    - topic: 必死
      result:
        question: 次の文の（　）に入る最も適切な漢字を、1～4の中から一つ選びなさい。彼は試験に合格するために、毎日（　）に勉強している。
        choices:
        - 必至
        - 必死
        - 必須
        - 決死
        correct_answer: 2
        explanation: この問題は、「ひっし」という同じ読みを持つ漢字語彙の中から、文脈に合う正しい漢字を選ぶ問題です。「必死」は「死ぬ覚悟で全力を尽くすこと」という意味で、日常的に「必死に勉強する」「必死に働く」など、何かに全力で取り組む様子を表します。選択肢の「必至」は「必ずそうなること」、「必須」は「必ず必要なこと」、「決死」は「死を決する覚悟」という意味で、主に戦いや非常に危険な状況で使われます。したがって、日常的な努力や勉強の場面では「必死」が最も適切です。この問題のポイントは、同音異義語の意味と使い方の違いを正確に理解しているかどうかです。「決死」は意味が似ていますが、日常の努力には通常使わないため、そこを区別できるかが重要です。
    - topic: 調子
      result:
        question: 次の文の（　）に入る正しい漢字を、下の1〜4から一つ選びなさい。最近、体のちょうしがよくないので、今日は早く寝ようと思います。
        choices:
        - 調市
        - 調子
        - 調氏
        - 調子子
        correct_answer: 2
        explanation: この問題は、「調子」の正しい漢字表記を選ばせるものです。N3レベルの学習者にとって、「調」「子」は基本的な漢字ですが、他の選択肢も見た目が似ていたり、日常で使う漢字の組み合わせなので混乱しやすいです。特に「市」「氏」などはよく目にするため、正確な語彙知識が問われます。「調子」は「物事や体の状態・具合」の意味で、日常会話でも頻出する重要語です。
    - topic: 破産
      result:
        question: 長年の赤字経営が続き、ついに会社が（はさん）することになった。
        choices:
        - 破滅
        - 破産
        - 破損
        - 破格
        correct_answer: 2
        explanation: 「破産（はさん）」は、借金などの支払いができなくなり、財産を失うことを意味します。他の選択肢は意味や読みが似ていますが、文脈に合いません。-
          「破滅（はめつ）」はすべてを失うこと- 「破損（はそん）」は壊れること- 「破格（はかく）」は基準から外れること。N3受験者は、漢字の意味と読みの混同に注意しましょう。特に「破損」「破滅」など似た語との区別がポイントです。
    - topic: 争う
      result:
        question: 彼らは優勝を目指してお互いに（あらそう）ことになった。
        choices:
        - 競走
        - 競う
        - 争う
        - 争議
        correct_answer: 2
        explanation: この文脈では「優勝を目指してお互いに（あらそう）」の意味は「互いに勝とうとして競争する」という意味になります。そのため、正しい漢字は「競う（きそう）」です。「争う（あらそう）」は「けんかする」「対立する」などのニュアンスが強く、スポーツや勝負での「競い合う」場面では「競う」が自然です。「競走（きょうそう）」は「走る競技」の意味、「争議（そうぎ）」は「もめごと」「争いごと」を指します。N3レベルでは「争う」と「競う」の意味や使い分けを正しく理解することが大切です。
    - topic: 題名
      result:
        question: この本のタイトルとして正しい言葉を選びなさい。
        choices:
        - 題目
        - 題名
        - 表題
        - 標題
        correct_answer: 2
        explanation: 「題名」（だいめい）は、本や映画、作品などのタイトルを意味します。選択肢の「題目」は主題や課題を指し、「表題」「標題」はどちらも見出しやタイトルの意味がありますが、一般的に本や作品のタイトルには「題名」が最も適切です。特に「表題」「標題」は書類や章のタイトルなど、使われる場面が異なります。似た語彙が多いため、文脈に合った正しい語を選ぶ力が求められる問題です。
    - topic: 要素
      result:
        question: この文章の（　）に入る正しい漢字を選びなさい。科学の授業で、水は酸素と水素という二つの（　）からできていると習いました。
        choices:
        - 要因
        - 要素
        - 要点
        - 要約
        correct_answer: 2
        explanation: この問題のポイントは、「ようそ」という読みから適切な漢字を選ぶことです。N3レベルの学習者にとって、「要素」「要因」「要点」「要約」など似た読み・意味の語が混乱しやすいです。「要素」は「ものごとを成り立たせている基本的な内容や成分」という意味で、この文脈に合います。他の選択肢は、原因（要因）、要点（main
          point）、要約（summary）と意味が異なるため、正しく区別できるかがポイントです。
    - topic: 湿気
      result:
        question: 梅雨の時期は、部屋の中に「しっけ（しっけ）」がたまりやすいので注意が必要です。次のうち、空気や物に含まれる水分のことを指す言葉はどれですか？
        choices:
        - 湿度
        - 湿気
        - 湿度計
        - 湿布
        correct_answer: 2
        explanation: この問題の正解は「湿気」です。「しっけ」は空気や物に含まれる水分のことを指し、梅雨の時期によく使われる語です。選択肢1「湿度（しつど）」は空気中の水分量をパーセントで表す言葉で、意味が異なります。選択肢3「湿度計（しつどけい）」は湿度を測る道具です。選択肢4「湿布（しっぷ）」は、けがや痛みに貼る薬です。音が似ている語も多く、読みと意味の区別がN3学習者には難しいポイントです。文脈から正しい語を選ぶ練習をしましょう。例文：・雨の日は湿気が多くて、髪の毛がまとまりません。・この部屋は湿気がたまりやすいので、換気をしてください。教師コメント：「湿気」は日本の生活や気候に密接な語彙であり、日常会話でも頻繁に使われます。問題文にふりがなをつけることで、学習者が漢字と読みを結びつけやすくなります。似た音の語との違いを意識しながら学習しましょう。
    - topic: 努力
      result:
        question: 彼は試験に合格するために毎日「どりょく」している。
        choices:
        - 動力
        - 努力
        - 労働
        - 勤勉
        correct_answer: 2
        explanation: この問題は、「どりょく」の正しい漢字表記を選ぶものです。- 「動力」（どうりょく）は機械などを動かす力を意味し、意味も読みも異なります。-
          「労働」（ろうどう）は仕事や働くことを指し、「どりょく」とは読みません。- 「勤勉」（きんべん）は「まじめに努力すること」を意味しますが、読みも異なります。-
          「努力」が正しい漢字表記で、「どりょく」と読みます。この問題の難しさは、似た意味や形の語が多い点にあります。N3レベルでは「努力」「労働」「勤勉」など、意味が近い語の漢字と読みの結びつきを正確に覚える必要があります。誤答選択肢も実際によく使われる語彙で、学習者の語彙力を適切に測ることができます。
    - topic: 礼儀
      result:
        question: 彼はとても（れいぎ）正しい人なので、みんなに好かれています。
        choices:
        - 礼義
        - 礼儀
        - 例儀
        - 礼技
        correct_answer: 2
        explanation: この問題は「れいぎ」の正しい漢字表記を選ぶ問題です。「礼儀」は社会的なマナーやルールを示す語で、N3レベルでよく出ます。選択肢の「義」「儀」「技」はそれぞれ意味が異なるので注意しましょう。特に「儀」と「義」は混同しやすいですが、「礼儀」が正解です。
    - topic: 資本
      result:
        question: 次の文の（　）に入る言葉として、最も適切な漢字を1つ選びなさい。新しい会社を作るためには、多くの「しほん」が必要です。
        choices:
        - 資産
        - 資本
        - 資源
        - 資金
        correct_answer: 2
        explanation: 「資本（しほん）」は、会社や事業を始めるときの元手となるお金や財産を意味します。他の選択肢も似た漢字ですが、「資産（しさん）」は持っている財産全体、「資源（しげん）」は自然や社会の中で使えるもの、「資金（しきん）」は使うためのお金のことです。それぞれ意味が違うので、文の内容に合う「資本」が正解です。
    - topic: 特徴
      result:
        question: 次の文の（　）に入る最も適切な漢字を、1～4の中から一つ選びなさい。彼の話し方には独特の（とくちょう）があるので、すぐに本人だとわかります。
        choices:
        - 特徴
        - 特長
        - 特性
        - 特質
        correct_answer: 1
        explanation: 「特徴」は、他と区別できる目立った点や性質を指します。例えば、「この鳥の特徴は赤いくちばしです」のように使います。「特長」は、他よりも優れている点を指し、「この車の特長は燃費が良いことです」といった使い方をします。「特性」は、物や人が本来持っている性質や傾向を指します（例：水の特性は100度で沸騰すること）。「特質」は、特に目立つ性質や特徴的な性格を強調する場合に使われます（例：彼の特質は忍耐強さです）。この文脈では、「独特の～がある」は「特徴」が最も適切です。特に「特徴」と「特長」の使い分けはN3レベルで重要なポイントとなります。
  - subsection_title: 同義語置き換え (synonym_substitution)
    description: 5問。文中の語句を同じ意味の語に置き換える。
    question_topics:
    - topic: 現代
      result:
        question: この小説は（現代）の若者について書かれています。
        choices:
        - 現在
        - 近代
        - 古代
        - 未来
        correct_answer: 1
        explanation: 「現代（げんだい）」は「今の時代」「現在」という意味で使われます。「現在（げんざい）」は「今」という意味で、「現代」と非常に近い意味を持つため、最も適切な選択肢です。「近代（きんだい）」は歴史的に見ると「明治時代から第二次世界大戦ごろまで」を指し、「現代」とは異なります。「古代（こだい）」は昔の時代、「未来（みらい）」はこれからの時代なので文脈に合いません。N3レベルの学習者にとっては、「現代」と「近代」を混同しやすいので、正確な意味の違いに注意しましょう。
    - topic: 種類
      result:
        question: この店にはいろいろな種類のパンがあります。この文の「種類」と最も近い意味の言葉はどれですか。
        choices:
        - 形
        - タイプ
        - 重さ
        - 時間
        correct_answer: 2
        explanation: 「種類」は、性質や特徴によって分けられたグループやカテゴリーを表す言葉です。「タイプ」も同じように、形や性質によって分けられたグループを指します。「形」は見た目、「重さ」は量、「時間」は時刻や期間を表すため、「種類」とは意味が異なります。カタカナ語の「タイプ」もN3レベルでよく使われるので、文脈から正しい意味を選ぶ必要があります。受験者にとっては、似ている言葉の中から文に合うものを選ぶ力が問われる問題です。
    - topic: 完全
      result:
        question: 次の文の「完全」と最も近い意味の言葉を選びなさい。この計画は完全に成功した。
        choices:
        - しっかり
        - すべて
        - まったく
        - かんたんに
        correct_answer: 3
        explanation: 「完全」は「欠けているところがなく、すべてがそろっていること」を意味し、「まったく」も「すべて」「完全に」という意味で使われます。N3レベルでは「しっかり（firmly）」「すべて（all）」「かんたんに（easily）」もよく出る言葉ですが、文脈的に「完全に成功した」は「まったく成功した」と言い換えられます。主な難しさは、「すべて」も「全部」という意味があるため、選択肢として迷いやすい点です。しかし、「完全に」は「まったく」「徹底的に」といった副詞がより近い意味となります。
    - topic: 冷やす
      result:
        question: 冷蔵庫でジュースを冷やしておきました。次の中から、「冷やす」の意味に最も近いものを選びなさい。
        choices:
        - あたためる
        - こおらせる
        - 冷たくする
        - さます
        correct_answer: 3
        explanation: この問題は、「冷やす」の意味を正しく理解し、似ている語彙と区別できるかどうかを問うものです。「冷やす」は、物の温度を下げて冷たくすることを表します。選択肢3「冷たくする」が最も近い意味です。選択肢4「さます」は、熱いものを適温に下げる場合に使うため、必ずしも冷たくするとは限りません。選択肢2「こおらせる」は「凍らせる」こと、選択肢1「あたためる」は反対の意味です。N3レベルでは、「冷やす」「さます」「冷たくする」などの使い分けが難しいため、意味の違いに注意しましょう。
    - topic: 自然
      result:
        question: 次の文の（　）に入る言葉と最も意味が近いものを、1～4の中から一つ選びなさい。休日には（自然）の中で散歩するのが好きです。
        choices:
        - 人工
        - 天然
        - 文化
        - 都会
        correct_answer: 2
        explanation: この文での「自然」は「人の手が加わっていない、ありのままの環境」という意味です。「天然」も「自然にできたもの、人が作ったものではない」という意味で、最も近いです。「人工」は「人が作ったもの」、「文化」は人間が作り出したものや活動、「都会」は都市のことなので、文脈から外れます。N3受験者は「自然」と「天然」の違いを正確に理解するのが難しい場合がありますが、この文脈では「自然＝天然の環境」と推測できるようにしました。「人工」「文化」「都会」は意味が異なるため、混乱しにくいですが、似た意味の語彙（天然）と対比語（人工）を含めることで、語彙力と読解力の両方を測ることができます。
  - subsection_title: 語彙使い方 (word_usage)
    description: 5問。語彙の使い方として正しい文を選ぶ。
    question_topics:
    - topic: 保存
      result:
        question: 次の文の中で、「保存」の用法として最も適切なものを一つ選びなさい。①　この美術館では、貴重な絵画を長い間きれいなまま保存しています。②　昨日のパーティーで使ったお皿を保存してから、もう一度洗いました。③　友達に手紙を保存したので、すぐに読んで返事を書きました。④　新しい靴を保存して、明日から毎日履くことにします。⑤　パソコンで作ったファイルをUSBに保存しておきました。
        choices:
        - ①　この美術館では、貴重な絵画を長い間きれいなまま保存しています。
        - ②　昨日のパーティーで使ったお皿を保存してから、もう一度洗いました。
        - ③　友達に手紙を保存したので、すぐに読んで返事を書きました。
        - ④　新しい靴を保存して、明日から毎日履くことにします。
        - ⑤　パソコンで作ったファイルをUSBに保存しておきました。
        correct_answer:
        - 1
        - 5
        explanation: 「保存」は、物やデータなどの状態を変えずに長く保つ場合に使います。①は、美術館で絵画を良い状態で保っている文脈で、最も基本的な使い方です。⑤は、IT分野で「データを保存する」という現代的な使い方で、N3レベルでも重要な例です。②や④は「保管」「しまう」が自然な表現です。③は、手紙を「保存する」という表現も場合によっては使えますが、「すぐに読む」という文脈では不自然です。
    - topic: 拍手
      result:
        question: 次の文の中で、「拍手（はくしゅ）」の使い方として最も適切なものをすべて選びなさい。
        choices:
        - 先生が教室に入ってきたとき、みんなで拍手しました。
        - 昨日の晩ご飯は、とてもおいしくて家族で拍手しました。
        - 神社でお参りするとき、まず拍手（はくしゅ）をしてからお願いごとをします。
        - 友達の誕生日に、みんなで拍手をしながら歌を歌いました。
        correct_answer:
        - 1
        - 4
        explanation: 1は、先生が入ってきたときに歓迎や感謝の気持ちを表すために「拍手」する、自然な使い方です。4は、誕生日のお祝いの場面で、みんなで歌いながら「拍手」するのもよくある使い方です。2は、「拍手」が賞賛や歓迎、祝福の場面で使われるのが一般的であり、食事が美味しかっただけで家族で拍手するのは、特別なイベント（例：料理コンテストやパーティ）を除き、日常的な日本の習慣としては自然ではありません。3は、神社で手を打つ行為は「柏手（かしわで）」と呼ばれ、神道の作法です。一般的な「拍手」とは目的や意味が異なります。日本語の文脈では「柏手」と「拍手」を区別する必要があり、ここで「拍手」を使うのは不適切です。
    - topic: 要点
      result:
        question: 次の文のうち、「要点」の使い方として最も適切なものをすべて選びなさい。
        explanation: 「要点」は「物事の中心となるところ、重要な点」という意味で、話・ニュース・スピーチなど抽象的内容の大切な部分に使います。1、2、3は正しい使い方ですが、4は「家」など物理的な対象には使いません。
        correct_answer:
        - 1
        - 2
        - 3
        choices:
        - この話の要点をまとめてください。
        - 今朝のニュースの要点をノートに書きました。
        - 彼女のスピーチは長かったが、要点はよく伝わった。
        - 家の要点をきれいに掃除してください。
    - topic: 礼儀
      result:
        question: 次の文の中で、「礼儀」の正しい使い方はどれですか。
        choices:
        - 彼は礼儀が正しいので、誰に対してもていねいに話します。
        - この料理は礼儀が強くて、少し苦い味がします。
        - 昨日の会議では、礼儀を使ってパソコンの問題を解決しました。
        - 山田さんは礼儀を集めるのが趣味で、たくさん持っています。
        correct_answer: 1
        explanation: この問題の主な難しさは、「礼儀」という語が人間関係や社会生活で守るべき行動様式やマナーを指すことを理解しているかどうかです。2～4の選択肢は、「礼儀」という語の意味から外れています（2は味に使う表現、3は道具のように使っている、4は物を集める趣味のように使っている）。「礼儀が正しい」は日本語の固定表現であり、N3レベルの学習者が正しい文脈で使えるかを問う良問です。
    - topic: 順調
      result:
        question: 次の文の中で、「順調」の使い方として最も適切なものを一つ選びなさい。
        choices:
        - 新しいプロジェクトは、今のところ順調に進んでいます。
        - 今日は順調な天気なので、ピクニックに行きましょう。
        - 彼から順調な返事をもらって、安心しました。
        - この店は順調な料理がたくさんあります。
        correct_answer: 1
        explanation: 「順調」は「物事が予定通りに、問題なく進むこと」を意味します。特に「順調に進む」「順調な経過」など、仕事や計画、進行状況に対して使われる表現です。1は「プロジェクトが順調に進む」という、もっとも一般的で自然な使い方です。2の「順調な天気」は不自然で、「天気がいい」「天候に恵まれて」などが自然です。3の「順調な返事」も不自然で、「返事が早い」「返事がスムーズ」などが適切です。4の「順調な料理」という表現は日本語として使いません。「順調」は「進む」「経過」「回復」「推移」など、進行や状態の良さを表す名詞とよく組み合わせて使います。物や天気、返事などには直接使わないので、コロケーション（よく使われる言葉の組み合わせ）に注意しましょう。
  - subsection_title: 文法穴埋め (sentence_grammar)
    description: 13問。空欄に入る最も適切な語句や助詞を選ぶ。
    question_topics:
    - topic: 店で価格を尋ねる
      result:
        question: （店で）次の会話の空欄に入る最も適切な語を、下の1～4から一つ選びなさい。客：「すみません、この商品の値段を（　　　）いただけますか。」
        choices:
        - 教えて
        - 話して
        - 聞いて
        - 書いて
        correct_answer: 1
        explanation: この文では、「～ていただけますか」という丁寧な依頼表現を使っています。「教えていただけますか」は「教える」の「て形」＋「いただけますか」で、「教えてください」よりも丁寧な表現です。「話して」や「聞いて」「書いて」は意味が合わず、不自然です。「教えて」を使うことで、店員に商品の値段を丁寧に尋ねることができます。N3レベルでは、「～ていただけますか」や「～てくれますか」などの依頼表現がよく問われます。正しい「て形」を使えるかどうかがポイントです。
    - topic: 助詞
      result:
        question: 以下の文の（　）に入る最も適切な助詞を1つ選びなさい。私は毎朝7時（　）起きて、学校に行きます。
        choices:
        - に
        - で
        - へ
        - と
        correct_answer: 1
        explanation: 「に」は特定の時刻や場所、目的地を表す格助詞として使われます。この文では「7時」という特定の時刻に起きることを表しているため、「に」が正しいです。他の選択肢について：-
          「で」は場所や手段を表します（例：教室で勉強する）。- 「へ」は方向・目的地を示します（例：学校へ行く）。- 「と」は「一緒に」などの意味で使います（例：友達と話す）。類似例文：-
          毎晩10時に寝ます。- 8時に家を出ます。このように、時刻を表す場合は必ず「に」を使うことを覚えましょう。
    - topic: キャリア目標について話す
      result:
        question: （面接練習） 面接官：「あなたのキャリア目標について教えてください。」 あなた：「はい、私のキャリア目標は、3年後にはプロジェクトリーダーとしてチームをまとめ、会社の売上に貢献できるようになることです。そのために、今は業務に必要な知識やスキルを（　　　　　）、毎日勉強しています。」
          （　　）に入る最も適切な語句を選びなさい。
        choices:
        - 身につけるようにして
        - 身につけるために
        - 身につけて
        - 身につけたあとで
        correct_answer: 2
        explanation: この文では、「目的」を表す表現が必要です。「～ために」は、「…する目的で」「…するために」という意味で、目標に向かって努力する姿勢を表します。他の選択肢は文脈に合いません。N3レベルでは、「～ために」の使い方を正確に理解し、目的を表す文を作れるかがポイントです。特に「ようにして」や「て」など、手段や継続・結果を表す表現との違いに注意が必要です。
    - topic: 場合
      result:
        question: 次の文の（　　　）に入る最も適切な語句を、1～4の中から一つ選びなさい。（会社の案内）「火事や地震が起こった（　　　）、落ち着いて行動してください。」
        choices:
        - こと
        - 場合
        - 時
        - もの
        correct_answer: 2
        explanation: 「場合」は「もし～のような状況が起きたとき」という意味で、公式な案内や指示文でよく使われます。「時」も似た意味ですが、より一般的な表現であり、「場合」の方が条件や状況を強調できます。「こと」「もの」はこの文脈には合いません。
    - topic: 程度
      result:
        question: 先生：「みなさんの日本語の（　　　）は、前よりずっと上がっていますね。」この文に入る最も適切な言葉はどれですか。
        choices:
        - 程度
        - 具合
        - 様子
        - 状態
        correct_answer: 1
        explanation: この文では、「みなさんの日本語の（　　　）は、前よりずっと上がっていますね。」という表現から、何がどのくらい上がったのかを問うています。「程度」は「レベル」や「どのくらいか」という意味で、能力や知識の高さを表すときによく使います。他の選択肢「具合」「様子」「状態」は、体調や状況、見た目などを表す言葉であり、「日本語の（　　　）が上がる」という文脈には適しません。この問題の主なチャレンジは、「程度」と他の似た意味の語彙（具合、様子、状態）との違いを理解し、正しく使い分けることです。N3レベルの学習者にとっては、抽象的な語彙の使い分けが難しいポイントとなります。
    - topic: 状態
      result:
        question: （病院にて）医者：「こんにちは。最近、体の（　　　）はどうですか。どこか痛いところはありませんか？」
        choices:
        - 具合
        - 状態
        - 事情
        - 理由
        correct_answer: 1
        explanation: 「体の具合」は、医者や家族が体調を尋ねるときによく使われる自然な表現です。日常会話や医療現場では、「体の具合が悪い」「具合はどうですか」のように使われます。「状態」も意味は通じますが、より医学的・客観的なニュアンスが強く、診断書や報告書などで使われる傾向があります。N3レベルでは、会話の場面では「具合」を優先して使うことを学びましょう。使い分け例-
          「最近、体の具合はどうですか？」（日常会話・医者が患者に）- 「患者の健康状態を記録します。」（報告・記録などやや硬い表現）この問題の主な難しさは、「具合」と「状態」の意味の違いと使い分けを理解することにあります。どちらもN3レベルで学ぶ重要な語彙ですが、場面によって適切な語を選ぶ力が求められます。
    - topic: 内容
      result:
        question: 【指示】次の会話文を読んで、（　　　）に入る最も適切な語を1つ選びなさい。（会議にて）部長：「この企画書の（　　　）を、もう一度確認してから提出してください。」
        choices:
        - 内容
        - 形式
        - 理由
        - 結果
        correct_answer: 1
        explanation: 「内容」は「中身」や「具体的な情報」という意味で、文書や企画書などの「何が書かれているか」を指します。「形式」は「書き方」や「フォーマット」、「理由」は「なぜそうしたか」、「結果」は「どうなったか」を表します。この文脈では、「企画書の内容を確認する」が最も自然です。例文：-
          レポートの内容を確認してください。- テストの形式が変わりました。
    - topic: 判断
      result:
        question: （　　　）に入る正しい言葉を選びなさい。（会議室）部長：「このプロジェクトを続けるかどうかは、今週中に（　　　）してください。」
        choices:
        - 判断して
        - 判断し
        - 判断する
        - 判断した
        correct_answer: 1
        explanation: 「〜してください」という依頼表現では、動詞の「て形」が必要です。「判断する」の「て形」は「判断して」なので、正解は1番です。他の選択肢は依頼の文型として使えません。学習者がつまずきやすいポイント：サ変動詞「判断する」の「て形」活用に注意が必要です。「て形＋ください」の依頼表現は頻出ですが、語幹や「し」「する」「した」と混同しやすいので、正しい活用を確認しましょう。補足例文：早く返事してください。よく考えてください。注意してください。
    - topic: 条件
      result:
        question: 部長：「このプロジェクトに参加したい人は、英語が話せることが（　）です。」
        choices:
        - 条件
        - 理由
        - 方法
        - 目的
        correct_answer: 1
        explanation: この文では、「英語が話せること」が「プロジェクトに参加したい人」にとって必要な事項であることを表しています。「条件」は、あることが成立するために必要な事柄を指します。他の選択肢「理由」「方法」「目的」は文脈に合いません。N3レベルの学習者にとって、「条件」という語の意味や使い方、特に「～が条件です」という表現に慣れることがポイントです。また、日常会話やビジネスの場面でよく使われる表現なので、語彙の定着が大切です。
    - topic: 関する
      result:
        question: この説明会は、留学生の生活（　　　　　）重要な情報を提供します。
        choices:
        - に関する
        - に対する
        - について
        - に関して
        correct_answer: 1
        explanation: 「～に関する＋名詞」は、「～に関連する」という意味で、公式な文書や案内などでよく使われる硬い表現です。「～について」は「～についての＋名詞」の形で使い、より口語的・柔らかい印象になります。例：留学生の生活についての説明（やや口語的）、日本の文化についての本（一般的な使い方）「に対する」は「Aに対するB」（Aに向けたB）となり、対象を示します。例：留学生に対する支援「に関して」は「に関して話す」など動詞を修飾する場合によく使われますが、名詞を直接修飾する場合は「に関する」の方が一般的です。ただし、口語では「に関しての情報」も使われます。N3レベルの学習者は「に関する」と「について」の違いでよく混乱します。特に、「に関する＋名詞」と「についての＋名詞」の使い分けがポイントです。また、「に対する」や「に関して」など似た表現も多いため、文脈に合った選択ができるかが問われます。
    - topic: 比較
      result:
        question: 次の文の（　）に入る最も適切な語句を１～４の中から一つ選びなさい。私の家の冷蔵庫は、新しい冷蔵庫（　）音が静かです。
        choices:
        - より
        - しか
        - だけ
        - でも
        correct_answer: 1
        explanation: この文は「比較」の文型を問う問題です。「AはBより～です」は「A is more ~ than B」という意味で、N3レベルで頻出の表現です。「新しい冷蔵庫より、私の家の冷蔵庫の方が音が静かです」と主語を明示すると、比較の関係がより分かりやすくなります。「より」は比較対象を示す助詞で、他の助詞（「しか」「だけ」「でも」）はこの文脈で使えません。学習者は、これらの助詞の使い分けをよく練習する必要があります。特に「より」と「だけ」などは混同しやすいので、注意しましょう。
    - topic: 場合
      result:
        question: （　）に入る最も適切な語を選びなさい。（教室にて）先生：「授業中に体調が悪くなった（　　　）、すぐに私に言ってください。」
        choices:
        - とき
        - 場合
        - もの
        - こと
        correct_answer: 2
        explanation: 「場合」は、ある条件や仮定が成立したときに、そのときどうするかを述べるときに使う表現です。「〜場合（は）」の形で、もしそのような状況になったら、という意味になります。一方、「とき」は単に特定の時間や瞬間を表しますが、「場合」は条件や仮定を強調したいときに使います。この文では、「授業中に体調が悪くなったとき」というよりも、「もしそのような状況になったら」という条件を伝えたいので、「場合」が最も適切です。
    - topic: 原因
      result:
        question: （病院にて）医者：「最近、体の調子が悪いそうですね。どんな（　）で来院されましたか。」
        choices:
        - 原因（げんいん）
        - 理由（りゆう）
        - 症状（しょうじょう）
        - 動機（どうき）
        correct_answer: 1
        explanation: この文では「どんな（　）で来院されましたか。」と、患者が病院に来たきっかけやもとになったことを尋ねています。「原因」は「物事を引き起こすもと」という意味で、体調不良や病気の「もと」になった出来事や状態を表すときに使います。「理由」は「自分の考えや選択の説明」に使うことが多く、例えば「どうしてその病院を選びましたか」という質問には合いますが、体の不調の「もと」を聞く場合には自然ではありません。「症状」は「体にどんな変化や異常があるか」を表す語で、「どんな症状がありますか」という使い方になります。「動機」は「行動を起こす気持ちや目的」を表し、主に犯罪や就職活動などの文脈で使われ、「病院に来た動機」とはあまり言いません。したがって、この文脈では「原因」が最も適切な語です。N3レベルの学習者は「原因」と「理由」の違いを混同しやすいです。また、「症状」や「動機」といった似た語彙も出題することで、より深い語彙理解と正しい使い分けが求められます。文脈をしっかり読む力と、語彙の意味の違いを意識させることがポイントです。
  - subsection_title: 文の並べ替え (sentence_sort)
    description: 5問。バラバラになった文を正しい順番に並べる。
    question_topics:
    - topic: 感謝
      result:
        question: 次の文の★の部分にくる文を、正しい順序に並べかえてください。先生に__★_、私は日本語の勉強を続けることができました。
        choices:
        - 1. 助けていただいたおかげで
        - 2. 日本語の勉強を
        - 3. 続けることができました
        - 4. 私は
        correct_answer: 1
        explanation: 「助けていただいたおかげで」は「原因・理由」を表し、主節（私は日本語の勉強を続けることができました）の前に置くのが自然です。「私は日本語の勉強を続けることができました」という主節の語順もN3レベルでよく問われます。この問題のポイントは、「おかげで」の前に感謝の対象となる行為（助けていただいた）が来て、主節（結果）が後ろに来るという日本語の自然な語順を理解することです。
    - topic: 決心
      result:
        question: 次の文の__★__に入る最も適切なものを、下の1～4から一つ選びなさい。大学を卒業したら、海外で働くことを__★__。
        choices:
        - 決心した
        - した決心
        - 決心する
        - する決心
        correct_answer: 1
        explanation: 「決心する」は「～することを決める」という意味の動詞です。「～ことを決心した」の形で「～することを決めた」となり、自然な日本語になります。2や4の語順は不自然で、3は時制が合いません（現在形）。時制や語順に注意しましょう。
    - topic: 説明
      result:
        question: 次の文の★に入る文節を、自然な順番に並べてください。先生は、__★_ ので、もっとゆっくり話してくださいと言いました。1. 説明が
          2. 少し 3. 分かりにくい 4. ときがある
        choices:
        - 1 → 2 → 3 → 4
        - 1 → 3 → 2 → 4
        - 2 → 1 → 3 → 4
        - 1 → 4 → 2 → 3
        correct_answer: 1
        explanation: この問題のポイントは、「少し」が「分かりにくい」を修飾し、「ときがある」が全体をまとめる表現となることです。「説明が少し分かりにくいときがある」で、「先生の説明が時々少し分かりにくいことがある」という意味になります。日本語では修飾語（少し）は述語（分かりにくい）の前に置くのが自然です。また、「ときがある」は前の内容全体を受ける表現なので、最後に来ます。N3レベルでは、修飾の順序や文のまとまりを意識することが重要なポイントとなります。
    - topic: 提案
      result:
        question: 下の「みんなで」「いくつか」「提案」「を」の４つの語（句）を正しい順番に並べて、文を完成させるのに最も適切なものを１～４から一つ選びなさい。
        choices:
        - みんなで　提案　いくつか　を
        - いくつか　提案　を　みんなで
        - 提案　いくつか　を　みんなで
        - みんなで　いくつか　提案　を
        correct_answer: 4
        explanation: この問題の主なポイントは、日本語における修飾語と目的語の語順です。「いくつか」は数量を表し、「提案」を修飾します。「いくつかの提案をする」という形が自然です。また、「みんなで」は動作主を表す副詞句で、文頭に置くことで「みんなが一緒に」という意味が明確になります。助詞「を」は目的語「提案」に直接つきますので、「いくつか提案を」が正しい形です。N3レベルでは、「副詞句＋数量＋名詞＋を＋動詞」という基本的な文の構造を理解しているかどうかが問われます。誤った語順では意味が通じにくく、不自然な日本語になってしまいます。この問題は、助詞の使い方、副詞句の位置、数量表現の修飾関係など、複数の文法ポイントを同時に確認できる良問です。
    - topic: 結論
      result:
        question: 次の文の（　★　）に入る最も自然な文になるように、1～4の語句を正しい順番に並べてください。このレポートの__★__、私たちは環境問題についてもっと考える必要があると分かった。1.　調査の　2.　結論として　3.　結果　4.　まとめると　A.　1→2→3→4　B.　2→1→3→4　C.　1→3→4→2　D.　2→1→4→3
        correct_answer: 2
        choices:
        - A.　1→2→3→4
        - B.　2→1→3→4
        - C.　1→3→4→2
        - D.　2→1→4→3
        explanation: この問題の主な難しさは、「結論として」という接続語句と、「調査の結果」「まとめると」の使い方・順序を正しく理解することです。「結論として、調査の結果、まとめると」とつなげることで、論理的な文脈が自然になります。N3レベルの語彙を使いましたが、接続詞や名詞の連結に注意が必要です。「結論」と「結果」の違いと、文章の論理展開を理解する力が問われる問題です。
  - subsection_title: 文法構造選択 (sentence_structure)
    description: 4問。文の構造として正しいものを選ぶ。
    question_topics:
    - topic: 原因・理由
      result:
        html_article: ありがとうございます。ご指摘いただいた点を反映し、さらに学習者の理解を深めるため、問題と解説をより丁寧に修正いたします。---###
          原因と理由の違い日本語には「原因」と「理由」という似ている言葉がありますが、使い方には違いがあります。「原因」は、何かが起こる**直接的なきっかけやもと**を表します。たとえば、「風邪をひいた原因はウイルスです」というように、悪い結果や出来事が起こる**もと**を説明するときに使います。一方、「理由」は、**行動や判断の根拠**や**背景**を表します。たとえば、「試験を受ける理由は、将来のためです」のように、なぜそうするのか、**自分の考えや事情**を説明するときに使います。「原因」は悪い結果に使うことが多いですが、「理由」は良いことにも悪いことにも使えます。もしどちらを使うか迷ったときは、「理由」を使うと安心です。---####
          次の文の（　）に入る最も適切なものを選びなさい。１　事故の（　）は、運転手の不注意でした。　１．理由　２．原因　３．目的　４．意味　２　私が日本語を勉強する（　）は、日本で働きたいからです。　１．原因　２．理由　３．方法　４．結果　３　台風が来た（　）、電車が止まりました。　１．ので　２．こと　３．とき　４．ため　４　「原因」と「理由」の違いについて（　）説明してください。　１．しっかり　２．ちゃんと　３．わかりやすく　４．すぐに---####
          正答と解説１　**２．原因**　「事故の原因」は決まった言い方で、直接的な理由やきっかけを表します。「理由」では不自然です。２　**２．理由**　「勉強する理由」は行動の根拠や動機を表すので「理由」がぴったりです。「原因」は使いません。３　**４．ため**　「台風が来たため、電車が止まりました」はN3でよく出る「〜ため」の使い方です。「ため」は書き言葉や正式な場面で使われることが多く、ニュースや説明文などで使われます。一方、「ので」も原因・理由を表しますが、より口語的で柔らかいニュアンスがあります。どちらも正しい文法ですが、ここでは公式な説明文として「ため」が最適です。４　**３．わかりやすく**　「わかりやすく説明してください」は、「説明」のしかたを最も適切に表す副詞です。「ちゃんと」「しっかり」も正しい副詞ですが、「きちんと説明する」という意味合いが強く、「説明の内容が理解しやすい」という意味では「わかりやすく」が最も自然です。---####
          教師からのポイント「原因」と「理由」の使い分けは、日本語学習者にとって混乱しやすいポイントです。「原因」は悪い結果のもと、「理由」は行動の動機や根拠という違いを例文で覚えましょう。また、「ため」と「ので」などの文型も、書き言葉・話し言葉の違いや場面による使い分けを意識すると、より自然な日本語表現が身につきます。---ご指摘いただいた点を反映し、より明確な解説と読みやすい形式に修正しました。今後もご意見を参考に、質の高い教材作成に努めてまいります。
        questions:
        - question: 事故の（　）は、運転手の不注意でした。
          choices:
          - 理由
          - 原因
          - 目的
          - 意味
          correct_answer: 2
          explanation: 「事故の原因」は決まった言い方で、直接的な理由やきっかけを表します。「理由」では不自然です。
        - question: 私が日本語を勉強する（　）は、日本で働きたいからです。
          choices:
          - 原因
          - 理由
          - 方法
          - 結果
          correct_answer: 2
          explanation: 「勉強する理由」は行動の根拠や動機を表すので「理由」がぴったりです。「原因」は使いません。
        - question: 台風が来た（　）、電車が止まりました。
          choices:
          - ので
          - こと
          - とき
          - ため
          correct_answer: 4
          explanation: 「台風が来たため、電車が止まりました」はN3でよく出る「〜ため」の使い方です。「ため」は書き言葉や正式な場面で使われることが多く、ニュースや説明文などで使われます。一方、「ので」も原因・理由を表しますが、より口語的で柔らかいニュアンスがあります。どちらも正しい文法ですが、ここでは公式な説明文として「ため」が最適です。
        - question: 「原因」と「理由」の違いについて（　）説明してください。
          choices:
          - しっかり
          - ちゃんと
          - わかりやすく
          - すぐに
          correct_answer: 3
          explanation: 「わかりやすく説明してください」は、「説明」のしかたを最も適切に表す副詞です。「ちゃんと」「しっかり」も正しい副詞ですが、「きちんと説明する」という意味合いが強く、「説明の内容が理解しやすい」という意味では「わかりやすく」が最も自然です。
    - topic: 逆接
      result:
        html_article: 承知しました。ご指摘を踏まえ、より明確で受験者にとって分かりやすいJLPT N3レベルの逆接問題を再構成します。---##
          次の文章の[A]〜[D]の空欄に入る最も適切な逆接の接続詞を、それぞれ下の選択肢から一つずつ選びなさい。---### 友達との旅行去年の春休みに、私は友達と京都に旅行に行きました。天気予報では、旅行の間ずっと晴れると言われていました。[A]京都に着いた日は、朝から雨が降っていました。雨が降ったので、予定していた観光地に行くかどうか、みんなで相談しました。[B]せっかく来たので、雨でも観光を楽しもうということになりました。私たちは傘をさして、有名なお寺や神社を回りました。[C]雨の中で見る桜はとてもきれいでした。写真を撮ると、普段よりも静かな雰囲気が伝わってきました。[D]雨の日の京都も、思っていたより楽しかったです。---###
          [A]に入る言葉を選びなさい。1. しかし 2. だから 3. それから 4. そして### [B]に入る言葉を選びなさい。1. けれども 2.
          しかし 3. でも 4. けれど### [C]に入る言葉を選びなさい。1. しかし 2. でも 3. けど 4. けれども### [D]に入る言葉を選びなさい。1.
          しかし 2. けど 3. けれど 4. でも---## 正答・解説### [A] 正解：1. しかし  解説：前の文で「晴れる」と言われていたが、実際には「雨が降った」ため、予想と異なる結果を表す逆接の「しかし」が最適です。###
          [B] 正解：1. けれども  解説：「雨が降ったので予定を相談した」→「せっかく来たので観光する」と、逆の行動を取ることを示す逆接。文中で使う「けれども」が自然です。###
          [C] 正解：2. でも  解説：「雨が降っている」という否定的な状況でも、「桜がきれいだった」と肯定的な内容を続けるため、「でも」が適切です。会話的な自然さもあります。###
          [D] 正解：3. けれど  解説：「普段と違う雨の日の京都」→「楽しかった」という予想外の感想を逆接で述べる。文末に使える「けれど」が自然です。---##
          教師からのポイント- 逆接の接続詞は、文の位置やフォーマル度によって使い分ける必要があります。- 「しかし」は文頭、「けれども」は文中、「でも」は会話的、「けれど」は文末や柔らかい印象で使われます。-
          文脈をしっかり読んで、どの逆接表現が最も自然かを考えましょう。---この形式で、JLPT N3受験者が逆接の使い分けをしっかりと練習できる内容となっています。
        questions:
        - question: '[A]に入る言葉を選びなさい。'
          choices:
          - しかし
          - だから
          - それから
          - そして
          correct_answer: 1
          explanation: 前の文で「晴れる」と言われていたが、実際には「雨が降った」ため、予想と異なる結果を表す逆接の「しかし」が最適です。
        - question: '[B]に入る言葉を選びなさい。'
          choices:
          - けれども
          - しかし
          - でも
          - けれど
          correct_answer: 1
          explanation: 「雨が降ったので予定を相談した」→「せっかく来たので観光する」と、逆の行動を取ることを示す逆接。文中で使う「けれども」が自然です。
        - question: '[C]に入る言葉を選びなさい。'
          choices:
          - しかし
          - でも
          - けど
          - けれども
          correct_answer: 2
          explanation: 「雨が降っている」という否定的な状況でも、「桜がきれいだった」と肯定的な内容を続けるため、「でも」が適切です。会話的な自然さもあります。
        - question: '[D]に入る言葉を選びなさい。'
          choices:
          - しかし
          - けど
          - けれど
          - でも
          correct_answer: 3
          explanation: 「普段と違う雨の日の京都」→「楽しかった」という予想外の感想を逆接で述べる。文末に使える「けれど」が自然です。
    - topic: 条件
      result:
        html_article: '# 日本語記事：条件について 私たちの生活には、さまざまな「条件」があります。たとえば、アルバイトを[　1　]、年齢や経験が必要な場合があります。会社に入るための条件も、学歴や資格などいろいろです。また、天気がいい日[　2　]、運動会が行われます。もし雨が降ったら、運動会は中止になります。このように、何かが起こるためには「条件」が必要なことが多いです。
          条件は、日常生活だけでなく、法律や約束にもよく使われます。たとえば、「宿題を全部終わらせたら、ゲームをしてもいい」という約束は、「宿題を終わらせること」がゲームをする[　3　]です。また、「この商品は、20歳以上の人だけが買うことができる」というのも、年齢が[　4　]です。
          このように、条件は私たちの生活の中でとても大切なものです。条件をよく考えて行動することで、トラブルを防ぐことができます。'
        questions:
        - question: 1. アルバイトを[　1　]、年齢や経験が必要な場合があります。
          choices:
          - したいとき
          - するために
          - してから
          - しないで
          correct_answer: 2
          explanation: 「アルバイトをするために、年齢や経験が必要な場合があります。」が自然です。「するために」は目的や理由を表すN3文型で、条件を説明するのに適切です。他の選択肢は文脈に合いません。
        - question: 2. 天気がいい日[　2　]、運動会が行われます。
          choices:
          - なら
          - ので
          - でも
          - ほど
          correct_answer: 1
          explanation: 「天気がいい日なら、運動会が行われます。」は仮定条件を表し、N3の「～なら」を使うのが正しいです。「ので」は理由、「でも」は逆接、「ほど」は程度を表すため不適切です。
        - question: 3. 「宿題を全部終わらせたら、ゲームをしてもいい」という約束は、「宿題を終わらせること」がゲームをする[　3　]です。
          choices:
          - 必要です
          - ことです
          - ものです
          - 条件です
          correct_answer: 4
          explanation: 「宿題を終わらせること」がゲームをする条件です、という説明です。「条件です」と明確に言うことで文意が通ります。「必要です」や「ものです」では具体性が足りません。
        - question: 4. 「この商品は、20歳以上の人だけが買うことができる」というのも、年齢が[　4　]です。
          choices:
          - 条件です
          - 必要です
          - できます
          - あります
          correct_answer: 1
          explanation: 「年齢が条件です。」が正しいです。「必要です」も近いですが、「条件です」の方がより明確にルールや制限を表します。
    - topic: 目的
      result:
        html_article: '# 目的について 私たちが何か新しいことを始めるとき、**目的**を持つことはとても大切です。目的があると、[1] 何をすればいいのかがはっきりします。たとえば、勉強する目的が「日本語を話せるようになること」なら、毎日少しずつ単語や文法を覚えることが大切だとわかります。
          目的は、[2] 目標や手段と違います。目的は「最終的に達成したいこと」、目標は「目的を達成するための小さなゴール」、手段は「目標を達成するための方法」です。たとえば、「健康になる」という目的のために、「毎日30分歩く」という目標を立てて、「朝早く起きて散歩する」という手段を選ぶことができます。
          目的があると、途中で困難があっても、[3] 続ける力が生まれます。目的を忘れてしまうと、なぜ頑張るのか分からなくなってしまいます。だから、何かを始めるときは、まず自分の目的を[4]
          明確にすることが大切です。 --- [1] 1. どこで 2. どうして 3. どのように 4. どれくらい  [2] 1. たとえば 2. しかし
          3. それとも 4. つまり  [3] 1. あきらめやすくなる 2. がんばり続けられる 3. つかれてしまう 4. いやになる  [4] 1.
          しっかり 2. はっきり 3. きちんと 4. すぐに --- ### 正解と解説 [1] **3. どのように** 「何をすればいいのかがはっきりします」という内容なので、「どのように（how）」が最も自然です。N3受験者は疑問詞の使い分けをよく練習しましょう。
          [2] **1. たとえば** 「目的は、目標や手段と違います。」の後に例を挙げるので、「たとえば」が適切です。「しかし」「それとも」「つまり」は文脈に合いません。
          [3] **2. がんばり続けられる** 「続ける力が生まれます」とあるので、「がんばり続けられる」が一番合っています。あきらめる・つかれる・いやになるは逆の意味です。
          [4] **2. はっきり** 「目的を明確にすることが大切です」に合うのは「はっきり」です。「しっかり」「きちんと」も意味が近いですが、「明確にする」と一番自然につながるのは「はっきり」です。「すぐに」は意味が違います。
          --- **教師の視点からの主な課題と指導アドバイス** この問題では、疑問詞や接続詞、副詞の微妙な使い分けがポイントです。N3学習者は、似た意味の語彙の使い分けや、文全体の流れを理解する力をつける必要があります。  **指導アドバイス:**  -
          疑問詞や副詞は例文を多く読み、実際の文脈でどのように使われているかを確認しましょう。  - 接続詞や副詞は、意味の近い言葉を比較しながら練習問題を解くと効果的です。  -
          「目的」「目標」「手段」など似た言葉は、図や表を使って整理し、具体例を挙げて理解を深めましょう。  --- 【模範解答例の改善点を反映し、選択肢の質や表記も整えました】'
        questions:
        - question: '[1] 何をすればいいのかがはっきりします。に入る最も適切な言葉はどれですか？'
          correct_answer: 3
          choices:
          - どこで
          - どうして
          - どのように
          - どれくらい
          explanation: 「何をすればいいのかがはっきりします」という内容なので、「どのように（how）」が最も自然です。N3受験者は疑問詞の使い分けをよく練習しましょう。
        - question: '[2] 目的は、目標や手段と違います。の後に続く最も適切な接続詞はどれですか？'
          correct_answer: 1
          choices:
          - たとえば
          - しかし
          - それとも
          - つまり
          explanation: 「目的は、目標や手段と違います。」の後に例を挙げるので、「たとえば」が適切です。「しかし」「それとも」「つまり」は文脈に合いません。
        - question: '[3] 途中で困難があっても、続ける力が生まれます。に入る最も適切な言葉はどれですか？'
          correct_answer: 2
          choices:
          - あきらめやすくなる
          - がんばり続けられる
          - つかれてしまう
          - いやになる
          explanation: 「続ける力が生まれます」とあるので、「がんばり続けられる」が一番合っています。あきらめる・つかれる・いやになるは逆の意味です。
        - question: '[4] まず自分の目的を明確にすることが大切です。に入る最も適切な副詞はどれですか？'
          correct_answer: 2
          choices:
          - しっかり
          - はっきり
          - きちんと
          - すぐに
          explanation: 「目的を明確にすることが大切です」に合うのは「はっきり」です。「しっかり」「きちんと」も意味が近いですが、「明確にする」と一番自然につながるのは「はっきり」です。「すぐに」は意味が違います。
- section_title: 第2部：読解
  subsections:
  - subsection_title: 短文読解 (short_passage_read)
    description: 4題。短い文章を読み、内容に関する問いに答える。
    question_topics:
    - topic: 店で価格を尋ねる
      result:
        question: このメールで、山田様は何を知ることができましたか。
        choices:
        - 冷蔵庫の価格と配送料について
        - 冷蔵庫の在庫状況について
        - 冷蔵庫の色について
        - 冷蔵庫のサイズについて
        correct_answer: 1
        explanation: この問題では、ビジネスメールの内容から必要な情報を正確に読み取る力が求められます。N3レベルの学習者にとって、「価格」「配送料」「ご案内」などの語彙や敬語表現の理解がポイントです。選択肢には本文にない情報（在庫状況・色・サイズ）が含まれており、正しく内容を把握できているか判断できます。メール形式や敬称も自然で、指示文も加わっているため、受験者が迷わず解答できる構成になっています。
    - topic: レストランで食べ物を注文する
      result:
        question: 山本さんは、レストランでどのように食事を楽しみましたか。
        choices:
        - 自分の料理だけを食べた
        - 田中さんの料理も少し食べた
        - デザートをたくさん食べた
        - 田中さんの注文を手伝った
        correct_answer: 2
        explanation: この問題の難しさは、メール本文から「田中さんが選んだスープも少しもらえて、うれしかったです。」という部分を正しく理解し、「自分の注文以外も食べた」ことを読み取る点にあります。N3レベルでは「もらえて」「うれしかった」などの表現や、「自分の料理だけ」や「デザートをたくさん」などの選択肢の違いに注意することが必要です。選択肢を丁寧に比較し、本文と合致する内容を選ぶ力が問われます。
    - topic: 道を尋ねる
      result:
        question: 次のメールを読んで、後の問いに答えなさい。これは、山田さんが友人のスミスさんに送ったメールです。スミスさんこんにちは。メールをありがとう。東京に来る日は決まりましたか。もしよかったら、東京駅まで迎えに行きます。東京駅はとても広いので、待ち合わせの場所を決めておきましょう。丸の内北口の改札の前でどうですか。駅で道に迷ったら、近くの駅員さんに「丸の内北口はどこですか」と聞いてください。当日は、私の携帯に電話してもいいですよ。会えるのを楽しみにしています。山田このメールで、山田さんはスミスさんにどんなアドバイスをしていますか。最も適切なものを一つ選びなさい。
        choices:
        - 東京駅で迷ったら、駅員に聞くように言っている。
        - 東京駅に着いたら、すぐに電話するように言っている。
        - 東京駅の南口で待ち合わせるように言っている。
        - 東京駅の近くのホテルに泊まるように言っている。
        correct_answer: 1
        explanation: この問題は、メール本文の内容を正確に読み取り、「道を尋ねる」というN3レベルの重要表現を理解できているかを問うものです。本文中に「駅で道に迷ったら、近くの駅員さんに『丸の内北口はどこですか』と聞いてください。」と明記されています。したがって、1が正解です。他の選択肢は、本文に直接書かれていない内容や、本文と異なる情報です。2は「すぐに電話」とは書かれておらず、3は「南口」ではなく「北口」、4はホテルについての記述がありません。誤答も本文の内容に近い表現を使うことで、受験者の本文理解力をより深く測ることができます。この問題の主なチャレンジは、「迷ったとき」「駅員に聞く」「待ち合わせ場所」といった日常的な日本語のやりとりを正確に理解し、適切な選択肢を選ぶ力です。N3レベルの語彙と文法がしっかり使われており、実用的な内容となっています。
    - topic: 週末の予定について話す
      result:
        question: 田中さんは、日曜日に何をする予定ですか。
        choices:
        - 家で本を読む
        - スーパーで買い物をする
        - 友達と映画を見に行く
        - 家族とカフェに行く
        correct_answer: 3
        explanation: この問題のポイントは、日付と予定を正確に読み取る力です。N3レベルでは、「〜つもりです」「〜予定です」など未来の表現や、日付ごとの行動を区別する読解力が求められます。「日曜日は、友達と映画を見に行く予定です。」という文を正しく理解できるかがカギです。選択肢の中には土曜日の予定や、本文にない内容が含まれているので、惑わされずに本文から直接答えを見つけることが大切です。
  - subsection_title: 中文章読解 (midsize_passage_read)
    description: 2題。中程度の長さの文章を読み、主旨や詳細情報について答える。
    question_topics:
    - topic: 趣味について話す
      result:
        question: この文章を書いた人は、旅行についてどのように考えていますか。
        choices:
        - 旅行は計画が大変なので、あまり好きではない。
        - 旅行を通じて新しい発見や経験ができるので楽しいと思っている。
        - 旅行は友達と行くのが一番大切だと思っている。
        - 旅行は国内だけで十分だと思っている。
        correct_answer: 2
        explanation: 本文では、「その土地の文化や食べ物を知ることができて、とても楽しいです」や、「旅行を通して、自分の世界が広がったと感じています」と書かれており、旅行を通じて新しい発見や経験ができることを楽しいと感じていることが読み取れます。選択肢1や4は本文と反対の内容であり、3も「友達と行くこともありますが、一人で…」とあるため、友達と行くことだけが大切とは考えていません。N3学習者にとっては、筆者の気持ちや考えを読み取る力、キーワード（「楽しい」「広がった」など）を正確に把握する力が問われる点が難しいポイントです。
    - topic: 家族について話す
      result:
        question: 家族について書かれているこの文章から、正しいものを一つ選びなさい。
        choices:
        - 家族は四人で、姉が二人いる。
        - 父は家で家事をしている。
        - 家族は毎週日曜日に一緒に夕食を食べる。
        - 弟は高校生で、バスケが好きだ。
        correct_answer: 3
        explanation: この問題の主なチャレンジは、家族構成や家族の活動、職業、趣味など、N3レベルの基礎的な語彙と文法を正しく読み取ることです。選択肢の中には、本文の内容と少し違う情報（例えば人数や職業、趣味）が混ざっているため、細かい情報を正確に把握する読解力が求められます。特に家族構成や日常の習慣に関する記述をしっかり確認することがポイントです。
  - subsection_title: 長文読解 (long_passage_read)
    description: 1題。やや長い文章を読み、全体の内容や筆者の意図を理解する。
    question_topics:
    - topic: 旅行の計画について話す
      result:
        html_article: 【次の文章を読んで、質問に答えなさい】田中さんは、来月友達と大阪に旅行に行く予定です。先週、田中さんは旅行の計画を立てるために、友達と集まりました。まず、みんなでどこに行きたいかを話し合いました。大阪城やユニバーサル・スタジオ・ジャパン、道頓堀など、人気の観光地がたくさんあります。田中さんは、歴史が好きなので、大阪城に一番行きたいと言いました。友達の一人は、テーマパークが好きなので、ユニバーサル・スタジオ・ジャパンに行きたいと言いました。みんなで相談して、二日目は大阪城、三日目はユニバーサル・スタジオ・ジャパンに行くことに決めました。ホテルは駅の近くに予約しました。駅から観光地まで電車で行けるので、とても便利です。また、旅行中に美味しいたこ焼きを食べることも楽しみにしています。田中さんたちは、旅行の日が来るのを楽しみにしています。
        questions:
        - question: 田中さんは旅行の計画を立てるために、何をしましたか。
          explanation: 主語の省略と「集まる」「計画を立てる」の文脈理解が必要。N3では複数人の行動や目的を正確に読み取ることが大切です。
          correct_answer: 1
          choices:
          - 友達と集まって、旅行の計画を立てました。
        - question: 田中さんたちは大阪でどんな観光地に行くことにしましたか。
          explanation: 複数の観光地が出てくるため、情報を整理して答える必要があります。N3では「相談して決めた」などの文型もポイント。
          correct_answer: 1
          choices:
          - 大阪城とユニバーサル・スタジオ・ジャパンに行くことにしました。
        - question: 田中さんたちがホテルを予約した場所はどこですか。
          explanation: 「場所＋に予約する」「駅の近く」など、場所を表す表現に注意しましょう。
          correct_answer: 1
          choices:
          - 駅の近くにホテルを予約しました。
        - question: 田中さんはなぜ大阪城に行きたいと言いましたか。
          explanation: 「ので」や「から」による理由説明を読み取る力が問われます。
          correct_answer: 1
          choices:
          - 歴史が好きだからです。
        - question: みんなはどのようにして行く場所を決めましたか。
          explanation: 「相談して決める」という表現を正しく理解できるかが重要です。
          correct_answer: 1
          choices:
          - みんなで相談して決めました。
        - question: 田中さんたちが旅行で楽しみにしている食べ物はどれですか。
          explanation: 旅行中の楽しみや具体的な名詞の読み取り。N3では本文中の具体的な情報を正確に選ぶ力が必要です。
          correct_answer: 2
          choices:
          - お好み焼き
          - たこ焼き
          - ラーメン
          - カレー
  - subsection_title: 情報検索 (info_retrieval)
    description: 1題。案内や掲示などの情報から必要な情報を探し出す。
    question_topics:
    - topic: 電車の切符を買う
      result:
        html_article: <h3>電車の切符の買い方と案内</h3><p>次の表と説明を読んで、後の質問に答えなさい。</p><table border="1"
          cellspacing="0" cellpadding="4" style="border-collapse:collapse;width:100%;text-align:center;"><tr
          style="background-color:#f0f0f0;"><th>購入場所</th><th>利用可能時間</th><th>支払い方法</th><th>できること</th></tr><tr><td>自動券売機</td><td>5:00～23:30</td><td>現金・ICカード</td><td>きっぷ購入、ICカードチャージ</td></tr><tr><td>窓口</td><td>7:00～20:00</td><td>現金・クレジットカード</td><td>新幹線・特急券も購入可</td></tr><tr><td>インターネット予約</td><td>24時間</td><td>クレジットカード</td><td>会員登録が必要、きっぷは駅で受け取り</td></tr><tr><td>旅行会社</td><td>10:00～18:00</td><td>現金</td><td>JRきっぷのみ、手数料なし</td></tr></table><ul><li>自動券売機は駅の入り口近くにあります。画面の案内に従って、行き先や人数を選び、お金を入れるときっぷが出ます。</li><li>ICカード（Suica、PASMOなど）は券売機でチャージできます。改札でタッチして使います。</li><li>特急や新幹線に乗る場合は、乗車券のほかに特急券や指定席券が必要です。窓口や高機能券売機、インターネットで購入できます。</li><li>インターネットで予約したきっぷは、出発当日または7日前から駅の指定席券売機で受け取ります。</li><li>旅行会社では、JRきっぷを手数料なしで購入できますが、店舗によっては取り扱いがない場合もあります。</li></ul><hr><p><b>【指示】</b>
          次の質問に最も合うものを一つずつ選びなさい。</p>
        questions:
        - question: 駅でICカードにお金を入れたいとき、どこでできますか。
          explanation: 自動券売機は駅の入り口近くにあり、ICカードのチャージもできます。
          correct_answer: 2
          choices:
          - ア　窓口
          - イ　自動券売機
          - ウ　インターネット予約
          - エ　旅行会社
        - question: インターネットで切符を予約した場合、どのように受け取りますか。
          explanation: インターネットで予約したきっぷは、出発当日または7日前から駅の指定席券売機で受け取ります。
          correct_answer: 2
          choices:
          - ア　家に郵送してもらう
          - イ　駅の指定席券売機で受け取る
          - ウ　改札で駅員から受け取る
          - エ　旅行会社のカウンターで受け取る
        - question: 窓口でできないことはどれですか。
          explanation: 窓口ではICカードへのチャージはできません。
          correct_answer: 2
          choices:
          - ア　新幹線の切符を買う
          - イ　ICカードにチャージする
          - ウ　現金で支払う
          - エ　クレジットカードで支払う
        - question: 旅行会社で切符を買うときに、正しいことはどれですか。
          explanation: 旅行会社ではJRきっぷのみ手数料なしで購入でき、支払いは現金のみです。
          correct_answer: 2
          choices:
          - ア　手数料がかかる
          - イ　現金で支払う
          - ウ　ICカードで支払う
          - エ　クレジットカードで支払う
        - question: 特急に乗りたいとき、必要なものは何ですか。
          explanation: 特急に乗る場合は乗車券と特急券の両方が必要です。
          correct_answer: 3
          choices:
          - ア　乗車券だけ
          - イ　特急券だけ
          - ウ　乗車券と特急券
          - エ　ICカードだけ
- section_title: 第3部：聴解
  subsections:
  - subsection_title: 話題理解 (topic_understanding)
    description: 6問。流れる会話や説明から話題を把握する。
    question_topics:
    - topic: お気に入りのレストランについて話す
      result:
        questions:
          question: この後、ユキさんは何をしたいと思っていますか。
          choices:
          - 新しいラーメン屋に行く
          - 森のカフェでチョコレートケーキを食べる
          - ケンさんの家に行く
          - フルーツタルトを作る
          correct_answer: 2
          explanation: この問題の難しさは、「お気に入り」「静か」「ケーキ」などN3レベルの話題語彙を聞き分け、会話の流れからユキさんの次の希望行動（＝お気に入りのカフェでチョコレートケーキを食べたい）を推測する点です。選択肢には似た内容（ケーキやフルーツタルト）が含まれているため、会話の具体的な内容を正確に聞き取る力が問われます。
        conversation:
        - gender: female
          context: ユキ：ケンくん、最近どこか美味しいお店に行った？
        - gender: male
          context: ケン：うーん、先週、駅前のラーメン屋に行ったよ。でも、ユキちゃんのお気に入りのお店ってどこ？
        - gender: female
          context: ユキ：私は「森のカフェ」が一番好きだよ。静かで、ケーキがとても美味しいの。
        - gender: male
          context: ケン：へえ、どんなケーキがあるの？
        - gender: female
          context: ユキ：チョコレートケーキや、季節のフルーツタルトが人気だよ。コーヒーも香りが良くて、落ち着くんだ。
        - gender: male
          context: ケン：いいね。今度一緒に行かない？
        - gender: female
          context: ユキ：うん、ぜひ行こう！私はまたチョコレートケーキが食べたいな。
    - topic: 健康とフィットネスについて話す
      result:
        questions:
          question: 友人Aはこの後、何をするつもりですか。
          explanation: この問題は、会話の最後で「じゃあ、私も明日から少し運動してみようかな」と友人Aが自分の行動を決めている点を聞き取る必要があります。会話の流れを理解し、実際に何をするつもりかを選ぶ力が求められます。選択肢1や2も会話に出てきますが、「明日から運動」と具体的に述べているのが3番です。N3レベルの語彙と表現を使い、会話の要点を聞き取る力を測る良問です。
          correct_answer: 3
          choices:
          - 毎朝ジョギングを始める
          - 食事のバランスを考える
          - 明日から運動をしてみる
          - プロテインを毎日飲む
        conversation:
        - gender: female
          context: 健康とフィットネスについて、友達同士で話しています。 友人A：最近、ちょっと太ってきた気がするんだ。健康診断でも、運動不足って言われちゃったし。
            友人B：そうなんだ。私は毎朝ジョギングしてるよ。最初は大変だったけど、今はすっかり習慣になったよ。 友人A：すごいね！でも、朝はなかなか起きられなくて…。運動以外で何か気をつけてることある？
            友人B：うん、食事もバランスよくするようにしてる。野菜を多めに食べたり、夜遅くに食べないようにしたりね。 友人A：なるほど。ダイエットって、食事も大事なんだね。プロテインとかも飲んでる？
            友人B：筋トレの後はたまに飲むよ。でも、無理なダイエットは体に良くないから、健康的に続けるのが一番だと思う。 友人A：うん、ありがとう。じゃあ、私も明日から少し運動してみようかな。
    - topic: 最近の映画について話す
      result:
        questions:
          question: この会話を読んで、男の人がこの後一番したいことに近いものを一つ選びなさい。
          explanation: この問題のポイントは、男の人の発言「たまにはそういう映画も見てみたいな」と、女の人の「今度一緒に見に行かない？」という誘いに「ぜひ行こう！」と答えている部分です。つまり、男の人は感動する映画（「青い空の約束」）を見に行くことに興味を持っています。N3レベルの語彙（映画館、感動、約束、俳優、演技、アクションなど）や表現（〜してみたい、〜ばかり、もしよかったら、ぜひ〜しよう）を適切に使用しています。選択肢には内容に関連するものや紛らわしいもの（アクション映画、音楽など）を含め、正しく会話内容を理解できているかを問う形式です。
          correct_answer: 3
          choices:
          - アクション映画を見る
          - 音楽を聞きに行く
          - 感動する映画を見に行く
          - 高校に行く
        conversation:
        - gender: male
          context: 佐藤さん、最近映画館に行った？
        - gender: female
          context: うん、先週「青い空の約束」っていう映画を見たよ。すごく感動したんだ。
        - gender: male
          context: その映画、気になってたんだ。どんな話？
        - gender: female
          context: 高校生の男女が卒業前に大切な約束をして、それを守るためにがんばる話だよ。音楽もきれいだったし、俳優さんの演技もよかった。
        - gender: male
          context: へえ、いいね。僕は最近アクション映画ばかり見てたから、たまにはそういう映画も見てみたいな。
        - gender: female
          context: おすすめだよ。もしよかったら、今度一緒に見に行かない？
        - gender: male
          context: うん、ぜひ行こう！
    - topic: 支払い方法について話す
      result:
        questions:
          question: この後、お客さんはまず何をしますか。最も適切なものを一つ選びなさい。
          explanation: この問題は、支払いの流れや店員とのやりとりを正確に理解する力が求められます。N3レベルの語彙（「ICカード」「タッチ」「ポイントカード」「お返しします」など）を使い、実際の日本の買い物場面を再現しています。選択肢には、会話の流れから外れるもの（現金で支払う、クレジットカードを渡す、店員に質問する）を含めることで、細部まで聞き取る力を測ります。正解を選ぶには、最後の店員の発言「レシートとポイントカードもお返しします。」に注目し、次にお客さんが何をするかを推測する必要があります。
          correct_answer: 3
          choices:
          - 現金で支払う
          - クレジットカードを渡す
          - レシートとポイントカードを受け取る
          - 店員に支払い方法について質問する
        conversation:
        - gender: male
          context: スーパーマーケットで、店員とお客さんが支払い方法について話しています。
        - gender: male
          context: 店員：お買い上げありがとうございます。お支払いはどの方法にされますか。
        - gender: female
          context: 客：クレジットカードは使えますか。
        - gender: male
          context: 店員：はい、ご利用いただけます。ICカードやスマートフォンでの支払いも可能です。
        - gender: female
          context: 客：じゃあ、ICカードでお願いします。
        - gender: male
          context: 店員：かしこまりました。こちらにカードをタッチしてください。
        - gender: female
          context: 客：（カードをタッチする）できました。
        - gender: male
          context: 店員：ありがとうございます。レシートとポイントカードもお返しします。
        - gender: female
          context: 客：ありがとうございました。
    - topic: 家事の分担について話す
      result:
        conversation:
        - gender: female
          context: ねえ、最近ちょっと家事が大変になってきたと思わない？私も仕事が忙しくなってきたし、もう少し分担を見直したいんだけど。
        - gender: male
          context: そうだね。僕もできるだけ手伝ってるつもりだけど、何か足りないことがあった？
        - gender: female
          context: ううん、ありがとう。でも、洗濯と掃除は今ほとんど私がやってるでしょ？洗濯は週に２回ぐらい、あなたにお願いできないかな。
        - gender: male
          context: うん、わかった。じゃあ、洗濯は僕がやるよ。掃除も週末ならできると思う。
        - gender: female
          context: ありがとう。あと、夕食の準備はどうする？
        - gender: male
          context: 平日は帰りが遅いから難しいけど、土日は僕が作るよ。
        - gender: female
          context: 助かる！じゃあ、今週末からお願いしてもいい？
        - gender: male
          context: もちろん。何かレシピを考えておくね。
        questions:
          question: この後、夫はまず何をしますか。
          choices:
          - 洗濯をする日を決める
          - 掃除をする時間を決める
          - 土日の夕食のメニューを考える
          - 妻に掃除のやり方を聞く
          correct_answer: 1
          explanation: この会話では、妻が洗濯を週に2回夫にお願いし、夫が「じゃあ、洗濯は僕がやるよ」と約束しています。直後に必要な行動は、いつ洗濯をするか日を決めることです。選択肢2と4も家事分担に関係しますが、掃除は「週末ならできる」と話しているだけで、すぐに時間を決める流れではありません。また、選択肢3の夕食メニューについては「考えておくね」と言っていますが、まず優先されるのは妻の要望である洗濯の分担です。受験者は会話の流れと約束された内容を正確に把握する必要があり、N3レベルの聞き取り力と実生活に基づく判断力が問われます。
    - topic: 公共施設の利用方法について話す
      result:
        questions:
          question: 女の人はこれから何をしますか。
          choices:
          - 会議室の掃除をする
          - 利用申し込みの紙を記入する
          - 会議室の利用料金をはらう
          - 会議室の場所を案内する
          correct_answer: 2
          explanation: この問題は、会話の流れを正確に理解し、次に行うべき行動を特定するリスニング力を測るものです。女の人は「申し込みを書きます」と最後に言っており、これから申し込み用紙に記入することが明らかです。選択肢2が正解となります。N3レベルの語彙と表現を使い、公共施設の利用手順という日常的な場面を設定しているので、実用的な日本語運用能力が問われます。設問と選択肢の整合性も保たれており、混乱が生じにくい良問です。
        conversation:
        - gender: female
          context: 女：こんにちは。公民館の会議室を使いたいのですが、どうすればいいですか？
        - gender: male
          context: 男：こんにちは。会議室の利用は、まず受付で申し込みが必要です。ご利用は何名様ですか？
        - gender: female
          context: 女：１０人くらいです。今日の午後、使いたいのですが、空いていますか？
        - gender: male
          context: 男：少々お待ちください。…はい、午後２時から４時まで空いています。
        - gender: female
          context: 女：ありがとうございます。申し込みはこの紙に書けばいいですか？
        - gender: male
          context: 男：はい、お名前とご連絡先、それから利用目的を記入してください。あと、利用後は机や椅子を元の場所に戻して、ゴミはきちんと持ち帰ってください。
        - gender: female
          context: 女：わかりました。利用料はどこで払いますか？
        - gender: male
          context: 男：受付でお願いします。現金かカードが使えます。
        - gender: female
          context: 女：ありがとうございます。では、申し込みを書きます。
  - subsection_title: 要点理解 (keypoint_understanding)
    description: 6問。会話の中の重要な情報や要点を聞き取る。
    question_topics:
    - topic: 仕事のプロジェクトについて話す
      result:
        questions:
          question: 女の人がまず最初にすることは何ですか。
          choices:
          - 新しいメンバーを探すこと
          - 業務の問題点を整理すること
          - 改善案をまとめること
          - 結果を報告すること
          correct_answer: 2
          explanation: この問題はN3レベルの語彙と文法を用いています。会話の中で「まずは現状の問題点を整理してから、改善案を考える予定」と述べられている部分を正確に理解できるかがポイントです。他の選択肢も会話内で言及されていますが、時系列や優先順位を正しく把握する力が求められます。選択肢の内容も混同しやすいため、注意深く聞き取る必要があります。
        conversation:
        - gender: male
          context: 田中さん、今月から新しいプロジェクトが始まりますね。担当のメンバーはもう決まりましたか。
        - gender: female
          context: はい、昨日、課長と相談して決めました。山本さんと佐藤さん、それから私の3人です。
        - gender: male
          context: そうなんですね。プロジェクトの目的は、今までの業務をもっと効率的にすることでしたよね。
        - gender: female
          context: はい。まずは現状の問題点を整理してから、改善案を考える予定です。来週、最初のミーティングをします。
        - gender: male
          context: スケジュールはどうなっていますか。
        - gender: female
          context: 今月中に現状の調査を終えて、来月から改善案をまとめていきます。最終的には、3か月後に結果を報告することになっています。
        - gender: male
          context: 3か月後ですか。けっこう短い期間ですね。
        - gender: female
          context: そうですね。でも、チームで協力すればきっとできると思います。もし何かアドバイスがあれば、ぜひ教えてください。
        - gender: male
          context: わかりました。何か手伝えることがあったら言ってください。
        - gender: female
          context: ありがとうございます。がんばります。
    - topic: おすすめを尋ねる
      result:
        questions:
          question: 男の人は女の人にどこをすすめられましたか。
          explanation: この問題のポイントは、「箱根はどう？」という女の人の発言が、男の人へのおすすめであることを聞き取ることです。会話の中で温泉や美味しいもの、景色などの希望条件が出てきますが、「それなら、箱根はどう？」という提案が正答の根拠です。また、「特急で行ける」「温泉」「黒たまご」などの情報もヒントになります。N3レベルでは、会話の流れからどの場所がすすめられたのかを正確に理解する力が求められるため、設問として適切です。固有名詞（箱根、黒たまごなど）は文脈で理解できるよう工夫しています。
          correct_answer: 2
          choices:
          - 京都
          - 箱根
          - 沖縄
          - 北海道
        conversation:
        - gender: male
          context: 来週、大学の友だちと旅行に行くんだけど、まだ行き先が決まっていないんだ。どこかおすすめの場所、ある？
        - gender: female
          context: そうだね。どんなところが好き？自然が多い場所とか、温泉とか、歴史のある町とか。
        - gender: male
          context: みんなでゆっくりできるところがいいな。できれば温泉も入りたいし、美味しいものも食べたい。
        - gender: female
          context: それなら、箱根はどう？温泉も有名だし、景色もきれいで美術館もあるよ。それに、名物の黒たまごも食べられるんだ。
        - gender: male
          context: いいね！箱根って、電車で行けるの？
        - gender: female
          context: うん、新宿から特急で1時間半くらいで行けるよ。駅からバスやケーブルカーもあって便利だよ。
        - gender: male
          context: ありがとう！みんなにも話してみる。他にもおすすめがあったら教えてね。
        - gender: female
          context: うん、思いついたらまた連絡するね。
    - topic: 交通状況について話す
      result:
        questions:
          question: 男の人は、なぜ会社に着くのが遅くなったと言っていますか。
          choices:
          - 交通事故があったから
          - 電車が少し遅れたから
          - 工事で道路が一方通行になっていたから
          - 朝寝坊したから
          correct_answer: 3
          explanation: 正答の根拠となる会話文は「工事中の道路があって、そこが一方通行になっていたから、みんな迂回しなきゃいけなかったんだよ。」です。事故はなかったと明言されており、電車の遅れについては女の人の話です。学習者は、会話の流れから正しい理由を聞き分け、他の選択肢に惑わされないよう注意が必要です。N3レベルでは、話題に出た情報と実際の理由を区別する聴解力が問われます。
        conversation:
        - gender: male
          context: 今日は朝から道路がすごく混んでいたよ。いつもより30分も早く家を出たのに、会社に着いたのはぎりぎりだったんだ。
        - gender: female
          context: そうなんだ。何か事故でもあったの？
        - gender: male
          context: ううん、事故はなかったけど、工事中の道路があって、そこが一方通行になっていたから、みんな迂回しなきゃいけなかったんだよ。
        - gender: female
          context: なるほど。それで渋滞ができていたんだね。私は電車で来たけど、電車も少し遅れていたよ。
        - gender: male
          context: 最近、通勤時間帯はどこも混んでいるね。明日はもっと早く出た方がいいかな。
        - gender: female
          context: そうだね。道路交通情報を朝チェックしてから出かけるといいかも。
        - gender: male
          context: うん、そうするよ。ありがとう。
    - topic: ショッピング体験を説明する
      result:
        questions:
          question: この会話からわかる「新しいショッピング体験」の特徴は何ですか。
          explanation: この問題は、会話全体から新しいショッピング体験の主な特徴を聞き取る力を問います。選択肢1が正解で、会話では「商品をためす」「店員が説明する」「安心して買い物ができる」など複数の特徴が述べられています。選択肢2はネットショッピングの特徴であり、会話内容と合いません。選択肢3はスペースについてのみを特徴としていますが、会話では他にも多くの特徴が語られているため不正解です。選択肢4は店員と話さない買い物ですが、会話では「店員と直接話すことの良さ」が強調されています。受験者は会話全体を聞き取り、要点をまとめて正しい選択肢を選ぶ必要があり、情報の取捨選択がポイントとなります。
          correct_answer: 1
          choices:
          - 商品をためしたり、店員と話しながら買い物ができること
          - すべての商品がネットでしか買えないこと
          - お店にコーヒーを飲みながら買い物ができるスペースがあることだけが特徴であること
          - 店員と話さずに買い物をすること
        conversation:
        - gender: male
          context: 最近、いろいろなお店で新しいショッピング体験ができるようになってきましたね。
        - gender: female
          context: そうですね。たとえば、実際に商品を手に取ってためすことができたり、店員さんが商品の使い方をていねいに説明してくれたりします。
        - gender: male
          context: 私も先週、駅前の新しい雑貨店に行ってみました。そこで気になった商品をためしてみたら、店員さんが「この商品はこうやって使うと便利ですよ」と教えてくれました。
        - gender: female
          context: それはいいですね。ネットショッピングだと、実際に商品を見たり、さわったりすることができませんから。
        - gender: male
          context: はい。あと、そのお店では、コーヒーを飲みながらゆっくり買い物ができるスペースもありました。
        - gender: female
          context: それは楽しそうですね。買い物がもっと楽しくなりそうです。
        - gender: male
          context: はい。やっぱり店員さんと直接話すことで、新しい発見もありますし、安心して買い物ができますね。
    - topic: ペットについて話す
      result:
        questions:
          question: 男の人はどうして猫を飼い始めましたか。
          choices:
          - 犬が好きだから
          - アパートで犬が飼えなかったから
          - 友だちにすすめられたから
          - 猫を偶然見つけたから
          correct_answer: 2
          explanation: この問題のポイントは、「犬を飼いたかったが、アパートが犬禁止で猫なら大丈夫だった」という理由を正確に聞き取ることです。N3レベルの語彙・文法で構成されていますが、「～なら大丈夫」「大家さんに言われて」などの表現に注意が必要です。選択肢には内容を混乱させやすいもの（1や4）が含まれており、正確なリスニング理解力が問われます。
        conversation:
        - gender: female
          context: 最近、猫を飼い始めたって本当？
        - gender: male
          context: うん、本当だよ。実は最初は犬を飼いたかったんだけど、このアパートは犬が飼えなくてね。大家さんに猫なら大丈夫だと言われて、猫を飼うことにしたんだ。
        - gender: female
          context: どこで猫を見つけたの？
        - gender: male
          context: 近くの動物保護施設で出会ったよ。小さくて、おとなしい子だったから、すぐに気に入ったんだ。
        - gender: female
          context: 名前はもう決めたの？
        - gender: male
          context: うん、「ミケ」って名前にしたよ。三毛猫だからね。
        - gender: female
          context: かわいい名前だね。猫を飼い始めて、大変なことはあった？
        - gender: male
          context: 最初は夜中に鳴かれて寝不足になったけど、今は慣れてきたよ。家に帰るとミケが待っててくれるから、毎日が楽しいんだ。
        - gender: female
          context: いいね。私もいつか猫を飼ってみたいな。
    - topic: 新しいスキルを学ぶ計画について話す
      result:
        questions:
          question: 次の会話を聞いて、話の目的として最も適切なものを選びなさい。
          choices:
          - 新しいスキルを学ぶための目標を決めるため
          - 学習の計画を見直すため
          - 友だちと一緒に勉強するため
          - 新しい本を買うため
          correct_answer: 2
          explanation: この問題では、会話の中で「計画を見直す」という目的が明確に語られるかを聞き取る必要があります。N3レベルの語彙や表現を正しく理解し、話し手の意図に注目することがポイントです。選択肢の中で似た内容があるため、計画の見直しが主題であることを正確に判断できるかが難しさとなります。
        conversation:
        - gender: male
          context: 次の会話を聞いて、話の目的として最も適切なものを選びなさい。1. 新しいスキルを学ぶための目標を決めるため 2. 学習の計画を見直すため
            3. 友だちと一緒に勉強するため 4. 新しい本を買うため
  - subsection_title: 概要理解 (summary_understanding)
    description: 3問。聴いた内容の大意やまとめを問う。
    question_topics:
    - topic: 課題と解決策について話す
      result:
        conversation:
        - gender: male
          context: 部長：最近、会社の中でどんな課題があると思いますか。
        - gender: female
          context: 社員：はい。私は、部門間のコミュニケーションがあまりうまくいっていないことが課題だと思います。例えば、営業部と開発部が情報を十分に共有できていません。
        - gender: male
          context: 部長：なるほど。たしかに、連絡ミスで仕事が遅れることがありましたね。
        - gender: female
          context: 社員：はい。ですから、定期的に部門ごとのミーティングを開いて情報を交換するのはどうでしょうか。
        - gender: male
          context: 部長：いいアイデアですね。では、来週から週に一度、部門ごとのミーティングを始めましょう。
        - gender: female
          context: 社員：ありがとうございます。私が日程を調整して、みんなに連絡します。
        questions:
          question: このあと、社員はまず何をしますか。
          choices:
          - 他の会社の会議に参加する
          - 部門ごとのミーティングの日程を決める
          - 営業部と開発部でランチ会を開く
          - 部長に仕事の進み具合を報告する
          correct_answer: 2
          explanation: この問題では、「課題」と「解決策」というビジネスシーンでよく使われるN3レベルの語彙が使われています。社員は「私が日程を調整して、みんなに連絡します」と話しているので、次にすることは「ミーティングの日程を決める」ことです。他の選択肢は会話の内容と合っていません。正答を選ぶには、会話の流れと要点を的確につかむ必要があります。
    - topic: 地域のイベントについて話す
      result:
        questions:
          question: このあと、男の人は何をするつもりですか。
          explanation: この問題では、聞き取りの中で「僕は写真を撮る係をやろうかな。イベントの写真をSNSにアップしたら、みんなにも見てもらえるし。」という男の人の発言がポイントです。N3レベルの語彙と表現（屋台、伝統的な踊り、ワークショップ、写真を撮るなど）を使いながら、会話の流れから「次に男の人がすること」を正確に把握する力が問われます。選択肢の中には似た内容も含まれているため、細かい聞き分けと要点の把握が主なチャレンジとなります。
          correct_answer: 2
          choices:
          - 伝統的な踊りに参加する
          - イベントで写真を撮る
          - 屋台で料理を作る
          - ワークショップに参加する
        conversation:
        - gender: female
          context: 今度の週末、町で「ふれあいフェスティバル」があるの知ってる？
        - gender: male
          context: うん、駅前の広場でやるんだよね？チラシを見たよ。何か面白そうなことあるかな。
        - gender: female
          context: いろいろあるみたいだよ。地元の野菜を使った料理の屋台とか、伝統的な踊りのステージもあるって。
        - gender: male
          context: へえ、楽しそうだね。僕、まだ町の伝統的な踊りを見たことがないから、見てみたいな。
        - gender: female
          context: 私も！あと、子ども向けのワークショップもあるらしいよ。弟を連れて行こうかな。
        - gender: male
          context: じゃあ、僕は写真を撮る係をやろうかな。イベントの写真をSNSにアップしたら、みんなにも見てもらえるし。
        - gender: female
          context: いいね！じゃあ、当日は現地で待ち合わせしよう。私は弟と一緒に早めに行くね。
    - topic: おすすめの旅行先について話す
      result:
        questions:
          question: 女の人はこのあとどうしますか。
          choices:
          - 京都に行くことを決める
          - 沖縄に行くことを決める
          - どちらに行くか、もう少し考える
          - 旅行に行くのをやめる
          correct_answer: 3
          explanation: この会話では、女の人が京都と沖縄、どちらに行くか迷っていて、「もう少し考えて、どっちに行くか決めるね」と言っています。したがって、選択肢3が正解です。N3レベルの学習者にとって、「もう少し考えて、どっちに行くか決めるね」という表現が、すぐに決めない＝まだ決めていない、という意味を理解できるかどうかがポイントになります。また、選択肢1や2に引っ張られないように、会話の最後までしっかり聞く必要があります。
        conversation:
        - gender: female
          context: 最近、どこか旅行に行きたいなと思ってるんだけど、どこかおすすめの場所ある？
        - gender: male
          context: うーん、僕は去年、京都に行ったんだけど、とても楽しかったよ。お寺もきれいだし、町の雰囲気も落ち着いていてよかった。
        - gender: female
          context: へえ、京都かあ。おいしいものもたくさんありそうだね。
        - gender: male
          context: うん、抹茶のスイーツとか、湯豆腐も食べたよ。あと、夜のライトアップもきれいだったよ。
        - gender: female
          context: いいなあ。私、まだ京都に行ったことがないから、今度行ってみようかな。でも、沖縄も気になってるんだ。
        - gender: male
          context: 沖縄もいいね。海がきれいだし、リゾート気分も味わえるし。夏だったら沖縄もおすすめだよ。
        - gender: female
          context: うーん、迷うなあ。もう少し考えて、どっちに行くか決めるね。
  - subsection_title: 積極的表現 (active_expression)
    description: 4問。会話の中で自分の意見や希望を述べる場面を聞き取る。
    question_topics:
    - topic: キャリア目標について話す
      result:
        image_background: 'The picture shows a company meeting room. Three people
          are sitting around a table: a manager, a young male employee (pointed by
          an arrow), and a female colleague. There are documents and laptops on the
          table. Everyone looks serious but positive. The manager is asking the young
          employee about his career goals as part of a regular evaluation meeting.'
        conversation:
        - gender: male
          context: 部長：山田さん、今日はこれからのキャリア目標について話しましょう。山田さんは、今後どんな仕事にチャレンジしたいと考えていますか？
        - gender: male
          context: 山田：はい、私は今の仕事にやりがいを感じていますが、もっとチームに貢献できるようになりたいと思っています。
        - gender: male
          context: 部長：具体的に、どんなことを目標にしていますか？
        questions:
          question: What will the person pointed by the arrow (the young employee)
            say next?
          choices:
          - まず、プロジェクトリーダーとしての経験を積みたいです。
          - 会社を辞めて、海外で働きたいです。
          - 休みを増やして、趣味の時間を増やしたいです。
          correct_answer: 1
          explanation: This question assesses students' ability to understand context
            and predict a logical, professional response in a career discussion. Options
            2 and 3 do not align with the positive and proactive tone expected in
            a formal career goal setting within a company. Option 1 uses N3-level
            vocabulary and is the most appropriate and realistic answer in a Japanese
            workplace setting, reflecting ambition and willingness to contribute to
            the team. The challenge for students is to distinguish between personal
            wishes and suitable career goals in a business context.
    - topic: 将来の抱負について話す
      result:
        image_background: The picture shows a Japanese classroom. Four students are
          sitting together in a group, with a teacher standing nearby. One girl with
          short hair (indicated by an arrow) is holding a sheet of paper and looks
          a little nervous. The other students are watching her. The teacher has just
          asked each student to talk about their “future goals” (将来の抱負).
        conversation:
        - gender: male
          context: 先生：それでは、みなさん、順番に将来の抱負について話してください。まず、田中さんからお願いします。
        - gender: female
          context: 田中：はい。私は将来、英語の先生になりたいです。そのために、もっと英語を勉強しようと思います。
        - gender: male
          context: 先生：いいですね。努力してください。では、次は山本さん、どうぞ。
        - gender: female
          context: 山本：はい。私は、大学で経済を勉強して、将来は自分の会社を作りたいです。そのために、今から経済の本をたくさん読もうと思っています。
        - gender: male
          context: 先生：すばらしいですね。では、矢印の女の子、佐藤さん、次はあなたの番です。
        questions:
          question: このあと、矢印の女の子（佐藤さん）は何と言いますか。
          choices:
          - 私は日本の歴史に興味があるので、歴史の先生になりたいです。
          - 私はサッカーが好きなので、毎日サッカーの練習をしています。
          - 私は料理が得意なので、今度みんなにお弁当を作ります。
          correct_answer: 1
          explanation: この問題のポイントは、「将来の抱負」というテーマに合う答えを選ぶことです。1番は「歴史の先生になりたい」という明確な将来の目標を理由とともに述べており、N3レベルの文法や語彙が使われています。2番と3番は現在の趣味や特技について述べていますが、「将来どうなりたいか」という抱負にはなっていません。N3学習者にとって、意志や希望を表す「〜たいです」「〜ので」などの文型を理解し、将来の目標を表現できるかが問われています。また、誤答選択肢も自然な内容なので、文脈理解力と将来の抱負表現の区別が試されます。
    - topic: 友人へのプレゼント選びについて話す
      result:
        image_background: In the picture, two university students are sitting at a
          café table. One student (pointed by the arrow) looks worried and is holding
          her smartphone, while the other is smiling and showing a small notebook.
          On the table, there are two cups of coffee and a gift bag. The atmosphere
          suggests they are discussing what to buy as a birthday present for a mutual
          friend.
        conversation:
        - gender: female
          context: もうすぐ佐藤さんの誕生日だけど、何をプレゼントしたらいいか、全然決められなくて困ってるの。去年はハンカチをあげたけど、今年は何か特別なものがいいなあ。
        - gender: female
          context: 佐藤さん、最近コーヒーにハマってるみたいだよ。コーヒーカップとかどう？
        - gender: female
          context: うーん、それもいいけど、他に何かアイデアあるかな。悩むなあ。
        questions:
          question: このあと、矢印の人は何と言いますか。
          choices:
          - じゃあ、コーヒーカップを見に行こうかな。
          - やっぱり、去年と同じハンカチにしよう。
          - もうプレゼントは買ってあるよ。
          correct_answer: 1
          explanation: The main challenge in this question is distinguishing between
            the options based on the context of the conversation. The student pointed
            by the arrow is indecisive about the gift and is considering new ideas.
            Option 1 shows a natural next step based on the friend's suggestion. Option
            2 contradicts her desire to choose something special and different from
            last year. Option 3 is not logical, as she is still undecided and hasn't
            bought anything yet. This tests careful listening and understanding of
            nuance at the N3 level.
    - topic: 特別な日の計画について話す
      result:
        image_background: There is a picture of a family of three—father, mother,
          and their teenage daughter—sitting together at the living room table. On
          the table are a cake box, a bouquet of flowers, and a small present. The
          father is pointing to a calendar where today’s date is circled in red. The
          mother, who is pointed by the arrow, is smiling and looking at her daughter,
          who looks surprised and happy. The recording is a conversation among the
          family members about today’s special plan.
        conversation:
        - gender: male
          context: 今日は家族みんなで集まる特別な日だね。
        - gender: female
          context: えっ？どうして？
        - gender: female
          context: 今日はあなたの誕生日でしょう？
        - gender: female
          context: あっ、そうだった！すっかり忘れてた！
        - gender: male
          context: それで、今夜はみんなで外で食事しようと思ってるんだ。
        - gender: female
          context: （娘に向かって）
        questions:
          question: What will the mother say next?
          choices:
          - じゃあ、一緒にケーキを作りましょうか。
          - 今日はあなたの好きなレストランに行く予定よ。
          - 今から公園で散歩しましょう。
          correct_answer: 2
          explanation: The main challenge for students is to listen for key information
            about the special day and the planned activity. The dialogue context makes
            it clear that the family is celebrating the daughter’s birthday and the
            father has suggested going out for dinner. Option 2 matches this context,
            using the N3-level phrase 「行く予定」 to express a future plan. Option 1 and
            3 mention other activities (making a cake, going for a walk) that are
            not mentioned in the dialogue, so students need to distinguish the correct
            future plan based on the listening.
  - subsection_title: 即時応答 (immediate_ack)
    description: 9問。短い会話や問いかけに対して即座に適切な返答を選ぶ。
    question_topics:
    - topic: バスの時刻表を尋ねる
      result:
        question: このあと、男の人はどうしますか。
        explanation: この問題のポイントは、会話の流れから「次のバスは10時15分に出発しますよ」と言われた後、男の人がどうするかを考えることです。N3レベルの学習者にとって、「時刻表」「出発」「待つ」など基本的な語彙の理解が問われます。選択肢1は「すぐにバスに乗る」ですが、バスはまだ来ていません。選択肢2は「もう一度聞く」ですが、すでに答えをもらっています。したがって、正しい選択は「10時15分まで待ちます」となります。時間表現や会話の流れを正確に理解することが主なチャレンジポイントです。
        suggestion: 3. 10時15分まで待ちます。
        conversation:
        - gender: male
          context: すみません、このバス停から駅まで行くバスは何時に出ますか。
        - gender: female
          context: ええと、時刻表を見ますね。次のバスは10時15分に出発しますよ。
        - gender: male
          context: ありがとうございます。
    - topic: 領収書を求める
      result:
        question: 会話を聞いて、次に男の人が何をするか、最もよいものを選びなさい。
        conversation:
        - gender: female
          context: すみません、こちらでコーヒーとサンドイッチをいただきました。領収書をいただけますか？
        - gender: male
          context: はい、少々お待ちください。今、レジで発行しますので。
        suggestion: 1. 男の人は領収書を発行する。
        explanation: この問題のポイントは、「領収書をいただけますか？」という依頼に対して、男の人が「今、レジで発行します」と答えているため、次に男の人がすることは領収書の発行です。N3レベルの学習者にとって、「発行する」「領収書」などの語彙の聞き取りと、会話の流れを正確に理解することが課題となります。選択肢の中には他の店員の業務（コーヒーを作る、サンドイッチを温める）も含まれているため、聞き取りの注意力が求められます。
    - topic: おすすめを尋ねる
      result:
        question: この後、男の人はどうしますか。
        explanation: この問題のポイントは、「今から行ってみます」という男の人の発言です。これは、女の人がすすめた駅前のイタリアンレストランに行く意志を表しています。N3レベルの語彙と表現を使っていますが、「おすすめ」「行ってみます」など、聞き取りが苦手な学習者は話の流れを正確に理解できないことがあります。「おすすめを尋ねる」→「すすめられた場所に行く」という自然な流れを押さえましょう。
        suggestion: 1. 駅の前のレストランに行く
        conversation:
        - gender: male
          context: すみません、この近くでおすすめのレストランはありますか？
        - gender: female
          context: そうですね、駅の前に新しいイタリアンレストランができましたよ。とてもおいしいですよ。
        - gender: male
          context: ありがとうございます。今から行ってみます。
    - topic: 引っ越しの準備について話す
      result:
        question: 男の人はこれから何をしますか。
        conversation:
        - gender: female
          context: 田中さん、引っ越しの準備は進んでいますか？
        - gender: male
          context: まだあまりできていません。今日はまず、いらないものを捨てようと思っています。
        - gender: female
          context: そうですか。荷物が少なくなると、荷造りも楽になりますよね。
        suggestion: 1. いらないものを捨てる
        explanation: この問題の主な難しさは、「これから何をするか」という未来の行動を聞き取る点です。会話では「今日はまず、いらないものを捨てようと思っています」という発言があり、意志表現「〜ようと思っています」に注目できるかがポイントです。N3レベルの語彙「いらないもの」「捨てる」も重要です。
    - topic: 学校や職場での一日の流れについて話す
      result:
        question: 次の会話を読んで、最も適切なものを一つ選びなさい。
        conversation:
        - gender: female
          context: 田中さん、今日の仕事はもう終わりましたか？
        - gender: male
          context: はい、さっき終わりました。これから教室の掃除をします。
        - gender: female
          context: じゃあ、掃除が終わったら帰るんですね。
        - gender: male
          context: はい、そうです。
        suggestion: 1. 田中さんは掃除の前に少し休憩します。 2. 田中さんはこれから教室の掃除をします。 3. 田中さんは掃除のあとにまた仕事をします。
        explanation: この会話では、田中さんが「これから教室の掃除をします」と言っています。選択肢1の「休憩」や選択肢3の「また仕事をします」といった内容は会話に出てきません。田中さんの次の行動は掃除をすることなので、2が正解です。会話の流れを正確に読み取り、登場人物の次の行動を理解することがポイントです。
    - topic: 言語学習のコツについて話す
      result:
        question: 会話の内容から、女の人が次に取る行動として最も適切なものを一つ選びなさい。
        explanation: この問題は、会話の流れを理解し、アドバイスをどう受け止めて行動を変えるかを問うものです。N3レベルの「日記を書く」「ドラマを見る」「楽しみながら勉強する」といった表現が理解できるかがポイントです。選択肢も具体的に違いが分かるように設定しており、正しい選択肢を選ぶためには、会話の意図を正確に読み取る必要があります。
        suggestion: 女の人は日本語で日記を書いたり、ドラマを見たりして勉強を続けることが最も適切です。
        conversation:
        - gender: female
          context: 最近、日本語の勉強がなかなか続かなくて困っています。
        - gender: male
          context: そうなんですか。僕は、自分の趣味について日本語で日記を書いたり、好きなドラマを日本語で見たりしていますよ。
        - gender: female
          context: なるほど。楽しみながら勉強するのが大事なんですね。
        - gender: male
          context: ええ、毎日少しずつ続けることもポイントですよ。
    - topic: 好きな季節について話す
      result:
        question: 今週末、男の人は何をしますか。
        explanation: 男の人は「今週末も友だちと山に行くつもりです」と言っています。「紅葉を見に行く」という前の発言と合わせて、1が正解です。この問題のポイントは、「紅葉」「山に行く」「つもり」の語彙や表現を正しく理解できるかどうかです。秋の季節感や日本の文化的な行動を反映した内容で、N3レベルの学習者に適しています。選択肢も季節や会話内容に関連し、消去法だけでなく内容理解が必要となっています。
        suggestion: 1. 山へ紅葉を見に行きます
        conversation:
        - gender: female
          context: 鈴木さん、秋は何をするのが好きですか。
        - gender: male
          context: 僕は秋になると、よく山へ紅葉を見に行きます。きれいな葉を見ながら歩くのが楽しいです。
        - gender: female
          context: いいですね。秋は涼しくて散歩にちょうどいい季節ですよね。
        - gender: male
          context: はい。今週末も友だちと山に行くつもりです。
    - topic: 地元の食べ物や特産品について話す
      result:
        question: 田中さん（女）はこの後どうしますか。
        explanation: この問題のポイントは、会話の流れを正しく理解し、田中さん（女）が次に何をするかを推測することです。最後に田中さん（男）が「僕も買ってみようかな」と言っているので、田中さん（女）は「おすすめですよ」などとすすめる可能性が高いです。選択肢2や3は会話からは自然ではありません。N3レベルの語彙や表現を使い、ふりがなも付けて、より分かりやすくしました。
        suggestion: 1. 男にまんじゅうをすすめる。
        conversation:
        - gender: male
          context: 田中さん（女）、週末はどこか行きましたか？
        - gender: female
          context: はい、地元の市場（いちば）に行ってきました。新鮮（しんせん）な野菜（やさい）や果物（くだもの）、それに有名（ゆうめい）な特産品（とくさんひん）のまんじゅうを買いました。
        - gender: male
          context: へえ、そのまんじゅうはどんな味（あじ）ですか？
        - gender: female
          context: 甘（あま）くて、とてもおいしいですよ。お土産（みやげ）にも人気（にんき）なんです。
        - gender: male
          context: そうなんですか。今度（こんど）、僕（ぼく）も買ってみようかな。
    - topic: 健康診断や医者への訪問について話す
      result:
        question: このあと、男の人は何をしますか。
        explanation: この問題のポイントは、「これからどうするんですか？」という質問に対して、男の人が「医者に相談しに行こうと思っています」と答えていることです。したがって、男の人が次にすることは「病院に行って医者に相談する」になります。N3レベルでは、「相談する」「健康診断」などの語彙の理解と、会話の流れから次の行動を推測する力が問われます。選択肢2や3にひきずられず、会話文の意図を正確につかむことが主なチャレンジです。
        suggestion: 1. 病院に行って医者に相談する
        conversation:
        - gender: female
          context: 田中さん、最近体の調子はどうですか？
        - gender: male
          context: 実は、会社の健康診断で血圧が高いと言われて…。
        - gender: female
          context: そうなんですか。それで、これからどうするんですか？
        - gender: male
          context: 医者に相談しに行こうと思っています。


In [10]:
output_data

{'sections': [{'section_title': '第1部：語彙・文法',
   'subsections': [{'subsection_title': '漢字読み (kanji_reading)',
     'description': '8問。与えられた漢字の読み仮名をひらがなで答える。',
     'question_topics': [{'topic': 'お気に入りのレストランについて話す',
       'result': {'question': '下線部の語の読み方として最も適当なものを、1・2・3・4の中から一つ選びなさい。私は休日によくお気に入りのレストランに行きます。',
        'choices': ['おきにいり', 'おきにい', 'おきあいり', 'おきにいみ'],
        'correct_answer': 1,
        'explanation': '「お気に入り」は「特に好きなもの」「愛着のあるもの」という意味で、日常会話や文章でよく使われる表現です。語全体の正しい読み方は「おきにいり」です。JLPT N3レベルの学習者にとっては、送り仮名を含む語の読み方を正確に覚えることが重要なポイントです。選択肢2、3、4は実際には使われない読み方なので、正しい音のつながりと送り仮名に注意して覚えましょう。'}},
      {'topic': '地元の観光名所について話す',
       'result': {'question': '地元の **観光名所** について話す。  1 かんこうみょうしょ  2 かんこうめいしょ  3 かんこうめいところ  4 かんこうなところ',
        'correct_answer': 2,
        'choices': ['かんこうみょうしょ', 'かんこうめいしょ', 'かんこうめいところ', 'かんこうなところ'],
        'explanation': '「観光名所」はN3レベルでよく出る語彙ですが、「名所（めいしょ）」の読みが難しいと感じる学習者が多いです。「名（めい）」は「有名（ゆうめい）」などで馴染みがあっても、「所（しょ）」の音読みとの組み合わせが分かりづらいからです。また、「名所」を「みょうしょ」と読んでしまうミス

In [11]:
def render_to_html(sections):  
    html = '<html><body>\n'  
    for section in sections:  
        html += f'<h1>{section.get("section_title", "")}</h1>\n'  
        for subsection in section.get("subsections", []):  
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'  
            if "description" in subsection:  
                html += f'<p>{subsection["description"]}</p>\n'  
            for qtopic in subsection.get("question_topics", []):  
                html += f'<h3>{qtopic.get("topic", "")}</h3>\n'  
                result = qtopic.get("result", {})  
                if isinstance(result, dict):  
                    if "html_article" in result:  
                        html += result["html_article"] + "\n"  
                    if "question" in result:  
                        html += f'<p><strong>Q: {result["question"]}</strong></p>\n'  
                        if "choices" in result:  
                            correct = result.get("correct_answer", -1)  
                            html += '<ul>\n'  
                            for idx, choice in enumerate(result["choices"], 1):  
                                if idx == correct:  
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"  
                                else:  
                                    html += f"<li>{choice}</li>\n"  
                            html += '</ul>\n'
                        if "explanation" in result:  
                            html += f'<p><em>Explanation: {result["explanation"]}</em></p>\n'                              
                    if "questions" in result and isinstance(result["questions"], list):  
                        for q in result["questions"]:  
                            html += f'<p><strong>Q: {q["question"]}</strong></p>\n'  
                            if "choices" in q:  
                                correct = q.get("correct_answer", -1)  
                                html += '<ul>\n'  
                                for idx, choice in enumerate(q["choices"], 1):  
                                    if idx == correct:  
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"  
                                    else:  
                                        html += f"<li>{choice}</li>\n"  
                                html += '</ul>\n'
                            if "explanation" in q:  
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'                                  
                    if "questions" in result and isinstance(result["questions"], dict):  
                        q = result["questions"]  
                        html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                        if "choices" in q:  
                            correct = q.get("correct_answer", -1)  
                            html += '<ul>\n'  
                            for idx, choice in enumerate(q["choices"], 1):  
                                if idx == correct:  
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"  
                                else:  
                                    html += f"<li>{choice}</li>\n"  
                            html += '</ul>\n'
                        if "explanation" in q:  
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n' 
    html += '</body></html>'  
    return html

In [12]:
html_output = render_to_html(output_data['sections'])  
display(Markdown(html_output))

<html><body>
<h1>第1部：語彙・文法</h1>
<h2>漢字読み (kanji_reading)</h2>
<p>8問。与えられた漢字の読み仮名をひらがなで答える。</p>
<h3>お気に入りのレストランについて話す</h3>
<p><strong>Q: 下線部の語の読み方として最も適当なものを、1・2・3・4の中から一つ選びなさい。私は休日によくお気に入りのレストランに行きます。</strong></p>
<ul>
<li><b>おきにいり</b> <span style='color:green;'>(correct)</span></li>
<li>おきにい</li>
<li>おきあいり</li>
<li>おきにいみ</li>
</ul>
<p><em>Explanation: 「お気に入り」は「特に好きなもの」「愛着のあるもの」という意味で、日常会話や文章でよく使われる表現です。語全体の正しい読み方は「おきにいり」です。JLPT N3レベルの学習者にとっては、送り仮名を含む語の読み方を正確に覚えることが重要なポイントです。選択肢2、3、4は実際には使われない読み方なので、正しい音のつながりと送り仮名に注意して覚えましょう。</em></p>
<h3>地元の観光名所について話す</h3>
<p><strong>Q: 地元の **観光名所** について話す。  1 かんこうみょうしょ  2 かんこうめいしょ  3 かんこうめいところ  4 かんこうなところ</strong></p>
<ul>
<li>かんこうみょうしょ</li>
<li><b>かんこうめいしょ</b> <span style='color:green;'>(correct)</span></li>
<li>かんこうめいところ</li>
<li>かんこうなところ</li>
</ul>
<p><em>Explanation: 「観光名所」はN3レベルでよく出る語彙ですが、「名所（めいしょ）」の読みが難しいと感じる学習者が多いです。「名（めい）」は「有名（ゆうめい）」などで馴染みがあっても、「所（しょ）」の音読みとの組み合わせが分かりづらいからです。また、「名所」を「みょうしょ」と読んでしまうミスや、「名」と「所」を訓読みで「めいところ」などと読んでしまうこともよく見られます。選択肢2「かんこうめいしょ」が正しい読みです。</em></p>
<h3>現れ</h3>
<p><strong>Q: 次の漢字の読み方として最もよいものを、1・2・3・4から一つ選びなさい。彼の努力がついに現れとなった。</strong></p>
<ul>
<li><b>あらわれ</b> <span style='color:green;'>(correct)</span></li>
<li>うつれ</li>
<li>ひかれ</li>
<li>みだれ</li>
</ul>
<p><em>Explanation: 「現れ（あらわれ）」は、目に見える形で現れることや、何かが表面に出てくることを意味する名詞です。N3レベルで頻出する語彙であり、動詞「現れる（あらわれる）」の名詞形です。他の選択肢「うつれ」「ひかれ」「みだれ」は、それぞれ「移れ」「光れ」「乱れ」という意味や読みですが、「現れ」とは関係ありません。学習者が混同しやすいポイントは、「現」という漢字が「げん」「うつ（す）」など複数の読み方を持つこと、また「現れ」と「表れ（あらわれ）」の使い分けがあることです。関連語：現れる（あらわれる／to appear）、表れる（あらわれる／to become apparent）</em></p>
<h3>冗談</h3>
<p><strong>Q: 次の漢字の読み方として正しいものを選びなさい。冗談は言わないでください。</strong></p>
<ul>
<li>じょうたん</li>
<li><b>じょうだん</b> <span style='color:green;'>(correct)</span></li>
<li>じょだん</li>
<li>じょうなん</li>
</ul>
<p><em>Explanation: 「冗談」は日常会話でよく使われるN3レベルの語彙です。「冗」は「じょう」、「談」は「だん」と読むため、正しい組み合わせは「じょうだん」となります。選択肢には、発音が似ているものや誤読しやすいものが含まれており、漢字の音読みの知識が問われます。特に「じょうたん」や「じょだん」は一見正しそうに見えるため、注意が必要です。</em></p>
<h3>栄養</h3>
<p><strong>Q: 次の漢字の正しい読み方を選びなさい。栄養</strong></p>
<ul>
<li><b>えいよう</b> <span style='color:green;'>(correct)</span></li>
<li>えいしょく</li>
<li>せいよう</li>
<li>せいしょく</li>
</ul>
<p><em>Explanation: 「栄養（えいよう）」は、食事や健康に関する会話・文章で頻繁に使われるN3レベルの重要語彙です。「栄」は「えい」、「養」は「よう」と読むのが一般的ですが、「養」には「やしな（う）」や「いく」などの読みもあり、漢字の音読み・訓読みの区別が混乱しやすい点が学習者にとっての難しさです。また、「栄」の読みは「えい」と「さか（える）」があり、「養」は「よう」と「やしな（う）」があります。選択肢の「せいよう」は「西洋（せいよう）」の読みと混同しやすく、「せいしょく」は「生殖（せいしょく）」の読みと混同しやすい例です。また、「えいしょく」は「栄食」という熟語は一般的ではありませんが、似た音の組み合わせとして誤答例に挙げられます。このように、実際に存在する語彙や音の似た熟語と混同しやすい点が、漢字語彙問題の主な難しさとなります。</em></p>
<h3>豊か</h3>
<p><strong>Q: 田中さんは都会での生活に疲れて、**自然が豊かな**町に引っ越すことにしました。次のうち、空欄に入る適切な読み方はどれですか？</strong></p>
<ul>
<li>ほうふな</li>
<li><b>ゆたかな</b> <span style='color:green;'>(correct)</span></li>
<li>とよかな</li>
<li>ほうかな</li>
</ul>
<p><em>Explanation: 「豊か」は「ゆたか」と読み、な形容詞（形容動詞）です。「自然が豊かな町」は、自然が十分にあって恵まれている町という意味になります。選択肢1の「ほうふな」は「豊富な」で、「ほうふ」と読む別の単語です。選択肢3「とよかな」、4「ほうかな」は誤読です。「豊か」は「豊富（ほうふ）」と混同しやすいため、漢字と言葉の使い分けに注意しましょう。主なポイントと難しさ：・「豊か」はN3レベルの重要語で、意味・読み・文法（な形容詞）を正確に押さえる必要があります。・「豊富（ほうふ）」との違いが混乱しやすいので、選択肢に入れることで識別力を問うことができます。・やや長めの文脈にすることで、文脈理解力も同時に測れます。</em></p>
<h3>筋肉</h3>
<p><strong>Q: 筋肉は身体を動かすために大切な組織です。この「筋肉」の正しい読み方を選びなさい。</strong></p>
<ul>
<li><b>きんにく</b> <span style='color:green;'>(correct)</span></li>
<li>きんしょく</li>
<li>すじにく</li>
<li>けんにく</li>
</ul>
<p><em>Explanation: 「筋肉」はJLPT N3レベルでよく出る語彙です。漢字「筋（きん）」と「肉（にく）」を正しく組み合わせて読めるかがポイントです。「筋」は「すじ」とも読めますが、熟語では「きん」と読むことが多く、「筋肉」は「きんにく」となります。他の選択肢は間違った読み方や意味の異なる語です。漢字の訓読みと音読みの区別が難しい点が、この問題の主なチャレンジです。</em></p>
<h3>答案</h3>
<p><strong>Q: この書類の規則をよく読んでください。次の中で「きそく」と読む漢字はどれですか？</strong></p>
<ul>
<li><b>きそく</b> <span style='color:green;'>(correct)</span></li>
<li>きぞく</li>
<li>きそん</li>
<li>きせつ</li>
</ul>
<p><em>Explanation: 「規則（きそく）」は「ルール」という意味で、会社や学校などでよく使われるN3レベルの重要語彙です。「きぞく（貴族）」「きそん（既存）」「きせつ（季節）」は実在する日本語の読みで、音が似ているため間違えやすいですが、意味が異なります。漢字の意味と音を正確に結びつけることがポイントです。</em></p>
<h2>漢字書き取り (write_chinese)</h2>
<p>6問。ひらがなで書かれた単語を漢字で書く。</p>
<h3>しゅつじょう（出場）</h3>
<p><strong>Q: 彼は毎年、マラソン大会に しゅつじょう しています。今年も参加する予定です。</strong></p>
<ul>
<li><b>出場</b> <span style='color:green;'>(correct)</span></li>
<li>出張</li>
<li>出演</li>
<li>出席</li>
</ul>
<p><em>Explanation: 「しゅつじょう」は「出場」と書き、スポーツ大会やコンクールなどに参加・出ることを意味します。選択肢2「出張」は仕事のために他の場所へ行くこと、選択肢3「出演」は舞台やテレビなどに出ること、選択肢4「出席」は会議や授業などに出ることを指し、いずれも「出場」とは意味が異なります。この問題では、似た読みや漢字を持つ語句が並んでいるため、文脈をよく読み取り、語彙の正確な使い分けができるかがポイントとなります。特に「出張」や「出演」など、日常でもよく使われる語と混同しやすいため、意味の違いをしっかり理解しているかを問う良問です。</em></p>
<h3>しょくひん（食品）</h3>
<p><strong>Q: 【指示】下線部のことばの正しい漢字表記を、1～4の中から一つ選びなさい。このスーパーでは新鮮な しょくひん がたくさん売られています。</strong></p>
<ul>
<li><b>食品</b> <span style='color:green;'>(correct)</span></li>
<li>食員</li>
<li>食品物</li>
<li>食員品</li>
</ul>
<p><em>Explanation: 「食品」はN3レベルで頻出する語彙です。類似する漢字語が多いので注意が必要です。</em></p>
<h3>せつやく（節約）</h3>
<p><strong>Q: 家計のために せつやく を心がけています。正しい漢字表記を選んでください。</strong></p>
<ul>
<li><b>節約</b> <span style='color:green;'>(correct)</span></li>
<li>節約化</li>
<li>節税</li>
<li>節約費</li>
</ul>
<p><em>Explanation: この問題のポイントは、「せつやく」という日常的な言葉に対して、正しい漢字表記を選ばせることです。 「節約（せつやく）」は「無駄遣いをしないで控えること、倹約すること」という意味で、N3の重要単語です。 選択肢2「節約化」は実際には使われない表現で、語尾の「化」がついていることで迷わせます。 選択肢3「節税」は「税金を節約すること」で、意味が異なりますが、似た漢字なので混同しやすいです。 選択肢4「節約費」も実際には使われない表現で、「費」がつくことで「費用」と混同しやすくなっています。 漢字の組み合わせや意味の違いに注意する必要があり、N3レベルの漢字力と語彙力を問う良問です。</em></p>
<h3>けんこう（健康）</h3>
<p><strong>Q: 次の文の（　　　）に入る正しい漢字を、1～4の中から一つ選びなさい。毎日運動をして、（けんこう）に気をつけています。</strong></p>
<ul>
<li>健考</li>
<li><b>健康</b> <span style='color:green;'>(correct)</span></li>
<li>建康</li>
<li>健体</li>
</ul>
<p><em>Explanation: 「けんこう」に正しく対応する漢字は「健康」です。1の「健考」は実際には存在しない熟語ですが、読みが似ているため迷いやすい選択肢です。3の「建康」は中国の地名で、日本語では一般的に使われません。4の「健体」は「けんたい」と読み、「健康な体」という意味を持ちますが、「けんこう」とは読みません。このように、似た漢字や読み方に注意して、正しい語彙知識を身につけることが大切です。N3レベルでは、日常会話や文章で頻出する語彙の正しい漢字表記を問うことが重要です。似た漢字や意味が近い単語が多く、混乱しやすいので、文脈と漢字の意味・読み方の両方を確認しましょう。また、日本語学習者にとっては、日本語で一般的に使われる表現かどうかも判断のポイントとなります。</em></p>
<h3>しゅうにゅう（収入）</h3>
<p><strong>Q: 彼は毎月の しゅうにゅう が安定しています。</strong></p>
<ul>
<li><b>収入</b> <span style='color:green;'>(correct)</span></li>
<li>所得</li>
<li>収納</li>
<li>入収</li>
</ul>
<p><em>Explanation: 「しゅうにゅう」の正しい漢字は「収入」です。他の選択肢も似た意味や読みを持つ熟語ですが、意味や使い方が異なります。「所得」は収入から経費などを差し引いた残りの金額、「収納」は物やお金をしまうこと、「入収」は実際には使われない言葉です。N3レベルでは、音読みの熟語で紛らわしいものが多いため、意味と正しい漢字の組み合わせを覚えているかが問われます。</em></p>
<h3>かんしゃ（感謝）</h3>
<p><strong>Q: 田中さんには、いつも助けてもらっているので、心から かんしゃ の気持ちを伝えました。</strong></p>
<ul>
<li>感情</li>
<li><b>感謝</b> <span style='color:green;'>(correct)</span></li>
<li>感激</li>
<li>感想</li>
</ul>
<p><em>Explanation: この問題では、「かんしゃ」の正しい漢字を選ぶことが求められています。「感謝」は、誰かの親切や助けに対して「ありがたい」と思う気持ちを表す言葉です。選択肢には「感情（かんじょう）」「感激（かんげき）」「感想（かんそう）」など、意味が似ているように見える語が並んでいるため、文の内容をよく読んで正しい言葉を選ぶ必要があります。「感謝」は「ありがとう」と同じ気持ちを表す時に使います。</em></p>
<h2>語彙意味選択 (word_meaning)</h2>
<p>11問。下線部の語の意味に最も近いものを選ぶ。</p>
<h3>必死</h3>
<p><strong>Q: 次の文の（　）に入る最も適切な漢字を、1～4の中から一つ選びなさい。彼は試験に合格するために、毎日（　）に勉強している。</strong></p>
<ul>
<li>必至</li>
<li><b>必死</b> <span style='color:green;'>(correct)</span></li>
<li>必須</li>
<li>決死</li>
</ul>
<p><em>Explanation: この問題は、「ひっし」という同じ読みを持つ漢字語彙の中から、文脈に合う正しい漢字を選ぶ問題です。「必死」は「死ぬ覚悟で全力を尽くすこと」という意味で、日常的に「必死に勉強する」「必死に働く」など、何かに全力で取り組む様子を表します。選択肢の「必至」は「必ずそうなること」、「必須」は「必ず必要なこと」、「決死」は「死を決する覚悟」という意味で、主に戦いや非常に危険な状況で使われます。したがって、日常的な努力や勉強の場面では「必死」が最も適切です。この問題のポイントは、同音異義語の意味と使い方の違いを正確に理解しているかどうかです。「決死」は意味が似ていますが、日常の努力には通常使わないため、そこを区別できるかが重要です。</em></p>
<h3>調子</h3>
<p><strong>Q: 次の文の（　）に入る正しい漢字を、下の1〜4から一つ選びなさい。最近、体のちょうしがよくないので、今日は早く寝ようと思います。</strong></p>
<ul>
<li>調市</li>
<li><b>調子</b> <span style='color:green;'>(correct)</span></li>
<li>調氏</li>
<li>調子子</li>
</ul>
<p><em>Explanation: この問題は、「調子」の正しい漢字表記を選ばせるものです。N3レベルの学習者にとって、「調」「子」は基本的な漢字ですが、他の選択肢も見た目が似ていたり、日常で使う漢字の組み合わせなので混乱しやすいです。特に「市」「氏」などはよく目にするため、正確な語彙知識が問われます。「調子」は「物事や体の状態・具合」の意味で、日常会話でも頻出する重要語です。</em></p>
<h3>破産</h3>
<p><strong>Q: 長年の赤字経営が続き、ついに会社が（はさん）することになった。</strong></p>
<ul>
<li>破滅</li>
<li><b>破産</b> <span style='color:green;'>(correct)</span></li>
<li>破損</li>
<li>破格</li>
</ul>
<p><em>Explanation: 「破産（はさん）」は、借金などの支払いができなくなり、財産を失うことを意味します。他の選択肢は意味や読みが似ていますが、文脈に合いません。- 「破滅（はめつ）」はすべてを失うこと- 「破損（はそん）」は壊れること- 「破格（はかく）」は基準から外れること。N3受験者は、漢字の意味と読みの混同に注意しましょう。特に「破損」「破滅」など似た語との区別がポイントです。</em></p>
<h3>争う</h3>
<p><strong>Q: 彼らは優勝を目指してお互いに（あらそう）ことになった。</strong></p>
<ul>
<li>競走</li>
<li><b>競う</b> <span style='color:green;'>(correct)</span></li>
<li>争う</li>
<li>争議</li>
</ul>
<p><em>Explanation: この文脈では「優勝を目指してお互いに（あらそう）」の意味は「互いに勝とうとして競争する」という意味になります。そのため、正しい漢字は「競う（きそう）」です。「争う（あらそう）」は「けんかする」「対立する」などのニュアンスが強く、スポーツや勝負での「競い合う」場面では「競う」が自然です。「競走（きょうそう）」は「走る競技」の意味、「争議（そうぎ）」は「もめごと」「争いごと」を指します。N3レベルでは「争う」と「競う」の意味や使い分けを正しく理解することが大切です。</em></p>
<h3>題名</h3>
<p><strong>Q: この本のタイトルとして正しい言葉を選びなさい。</strong></p>
<ul>
<li>題目</li>
<li><b>題名</b> <span style='color:green;'>(correct)</span></li>
<li>表題</li>
<li>標題</li>
</ul>
<p><em>Explanation: 「題名」（だいめい）は、本や映画、作品などのタイトルを意味します。選択肢の「題目」は主題や課題を指し、「表題」「標題」はどちらも見出しやタイトルの意味がありますが、一般的に本や作品のタイトルには「題名」が最も適切です。特に「表題」「標題」は書類や章のタイトルなど、使われる場面が異なります。似た語彙が多いため、文脈に合った正しい語を選ぶ力が求められる問題です。</em></p>
<h3>要素</h3>
<p><strong>Q: この文章の（　）に入る正しい漢字を選びなさい。科学の授業で、水は酸素と水素という二つの（　）からできていると習いました。</strong></p>
<ul>
<li>要因</li>
<li><b>要素</b> <span style='color:green;'>(correct)</span></li>
<li>要点</li>
<li>要約</li>
</ul>
<p><em>Explanation: この問題のポイントは、「ようそ」という読みから適切な漢字を選ぶことです。N3レベルの学習者にとって、「要素」「要因」「要点」「要約」など似た読み・意味の語が混乱しやすいです。「要素」は「ものごとを成り立たせている基本的な内容や成分」という意味で、この文脈に合います。他の選択肢は、原因（要因）、要点（main point）、要約（summary）と意味が異なるため、正しく区別できるかがポイントです。</em></p>
<h3>湿気</h3>
<p><strong>Q: 梅雨の時期は、部屋の中に「しっけ（しっけ）」がたまりやすいので注意が必要です。次のうち、空気や物に含まれる水分のことを指す言葉はどれですか？</strong></p>
<ul>
<li>湿度</li>
<li><b>湿気</b> <span style='color:green;'>(correct)</span></li>
<li>湿度計</li>
<li>湿布</li>
</ul>
<p><em>Explanation: この問題の正解は「湿気」です。「しっけ」は空気や物に含まれる水分のことを指し、梅雨の時期によく使われる語です。選択肢1「湿度（しつど）」は空気中の水分量をパーセントで表す言葉で、意味が異なります。選択肢3「湿度計（しつどけい）」は湿度を測る道具です。選択肢4「湿布（しっぷ）」は、けがや痛みに貼る薬です。音が似ている語も多く、読みと意味の区別がN3学習者には難しいポイントです。文脈から正しい語を選ぶ練習をしましょう。例文：・雨の日は湿気が多くて、髪の毛がまとまりません。・この部屋は湿気がたまりやすいので、換気をしてください。教師コメント：「湿気」は日本の生活や気候に密接な語彙であり、日常会話でも頻繁に使われます。問題文にふりがなをつけることで、学習者が漢字と読みを結びつけやすくなります。似た音の語との違いを意識しながら学習しましょう。</em></p>
<h3>努力</h3>
<p><strong>Q: 彼は試験に合格するために毎日「どりょく」している。</strong></p>
<ul>
<li>動力</li>
<li><b>努力</b> <span style='color:green;'>(correct)</span></li>
<li>労働</li>
<li>勤勉</li>
</ul>
<p><em>Explanation: この問題は、「どりょく」の正しい漢字表記を選ぶものです。- 「動力」（どうりょく）は機械などを動かす力を意味し、意味も読みも異なります。- 「労働」（ろうどう）は仕事や働くことを指し、「どりょく」とは読みません。- 「勤勉」（きんべん）は「まじめに努力すること」を意味しますが、読みも異なります。- 「努力」が正しい漢字表記で、「どりょく」と読みます。この問題の難しさは、似た意味や形の語が多い点にあります。N3レベルでは「努力」「労働」「勤勉」など、意味が近い語の漢字と読みの結びつきを正確に覚える必要があります。誤答選択肢も実際によく使われる語彙で、学習者の語彙力を適切に測ることができます。</em></p>
<h3>礼儀</h3>
<p><strong>Q: 彼はとても（れいぎ）正しい人なので、みんなに好かれています。</strong></p>
<ul>
<li>礼義</li>
<li><b>礼儀</b> <span style='color:green;'>(correct)</span></li>
<li>例儀</li>
<li>礼技</li>
</ul>
<p><em>Explanation: この問題は「れいぎ」の正しい漢字表記を選ぶ問題です。「礼儀」は社会的なマナーやルールを示す語で、N3レベルでよく出ます。選択肢の「義」「儀」「技」はそれぞれ意味が異なるので注意しましょう。特に「儀」と「義」は混同しやすいですが、「礼儀」が正解です。</em></p>
<h3>資本</h3>
<p><strong>Q: 次の文の（　）に入る言葉として、最も適切な漢字を1つ選びなさい。新しい会社を作るためには、多くの「しほん」が必要です。</strong></p>
<ul>
<li>資産</li>
<li><b>資本</b> <span style='color:green;'>(correct)</span></li>
<li>資源</li>
<li>資金</li>
</ul>
<p><em>Explanation: 「資本（しほん）」は、会社や事業を始めるときの元手となるお金や財産を意味します。他の選択肢も似た漢字ですが、「資産（しさん）」は持っている財産全体、「資源（しげん）」は自然や社会の中で使えるもの、「資金（しきん）」は使うためのお金のことです。それぞれ意味が違うので、文の内容に合う「資本」が正解です。</em></p>
<h3>特徴</h3>
<p><strong>Q: 次の文の（　）に入る最も適切な漢字を、1～4の中から一つ選びなさい。彼の話し方には独特の（とくちょう）があるので、すぐに本人だとわかります。</strong></p>
<ul>
<li><b>特徴</b> <span style='color:green;'>(correct)</span></li>
<li>特長</li>
<li>特性</li>
<li>特質</li>
</ul>
<p><em>Explanation: 「特徴」は、他と区別できる目立った点や性質を指します。例えば、「この鳥の特徴は赤いくちばしです」のように使います。「特長」は、他よりも優れている点を指し、「この車の特長は燃費が良いことです」といった使い方をします。「特性」は、物や人が本来持っている性質や傾向を指します（例：水の特性は100度で沸騰すること）。「特質」は、特に目立つ性質や特徴的な性格を強調する場合に使われます（例：彼の特質は忍耐強さです）。この文脈では、「独特の～がある」は「特徴」が最も適切です。特に「特徴」と「特長」の使い分けはN3レベルで重要なポイントとなります。</em></p>
<h2>同義語置き換え (synonym_substitution)</h2>
<p>5問。文中の語句を同じ意味の語に置き換える。</p>
<h3>現代</h3>
<p><strong>Q: この小説は（現代）の若者について書かれています。</strong></p>
<ul>
<li><b>現在</b> <span style='color:green;'>(correct)</span></li>
<li>近代</li>
<li>古代</li>
<li>未来</li>
</ul>
<p><em>Explanation: 「現代（げんだい）」は「今の時代」「現在」という意味で使われます。「現在（げんざい）」は「今」という意味で、「現代」と非常に近い意味を持つため、最も適切な選択肢です。「近代（きんだい）」は歴史的に見ると「明治時代から第二次世界大戦ごろまで」を指し、「現代」とは異なります。「古代（こだい）」は昔の時代、「未来（みらい）」はこれからの時代なので文脈に合いません。N3レベルの学習者にとっては、「現代」と「近代」を混同しやすいので、正確な意味の違いに注意しましょう。</em></p>
<h3>種類</h3>
<p><strong>Q: この店にはいろいろな種類のパンがあります。この文の「種類」と最も近い意味の言葉はどれですか。</strong></p>
<ul>
<li>形</li>
<li><b>タイプ</b> <span style='color:green;'>(correct)</span></li>
<li>重さ</li>
<li>時間</li>
</ul>
<p><em>Explanation: 「種類」は、性質や特徴によって分けられたグループやカテゴリーを表す言葉です。「タイプ」も同じように、形や性質によって分けられたグループを指します。「形」は見た目、「重さ」は量、「時間」は時刻や期間を表すため、「種類」とは意味が異なります。カタカナ語の「タイプ」もN3レベルでよく使われるので、文脈から正しい意味を選ぶ必要があります。受験者にとっては、似ている言葉の中から文に合うものを選ぶ力が問われる問題です。</em></p>
<h3>完全</h3>
<p><strong>Q: 次の文の「完全」と最も近い意味の言葉を選びなさい。この計画は完全に成功した。</strong></p>
<ul>
<li>しっかり</li>
<li>すべて</li>
<li><b>まったく</b> <span style='color:green;'>(correct)</span></li>
<li>かんたんに</li>
</ul>
<p><em>Explanation: 「完全」は「欠けているところがなく、すべてがそろっていること」を意味し、「まったく」も「すべて」「完全に」という意味で使われます。N3レベルでは「しっかり（firmly）」「すべて（all）」「かんたんに（easily）」もよく出る言葉ですが、文脈的に「完全に成功した」は「まったく成功した」と言い換えられます。主な難しさは、「すべて」も「全部」という意味があるため、選択肢として迷いやすい点です。しかし、「完全に」は「まったく」「徹底的に」といった副詞がより近い意味となります。</em></p>
<h3>冷やす</h3>
<p><strong>Q: 冷蔵庫でジュースを冷やしておきました。次の中から、「冷やす」の意味に最も近いものを選びなさい。</strong></p>
<ul>
<li>あたためる</li>
<li>こおらせる</li>
<li><b>冷たくする</b> <span style='color:green;'>(correct)</span></li>
<li>さます</li>
</ul>
<p><em>Explanation: この問題は、「冷やす」の意味を正しく理解し、似ている語彙と区別できるかどうかを問うものです。「冷やす」は、物の温度を下げて冷たくすることを表します。選択肢3「冷たくする」が最も近い意味です。選択肢4「さます」は、熱いものを適温に下げる場合に使うため、必ずしも冷たくするとは限りません。選択肢2「こおらせる」は「凍らせる」こと、選択肢1「あたためる」は反対の意味です。N3レベルでは、「冷やす」「さます」「冷たくする」などの使い分けが難しいため、意味の違いに注意しましょう。</em></p>
<h3>自然</h3>
<p><strong>Q: 次の文の（　）に入る言葉と最も意味が近いものを、1～4の中から一つ選びなさい。休日には（自然）の中で散歩するのが好きです。</strong></p>
<ul>
<li>人工</li>
<li><b>天然</b> <span style='color:green;'>(correct)</span></li>
<li>文化</li>
<li>都会</li>
</ul>
<p><em>Explanation: この文での「自然」は「人の手が加わっていない、ありのままの環境」という意味です。「天然」も「自然にできたもの、人が作ったものではない」という意味で、最も近いです。「人工」は「人が作ったもの」、「文化」は人間が作り出したものや活動、「都会」は都市のことなので、文脈から外れます。N3受験者は「自然」と「天然」の違いを正確に理解するのが難しい場合がありますが、この文脈では「自然＝天然の環境」と推測できるようにしました。「人工」「文化」「都会」は意味が異なるため、混乱しにくいですが、似た意味の語彙（天然）と対比語（人工）を含めることで、語彙力と読解力の両方を測ることができます。</em></p>
<h2>語彙使い方 (word_usage)</h2>
<p>5問。語彙の使い方として正しい文を選ぶ。</p>
<h3>保存</h3>
<p><strong>Q: 次の文の中で、「保存」の用法として最も適切なものを一つ選びなさい。①　この美術館では、貴重な絵画を長い間きれいなまま保存しています。②　昨日のパーティーで使ったお皿を保存してから、もう一度洗いました。③　友達に手紙を保存したので、すぐに読んで返事を書きました。④　新しい靴を保存して、明日から毎日履くことにします。⑤　パソコンで作ったファイルをUSBに保存しておきました。</strong></p>
<ul>
<li>①　この美術館では、貴重な絵画を長い間きれいなまま保存しています。</li>
<li>②　昨日のパーティーで使ったお皿を保存してから、もう一度洗いました。</li>
<li>③　友達に手紙を保存したので、すぐに読んで返事を書きました。</li>
<li>④　新しい靴を保存して、明日から毎日履くことにします。</li>
<li>⑤　パソコンで作ったファイルをUSBに保存しておきました。</li>
</ul>
<p><em>Explanation: 「保存」は、物やデータなどの状態を変えずに長く保つ場合に使います。①は、美術館で絵画を良い状態で保っている文脈で、最も基本的な使い方です。⑤は、IT分野で「データを保存する」という現代的な使い方で、N3レベルでも重要な例です。②や④は「保管」「しまう」が自然な表現です。③は、手紙を「保存する」という表現も場合によっては使えますが、「すぐに読む」という文脈では不自然です。</em></p>
<h3>拍手</h3>
<p><strong>Q: 次の文の中で、「拍手（はくしゅ）」の使い方として最も適切なものをすべて選びなさい。</strong></p>
<ul>
<li>先生が教室に入ってきたとき、みんなで拍手しました。</li>
<li>昨日の晩ご飯は、とてもおいしくて家族で拍手しました。</li>
<li>神社でお参りするとき、まず拍手（はくしゅ）をしてからお願いごとをします。</li>
<li>友達の誕生日に、みんなで拍手をしながら歌を歌いました。</li>
</ul>
<p><em>Explanation: 1は、先生が入ってきたときに歓迎や感謝の気持ちを表すために「拍手」する、自然な使い方です。4は、誕生日のお祝いの場面で、みんなで歌いながら「拍手」するのもよくある使い方です。2は、「拍手」が賞賛や歓迎、祝福の場面で使われるのが一般的であり、食事が美味しかっただけで家族で拍手するのは、特別なイベント（例：料理コンテストやパーティ）を除き、日常的な日本の習慣としては自然ではありません。3は、神社で手を打つ行為は「柏手（かしわで）」と呼ばれ、神道の作法です。一般的な「拍手」とは目的や意味が異なります。日本語の文脈では「柏手」と「拍手」を区別する必要があり、ここで「拍手」を使うのは不適切です。</em></p>
<h3>要点</h3>
<p><strong>Q: 次の文のうち、「要点」の使い方として最も適切なものをすべて選びなさい。</strong></p>
<ul>
<li>この話の要点をまとめてください。</li>
<li>今朝のニュースの要点をノートに書きました。</li>
<li>彼女のスピーチは長かったが、要点はよく伝わった。</li>
<li>家の要点をきれいに掃除してください。</li>
</ul>
<p><em>Explanation: 「要点」は「物事の中心となるところ、重要な点」という意味で、話・ニュース・スピーチなど抽象的内容の大切な部分に使います。1、2、3は正しい使い方ですが、4は「家」など物理的な対象には使いません。</em></p>
<h3>礼儀</h3>
<p><strong>Q: 次の文の中で、「礼儀」の正しい使い方はどれですか。</strong></p>
<ul>
<li><b>彼は礼儀が正しいので、誰に対してもていねいに話します。</b> <span style='color:green;'>(correct)</span></li>
<li>この料理は礼儀が強くて、少し苦い味がします。</li>
<li>昨日の会議では、礼儀を使ってパソコンの問題を解決しました。</li>
<li>山田さんは礼儀を集めるのが趣味で、たくさん持っています。</li>
</ul>
<p><em>Explanation: この問題の主な難しさは、「礼儀」という語が人間関係や社会生活で守るべき行動様式やマナーを指すことを理解しているかどうかです。2～4の選択肢は、「礼儀」という語の意味から外れています（2は味に使う表現、3は道具のように使っている、4は物を集める趣味のように使っている）。「礼儀が正しい」は日本語の固定表現であり、N3レベルの学習者が正しい文脈で使えるかを問う良問です。</em></p>
<h3>順調</h3>
<p><strong>Q: 次の文の中で、「順調」の使い方として最も適切なものを一つ選びなさい。</strong></p>
<ul>
<li><b>新しいプロジェクトは、今のところ順調に進んでいます。</b> <span style='color:green;'>(correct)</span></li>
<li>今日は順調な天気なので、ピクニックに行きましょう。</li>
<li>彼から順調な返事をもらって、安心しました。</li>
<li>この店は順調な料理がたくさんあります。</li>
</ul>
<p><em>Explanation: 「順調」は「物事が予定通りに、問題なく進むこと」を意味します。特に「順調に進む」「順調な経過」など、仕事や計画、進行状況に対して使われる表現です。1は「プロジェクトが順調に進む」という、もっとも一般的で自然な使い方です。2の「順調な天気」は不自然で、「天気がいい」「天候に恵まれて」などが自然です。3の「順調な返事」も不自然で、「返事が早い」「返事がスムーズ」などが適切です。4の「順調な料理」という表現は日本語として使いません。「順調」は「進む」「経過」「回復」「推移」など、進行や状態の良さを表す名詞とよく組み合わせて使います。物や天気、返事などには直接使わないので、コロケーション（よく使われる言葉の組み合わせ）に注意しましょう。</em></p>
<h2>文法穴埋め (sentence_grammar)</h2>
<p>13問。空欄に入る最も適切な語句や助詞を選ぶ。</p>
<h3>店で価格を尋ねる</h3>
<p><strong>Q: （店で）次の会話の空欄に入る最も適切な語を、下の1～4から一つ選びなさい。客：「すみません、この商品の値段を（　　　）いただけますか。」</strong></p>
<ul>
<li><b>教えて</b> <span style='color:green;'>(correct)</span></li>
<li>話して</li>
<li>聞いて</li>
<li>書いて</li>
</ul>
<p><em>Explanation: この文では、「～ていただけますか」という丁寧な依頼表現を使っています。「教えていただけますか」は「教える」の「て形」＋「いただけますか」で、「教えてください」よりも丁寧な表現です。「話して」や「聞いて」「書いて」は意味が合わず、不自然です。「教えて」を使うことで、店員に商品の値段を丁寧に尋ねることができます。N3レベルでは、「～ていただけますか」や「～てくれますか」などの依頼表現がよく問われます。正しい「て形」を使えるかどうかがポイントです。</em></p>
<h3>助詞</h3>
<p><strong>Q: 以下の文の（　）に入る最も適切な助詞を1つ選びなさい。私は毎朝7時（　）起きて、学校に行きます。</strong></p>
<ul>
<li><b>に</b> <span style='color:green;'>(correct)</span></li>
<li>で</li>
<li>へ</li>
<li>と</li>
</ul>
<p><em>Explanation: 「に」は特定の時刻や場所、目的地を表す格助詞として使われます。この文では「7時」という特定の時刻に起きることを表しているため、「に」が正しいです。他の選択肢について：- 「で」は場所や手段を表します（例：教室で勉強する）。- 「へ」は方向・目的地を示します（例：学校へ行く）。- 「と」は「一緒に」などの意味で使います（例：友達と話す）。類似例文：- 毎晩10時に寝ます。- 8時に家を出ます。このように、時刻を表す場合は必ず「に」を使うことを覚えましょう。</em></p>
<h3>キャリア目標について話す</h3>
<p><strong>Q: （面接練習） 面接官：「あなたのキャリア目標について教えてください。」 あなた：「はい、私のキャリア目標は、3年後にはプロジェクトリーダーとしてチームをまとめ、会社の売上に貢献できるようになることです。そのために、今は業務に必要な知識やスキルを（　　　　　）、毎日勉強しています。」 （　　）に入る最も適切な語句を選びなさい。</strong></p>
<ul>
<li>身につけるようにして</li>
<li><b>身につけるために</b> <span style='color:green;'>(correct)</span></li>
<li>身につけて</li>
<li>身につけたあとで</li>
</ul>
<p><em>Explanation: この文では、「目的」を表す表現が必要です。「～ために」は、「…する目的で」「…するために」という意味で、目標に向かって努力する姿勢を表します。他の選択肢は文脈に合いません。N3レベルでは、「～ために」の使い方を正確に理解し、目的を表す文を作れるかがポイントです。特に「ようにして」や「て」など、手段や継続・結果を表す表現との違いに注意が必要です。</em></p>
<h3>場合</h3>
<p><strong>Q: 次の文の（　　　）に入る最も適切な語句を、1～4の中から一つ選びなさい。（会社の案内）「火事や地震が起こった（　　　）、落ち着いて行動してください。」</strong></p>
<ul>
<li>こと</li>
<li><b>場合</b> <span style='color:green;'>(correct)</span></li>
<li>時</li>
<li>もの</li>
</ul>
<p><em>Explanation: 「場合」は「もし～のような状況が起きたとき」という意味で、公式な案内や指示文でよく使われます。「時」も似た意味ですが、より一般的な表現であり、「場合」の方が条件や状況を強調できます。「こと」「もの」はこの文脈には合いません。</em></p>
<h3>程度</h3>
<p><strong>Q: 先生：「みなさんの日本語の（　　　）は、前よりずっと上がっていますね。」この文に入る最も適切な言葉はどれですか。</strong></p>
<ul>
<li><b>程度</b> <span style='color:green;'>(correct)</span></li>
<li>具合</li>
<li>様子</li>
<li>状態</li>
</ul>
<p><em>Explanation: この文では、「みなさんの日本語の（　　　）は、前よりずっと上がっていますね。」という表現から、何がどのくらい上がったのかを問うています。「程度」は「レベル」や「どのくらいか」という意味で、能力や知識の高さを表すときによく使います。他の選択肢「具合」「様子」「状態」は、体調や状況、見た目などを表す言葉であり、「日本語の（　　　）が上がる」という文脈には適しません。この問題の主なチャレンジは、「程度」と他の似た意味の語彙（具合、様子、状態）との違いを理解し、正しく使い分けることです。N3レベルの学習者にとっては、抽象的な語彙の使い分けが難しいポイントとなります。</em></p>
<h3>状態</h3>
<p><strong>Q: （病院にて）医者：「こんにちは。最近、体の（　　　）はどうですか。どこか痛いところはありませんか？」</strong></p>
<ul>
<li><b>具合</b> <span style='color:green;'>(correct)</span></li>
<li>状態</li>
<li>事情</li>
<li>理由</li>
</ul>
<p><em>Explanation: 「体の具合」は、医者や家族が体調を尋ねるときによく使われる自然な表現です。日常会話や医療現場では、「体の具合が悪い」「具合はどうですか」のように使われます。「状態」も意味は通じますが、より医学的・客観的なニュアンスが強く、診断書や報告書などで使われる傾向があります。N3レベルでは、会話の場面では「具合」を優先して使うことを学びましょう。使い分け例- 「最近、体の具合はどうですか？」（日常会話・医者が患者に）- 「患者の健康状態を記録します。」（報告・記録などやや硬い表現）この問題の主な難しさは、「具合」と「状態」の意味の違いと使い分けを理解することにあります。どちらもN3レベルで学ぶ重要な語彙ですが、場面によって適切な語を選ぶ力が求められます。</em></p>
<h3>内容</h3>
<p><strong>Q: 【指示】次の会話文を読んで、（　　　）に入る最も適切な語を1つ選びなさい。（会議にて）部長：「この企画書の（　　　）を、もう一度確認してから提出してください。」</strong></p>
<ul>
<li><b>内容</b> <span style='color:green;'>(correct)</span></li>
<li>形式</li>
<li>理由</li>
<li>結果</li>
</ul>
<p><em>Explanation: 「内容」は「中身」や「具体的な情報」という意味で、文書や企画書などの「何が書かれているか」を指します。「形式」は「書き方」や「フォーマット」、「理由」は「なぜそうしたか」、「結果」は「どうなったか」を表します。この文脈では、「企画書の内容を確認する」が最も自然です。例文：- レポートの内容を確認してください。- テストの形式が変わりました。</em></p>
<h3>判断</h3>
<p><strong>Q: （　　　）に入る正しい言葉を選びなさい。（会議室）部長：「このプロジェクトを続けるかどうかは、今週中に（　　　）してください。」</strong></p>
<ul>
<li><b>判断して</b> <span style='color:green;'>(correct)</span></li>
<li>判断し</li>
<li>判断する</li>
<li>判断した</li>
</ul>
<p><em>Explanation: 「〜してください」という依頼表現では、動詞の「て形」が必要です。「判断する」の「て形」は「判断して」なので、正解は1番です。他の選択肢は依頼の文型として使えません。学習者がつまずきやすいポイント：サ変動詞「判断する」の「て形」活用に注意が必要です。「て形＋ください」の依頼表現は頻出ですが、語幹や「し」「する」「した」と混同しやすいので、正しい活用を確認しましょう。補足例文：早く返事してください。よく考えてください。注意してください。</em></p>
<h3>条件</h3>
<p><strong>Q: 部長：「このプロジェクトに参加したい人は、英語が話せることが（　）です。」</strong></p>
<ul>
<li><b>条件</b> <span style='color:green;'>(correct)</span></li>
<li>理由</li>
<li>方法</li>
<li>目的</li>
</ul>
<p><em>Explanation: この文では、「英語が話せること」が「プロジェクトに参加したい人」にとって必要な事項であることを表しています。「条件」は、あることが成立するために必要な事柄を指します。他の選択肢「理由」「方法」「目的」は文脈に合いません。N3レベルの学習者にとって、「条件」という語の意味や使い方、特に「～が条件です」という表現に慣れることがポイントです。また、日常会話やビジネスの場面でよく使われる表現なので、語彙の定着が大切です。</em></p>
<h3>関する</h3>
<p><strong>Q: この説明会は、留学生の生活（　　　　　）重要な情報を提供します。</strong></p>
<ul>
<li><b>に関する</b> <span style='color:green;'>(correct)</span></li>
<li>に対する</li>
<li>について</li>
<li>に関して</li>
</ul>
<p><em>Explanation: 「～に関する＋名詞」は、「～に関連する」という意味で、公式な文書や案内などでよく使われる硬い表現です。「～について」は「～についての＋名詞」の形で使い、より口語的・柔らかい印象になります。例：留学生の生活についての説明（やや口語的）、日本の文化についての本（一般的な使い方）「に対する」は「Aに対するB」（Aに向けたB）となり、対象を示します。例：留学生に対する支援「に関して」は「に関して話す」など動詞を修飾する場合によく使われますが、名詞を直接修飾する場合は「に関する」の方が一般的です。ただし、口語では「に関しての情報」も使われます。N3レベルの学習者は「に関する」と「について」の違いでよく混乱します。特に、「に関する＋名詞」と「についての＋名詞」の使い分けがポイントです。また、「に対する」や「に関して」など似た表現も多いため、文脈に合った選択ができるかが問われます。</em></p>
<h3>比較</h3>
<p><strong>Q: 次の文の（　）に入る最も適切な語句を１～４の中から一つ選びなさい。私の家の冷蔵庫は、新しい冷蔵庫（　）音が静かです。</strong></p>
<ul>
<li><b>より</b> <span style='color:green;'>(correct)</span></li>
<li>しか</li>
<li>だけ</li>
<li>でも</li>
</ul>
<p><em>Explanation: この文は「比較」の文型を問う問題です。「AはBより～です」は「A is more ~ than B」という意味で、N3レベルで頻出の表現です。「新しい冷蔵庫より、私の家の冷蔵庫の方が音が静かです」と主語を明示すると、比較の関係がより分かりやすくなります。「より」は比較対象を示す助詞で、他の助詞（「しか」「だけ」「でも」）はこの文脈で使えません。学習者は、これらの助詞の使い分けをよく練習する必要があります。特に「より」と「だけ」などは混同しやすいので、注意しましょう。</em></p>
<h3>場合</h3>
<p><strong>Q: （　）に入る最も適切な語を選びなさい。（教室にて）先生：「授業中に体調が悪くなった（　　　）、すぐに私に言ってください。」</strong></p>
<ul>
<li>とき</li>
<li><b>場合</b> <span style='color:green;'>(correct)</span></li>
<li>もの</li>
<li>こと</li>
</ul>
<p><em>Explanation: 「場合」は、ある条件や仮定が成立したときに、そのときどうするかを述べるときに使う表現です。「〜場合（は）」の形で、もしそのような状況になったら、という意味になります。一方、「とき」は単に特定の時間や瞬間を表しますが、「場合」は条件や仮定を強調したいときに使います。この文では、「授業中に体調が悪くなったとき」というよりも、「もしそのような状況になったら」という条件を伝えたいので、「場合」が最も適切です。</em></p>
<h3>原因</h3>
<p><strong>Q: （病院にて）医者：「最近、体の調子が悪いそうですね。どんな（　）で来院されましたか。」</strong></p>
<ul>
<li><b>原因（げんいん）</b> <span style='color:green;'>(correct)</span></li>
<li>理由（りゆう）</li>
<li>症状（しょうじょう）</li>
<li>動機（どうき）</li>
</ul>
<p><em>Explanation: この文では「どんな（　）で来院されましたか。」と、患者が病院に来たきっかけやもとになったことを尋ねています。「原因」は「物事を引き起こすもと」という意味で、体調不良や病気の「もと」になった出来事や状態を表すときに使います。「理由」は「自分の考えや選択の説明」に使うことが多く、例えば「どうしてその病院を選びましたか」という質問には合いますが、体の不調の「もと」を聞く場合には自然ではありません。「症状」は「体にどんな変化や異常があるか」を表す語で、「どんな症状がありますか」という使い方になります。「動機」は「行動を起こす気持ちや目的」を表し、主に犯罪や就職活動などの文脈で使われ、「病院に来た動機」とはあまり言いません。したがって、この文脈では「原因」が最も適切な語です。N3レベルの学習者は「原因」と「理由」の違いを混同しやすいです。また、「症状」や「動機」といった似た語彙も出題することで、より深い語彙理解と正しい使い分けが求められます。文脈をしっかり読む力と、語彙の意味の違いを意識させることがポイントです。</em></p>
<h2>文の並べ替え (sentence_sort)</h2>
<p>5問。バラバラになった文を正しい順番に並べる。</p>
<h3>感謝</h3>
<p><strong>Q: 次の文の★の部分にくる文を、正しい順序に並べかえてください。先生に__★_、私は日本語の勉強を続けることができました。</strong></p>
<ul>
<li><b>1. 助けていただいたおかげで</b> <span style='color:green;'>(correct)</span></li>
<li>2. 日本語の勉強を</li>
<li>3. 続けることができました</li>
<li>4. 私は</li>
</ul>
<p><em>Explanation: 「助けていただいたおかげで」は「原因・理由」を表し、主節（私は日本語の勉強を続けることができました）の前に置くのが自然です。「私は日本語の勉強を続けることができました」という主節の語順もN3レベルでよく問われます。この問題のポイントは、「おかげで」の前に感謝の対象となる行為（助けていただいた）が来て、主節（結果）が後ろに来るという日本語の自然な語順を理解することです。</em></p>
<h3>決心</h3>
<p><strong>Q: 次の文の__★__に入る最も適切なものを、下の1～4から一つ選びなさい。大学を卒業したら、海外で働くことを__★__。</strong></p>
<ul>
<li><b>決心した</b> <span style='color:green;'>(correct)</span></li>
<li>した決心</li>
<li>決心する</li>
<li>する決心</li>
</ul>
<p><em>Explanation: 「決心する」は「～することを決める」という意味の動詞です。「～ことを決心した」の形で「～することを決めた」となり、自然な日本語になります。2や4の語順は不自然で、3は時制が合いません（現在形）。時制や語順に注意しましょう。</em></p>
<h3>説明</h3>
<p><strong>Q: 次の文の★に入る文節を、自然な順番に並べてください。先生は、__★_ ので、もっとゆっくり話してくださいと言いました。1. 説明が 2. 少し 3. 分かりにくい 4. ときがある</strong></p>
<ul>
<li><b>1 → 2 → 3 → 4</b> <span style='color:green;'>(correct)</span></li>
<li>1 → 3 → 2 → 4</li>
<li>2 → 1 → 3 → 4</li>
<li>1 → 4 → 2 → 3</li>
</ul>
<p><em>Explanation: この問題のポイントは、「少し」が「分かりにくい」を修飾し、「ときがある」が全体をまとめる表現となることです。「説明が少し分かりにくいときがある」で、「先生の説明が時々少し分かりにくいことがある」という意味になります。日本語では修飾語（少し）は述語（分かりにくい）の前に置くのが自然です。また、「ときがある」は前の内容全体を受ける表現なので、最後に来ます。N3レベルでは、修飾の順序や文のまとまりを意識することが重要なポイントとなります。</em></p>
<h3>提案</h3>
<p><strong>Q: 下の「みんなで」「いくつか」「提案」「を」の４つの語（句）を正しい順番に並べて、文を完成させるのに最も適切なものを１～４から一つ選びなさい。</strong></p>
<ul>
<li>みんなで　提案　いくつか　を</li>
<li>いくつか　提案　を　みんなで</li>
<li>提案　いくつか　を　みんなで</li>
<li><b>みんなで　いくつか　提案　を</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: この問題の主なポイントは、日本語における修飾語と目的語の語順です。「いくつか」は数量を表し、「提案」を修飾します。「いくつかの提案をする」という形が自然です。また、「みんなで」は動作主を表す副詞句で、文頭に置くことで「みんなが一緒に」という意味が明確になります。助詞「を」は目的語「提案」に直接つきますので、「いくつか提案を」が正しい形です。N3レベルでは、「副詞句＋数量＋名詞＋を＋動詞」という基本的な文の構造を理解しているかどうかが問われます。誤った語順では意味が通じにくく、不自然な日本語になってしまいます。この問題は、助詞の使い方、副詞句の位置、数量表現の修飾関係など、複数の文法ポイントを同時に確認できる良問です。</em></p>
<h3>結論</h3>
<p><strong>Q: 次の文の（　★　）に入る最も自然な文になるように、1～4の語句を正しい順番に並べてください。このレポートの__★__、私たちは環境問題についてもっと考える必要があると分かった。1.　調査の　2.　結論として　3.　結果　4.　まとめると　A.　1→2→3→4　B.　2→1→3→4　C.　1→3→4→2　D.　2→1→4→3</strong></p>
<ul>
<li>A.　1→2→3→4</li>
<li><b>B.　2→1→3→4</b> <span style='color:green;'>(correct)</span></li>
<li>C.　1→3→4→2</li>
<li>D.　2→1→4→3</li>
</ul>
<p><em>Explanation: この問題の主な難しさは、「結論として」という接続語句と、「調査の結果」「まとめると」の使い方・順序を正しく理解することです。「結論として、調査の結果、まとめると」とつなげることで、論理的な文脈が自然になります。N3レベルの語彙を使いましたが、接続詞や名詞の連結に注意が必要です。「結論」と「結果」の違いと、文章の論理展開を理解する力が問われる問題です。</em></p>
<h2>文法構造選択 (sentence_structure)</h2>
<p>4問。文の構造として正しいものを選ぶ。</p>
<h3>原因・理由</h3>
ありがとうございます。ご指摘いただいた点を反映し、さらに学習者の理解を深めるため、問題と解説をより丁寧に修正いたします。---### 原因と理由の違い日本語には「原因」と「理由」という似ている言葉がありますが、使い方には違いがあります。「原因」は、何かが起こる**直接的なきっかけやもと**を表します。たとえば、「風邪をひいた原因はウイルスです」というように、悪い結果や出来事が起こる**もと**を説明するときに使います。一方、「理由」は、**行動や判断の根拠**や**背景**を表します。たとえば、「試験を受ける理由は、将来のためです」のように、なぜそうするのか、**自分の考えや事情**を説明するときに使います。「原因」は悪い結果に使うことが多いですが、「理由」は良いことにも悪いことにも使えます。もしどちらを使うか迷ったときは、「理由」を使うと安心です。---#### 次の文の（　）に入る最も適切なものを選びなさい。１　事故の（　）は、運転手の不注意でした。　１．理由　２．原因　３．目的　４．意味　２　私が日本語を勉強する（　）は、日本で働きたいからです。　１．原因　２．理由　３．方法　４．結果　３　台風が来た（　）、電車が止まりました。　１．ので　２．こと　３．とき　４．ため　４　「原因」と「理由」の違いについて（　）説明してください。　１．しっかり　２．ちゃんと　３．わかりやすく　４．すぐに---#### 正答と解説１　**２．原因**　「事故の原因」は決まった言い方で、直接的な理由やきっかけを表します。「理由」では不自然です。２　**２．理由**　「勉強する理由」は行動の根拠や動機を表すので「理由」がぴったりです。「原因」は使いません。３　**４．ため**　「台風が来たため、電車が止まりました」はN3でよく出る「〜ため」の使い方です。「ため」は書き言葉や正式な場面で使われることが多く、ニュースや説明文などで使われます。一方、「ので」も原因・理由を表しますが、より口語的で柔らかいニュアンスがあります。どちらも正しい文法ですが、ここでは公式な説明文として「ため」が最適です。４　**３．わかりやすく**　「わかりやすく説明してください」は、「説明」のしかたを最も適切に表す副詞です。「ちゃんと」「しっかり」も正しい副詞ですが、「きちんと説明する」という意味合いが強く、「説明の内容が理解しやすい」という意味では「わかりやすく」が最も自然です。---#### 教師からのポイント「原因」と「理由」の使い分けは、日本語学習者にとって混乱しやすいポイントです。「原因」は悪い結果のもと、「理由」は行動の動機や根拠という違いを例文で覚えましょう。また、「ため」と「ので」などの文型も、書き言葉・話し言葉の違いや場面による使い分けを意識すると、より自然な日本語表現が身につきます。---ご指摘いただいた点を反映し、より明確な解説と読みやすい形式に修正しました。今後もご意見を参考に、質の高い教材作成に努めてまいります。
<p><strong>Q: 事故の（　）は、運転手の不注意でした。</strong></p>
<ul>
<li>理由</li>
<li><b>原因</b> <span style='color:green;'>(correct)</span></li>
<li>目的</li>
<li>意味</li>
</ul>
<p><em>Explanation: 「事故の原因」は決まった言い方で、直接的な理由やきっかけを表します。「理由」では不自然です。</em></p>
<p><strong>Q: 私が日本語を勉強する（　）は、日本で働きたいからです。</strong></p>
<ul>
<li>原因</li>
<li><b>理由</b> <span style='color:green;'>(correct)</span></li>
<li>方法</li>
<li>結果</li>
</ul>
<p><em>Explanation: 「勉強する理由」は行動の根拠や動機を表すので「理由」がぴったりです。「原因」は使いません。</em></p>
<p><strong>Q: 台風が来た（　）、電車が止まりました。</strong></p>
<ul>
<li>ので</li>
<li>こと</li>
<li>とき</li>
<li><b>ため</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 「台風が来たため、電車が止まりました」はN3でよく出る「〜ため」の使い方です。「ため」は書き言葉や正式な場面で使われることが多く、ニュースや説明文などで使われます。一方、「ので」も原因・理由を表しますが、より口語的で柔らかいニュアンスがあります。どちらも正しい文法ですが、ここでは公式な説明文として「ため」が最適です。</em></p>
<p><strong>Q: 「原因」と「理由」の違いについて（　）説明してください。</strong></p>
<ul>
<li>しっかり</li>
<li>ちゃんと</li>
<li><b>わかりやすく</b> <span style='color:green;'>(correct)</span></li>
<li>すぐに</li>
</ul>
<p><em>Explanation: 「わかりやすく説明してください」は、「説明」のしかたを最も適切に表す副詞です。「ちゃんと」「しっかり」も正しい副詞ですが、「きちんと説明する」という意味合いが強く、「説明の内容が理解しやすい」という意味では「わかりやすく」が最も自然です。</em></p>
<h3>逆接</h3>
承知しました。ご指摘を踏まえ、より明確で受験者にとって分かりやすいJLPT N3レベルの逆接問題を再構成します。---## 次の文章の[A]〜[D]の空欄に入る最も適切な逆接の接続詞を、それぞれ下の選択肢から一つずつ選びなさい。---### 友達との旅行去年の春休みに、私は友達と京都に旅行に行きました。天気予報では、旅行の間ずっと晴れると言われていました。[A]京都に着いた日は、朝から雨が降っていました。雨が降ったので、予定していた観光地に行くかどうか、みんなで相談しました。[B]せっかく来たので、雨でも観光を楽しもうということになりました。私たちは傘をさして、有名なお寺や神社を回りました。[C]雨の中で見る桜はとてもきれいでした。写真を撮ると、普段よりも静かな雰囲気が伝わってきました。[D]雨の日の京都も、思っていたより楽しかったです。---### [A]に入る言葉を選びなさい。1. しかし 2. だから 3. それから 4. そして### [B]に入る言葉を選びなさい。1. けれども 2. しかし 3. でも 4. けれど### [C]に入る言葉を選びなさい。1. しかし 2. でも 3. けど 4. けれども### [D]に入る言葉を選びなさい。1. しかし 2. けど 3. けれど 4. でも---## 正答・解説### [A] 正解：1. しかし  解説：前の文で「晴れる」と言われていたが、実際には「雨が降った」ため、予想と異なる結果を表す逆接の「しかし」が最適です。### [B] 正解：1. けれども  解説：「雨が降ったので予定を相談した」→「せっかく来たので観光する」と、逆の行動を取ることを示す逆接。文中で使う「けれども」が自然です。### [C] 正解：2. でも  解説：「雨が降っている」という否定的な状況でも、「桜がきれいだった」と肯定的な内容を続けるため、「でも」が適切です。会話的な自然さもあります。### [D] 正解：3. けれど  解説：「普段と違う雨の日の京都」→「楽しかった」という予想外の感想を逆接で述べる。文末に使える「けれど」が自然です。---## 教師からのポイント- 逆接の接続詞は、文の位置やフォーマル度によって使い分ける必要があります。- 「しかし」は文頭、「けれども」は文中、「でも」は会話的、「けれど」は文末や柔らかい印象で使われます。- 文脈をしっかり読んで、どの逆接表現が最も自然かを考えましょう。---この形式で、JLPT N3受験者が逆接の使い分けをしっかりと練習できる内容となっています。
<p><strong>Q: [A]に入る言葉を選びなさい。</strong></p>
<ul>
<li><b>しかし</b> <span style='color:green;'>(correct)</span></li>
<li>だから</li>
<li>それから</li>
<li>そして</li>
</ul>
<p><em>Explanation: 前の文で「晴れる」と言われていたが、実際には「雨が降った」ため、予想と異なる結果を表す逆接の「しかし」が最適です。</em></p>
<p><strong>Q: [B]に入る言葉を選びなさい。</strong></p>
<ul>
<li><b>けれども</b> <span style='color:green;'>(correct)</span></li>
<li>しかし</li>
<li>でも</li>
<li>けれど</li>
</ul>
<p><em>Explanation: 「雨が降ったので予定を相談した」→「せっかく来たので観光する」と、逆の行動を取ることを示す逆接。文中で使う「けれども」が自然です。</em></p>
<p><strong>Q: [C]に入る言葉を選びなさい。</strong></p>
<ul>
<li>しかし</li>
<li><b>でも</b> <span style='color:green;'>(correct)</span></li>
<li>けど</li>
<li>けれども</li>
</ul>
<p><em>Explanation: 「雨が降っている」という否定的な状況でも、「桜がきれいだった」と肯定的な内容を続けるため、「でも」が適切です。会話的な自然さもあります。</em></p>
<p><strong>Q: [D]に入る言葉を選びなさい。</strong></p>
<ul>
<li>しかし</li>
<li>けど</li>
<li><b>けれど</b> <span style='color:green;'>(correct)</span></li>
<li>でも</li>
</ul>
<p><em>Explanation: 「普段と違う雨の日の京都」→「楽しかった」という予想外の感想を逆接で述べる。文末に使える「けれど」が自然です。</em></p>
<h3>条件</h3>
# 日本語記事：条件について 私たちの生活には、さまざまな「条件」があります。たとえば、アルバイトを[　1　]、年齢や経験が必要な場合があります。会社に入るための条件も、学歴や資格などいろいろです。また、天気がいい日[　2　]、運動会が行われます。もし雨が降ったら、運動会は中止になります。このように、何かが起こるためには「条件」が必要なことが多いです。 条件は、日常生活だけでなく、法律や約束にもよく使われます。たとえば、「宿題を全部終わらせたら、ゲームをしてもいい」という約束は、「宿題を終わらせること」がゲームをする[　3　]です。また、「この商品は、20歳以上の人だけが買うことができる」というのも、年齢が[　4　]です。 このように、条件は私たちの生活の中でとても大切なものです。条件をよく考えて行動することで、トラブルを防ぐことができます。
<p><strong>Q: 1. アルバイトを[　1　]、年齢や経験が必要な場合があります。</strong></p>
<ul>
<li>したいとき</li>
<li><b>するために</b> <span style='color:green;'>(correct)</span></li>
<li>してから</li>
<li>しないで</li>
</ul>
<p><em>Explanation: 「アルバイトをするために、年齢や経験が必要な場合があります。」が自然です。「するために」は目的や理由を表すN3文型で、条件を説明するのに適切です。他の選択肢は文脈に合いません。</em></p>
<p><strong>Q: 2. 天気がいい日[　2　]、運動会が行われます。</strong></p>
<ul>
<li><b>なら</b> <span style='color:green;'>(correct)</span></li>
<li>ので</li>
<li>でも</li>
<li>ほど</li>
</ul>
<p><em>Explanation: 「天気がいい日なら、運動会が行われます。」は仮定条件を表し、N3の「～なら」を使うのが正しいです。「ので」は理由、「でも」は逆接、「ほど」は程度を表すため不適切です。</em></p>
<p><strong>Q: 3. 「宿題を全部終わらせたら、ゲームをしてもいい」という約束は、「宿題を終わらせること」がゲームをする[　3　]です。</strong></p>
<ul>
<li>必要です</li>
<li>ことです</li>
<li>ものです</li>
<li><b>条件です</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 「宿題を終わらせること」がゲームをする条件です、という説明です。「条件です」と明確に言うことで文意が通ります。「必要です」や「ものです」では具体性が足りません。</em></p>
<p><strong>Q: 4. 「この商品は、20歳以上の人だけが買うことができる」というのも、年齢が[　4　]です。</strong></p>
<ul>
<li><b>条件です</b> <span style='color:green;'>(correct)</span></li>
<li>必要です</li>
<li>できます</li>
<li>あります</li>
</ul>
<p><em>Explanation: 「年齢が条件です。」が正しいです。「必要です」も近いですが、「条件です」の方がより明確にルールや制限を表します。</em></p>
<h3>目的</h3>
# 目的について 私たちが何か新しいことを始めるとき、**目的**を持つことはとても大切です。目的があると、[1] 何をすればいいのかがはっきりします。たとえば、勉強する目的が「日本語を話せるようになること」なら、毎日少しずつ単語や文法を覚えることが大切だとわかります。 目的は、[2] 目標や手段と違います。目的は「最終的に達成したいこと」、目標は「目的を達成するための小さなゴール」、手段は「目標を達成するための方法」です。たとえば、「健康になる」という目的のために、「毎日30分歩く」という目標を立てて、「朝早く起きて散歩する」という手段を選ぶことができます。 目的があると、途中で困難があっても、[3] 続ける力が生まれます。目的を忘れてしまうと、なぜ頑張るのか分からなくなってしまいます。だから、何かを始めるときは、まず自分の目的を[4] 明確にすることが大切です。 --- [1] 1. どこで 2. どうして 3. どのように 4. どれくらい  [2] 1. たとえば 2. しかし 3. それとも 4. つまり  [3] 1. あきらめやすくなる 2. がんばり続けられる 3. つかれてしまう 4. いやになる  [4] 1. しっかり 2. はっきり 3. きちんと 4. すぐに --- ### 正解と解説 [1] **3. どのように** 「何をすればいいのかがはっきりします」という内容なので、「どのように（how）」が最も自然です。N3受験者は疑問詞の使い分けをよく練習しましょう。 [2] **1. たとえば** 「目的は、目標や手段と違います。」の後に例を挙げるので、「たとえば」が適切です。「しかし」「それとも」「つまり」は文脈に合いません。 [3] **2. がんばり続けられる** 「続ける力が生まれます」とあるので、「がんばり続けられる」が一番合っています。あきらめる・つかれる・いやになるは逆の意味です。 [4] **2. はっきり** 「目的を明確にすることが大切です」に合うのは「はっきり」です。「しっかり」「きちんと」も意味が近いですが、「明確にする」と一番自然につながるのは「はっきり」です。「すぐに」は意味が違います。 --- **教師の視点からの主な課題と指導アドバイス** この問題では、疑問詞や接続詞、副詞の微妙な使い分けがポイントです。N3学習者は、似た意味の語彙の使い分けや、文全体の流れを理解する力をつける必要があります。  **指導アドバイス:**  - 疑問詞や副詞は例文を多く読み、実際の文脈でどのように使われているかを確認しましょう。  - 接続詞や副詞は、意味の近い言葉を比較しながら練習問題を解くと効果的です。  - 「目的」「目標」「手段」など似た言葉は、図や表を使って整理し、具体例を挙げて理解を深めましょう。  --- 【模範解答例の改善点を反映し、選択肢の質や表記も整えました】
<p><strong>Q: [1] 何をすればいいのかがはっきりします。に入る最も適切な言葉はどれですか？</strong></p>
<ul>
<li>どこで</li>
<li>どうして</li>
<li><b>どのように</b> <span style='color:green;'>(correct)</span></li>
<li>どれくらい</li>
</ul>
<p><em>Explanation: 「何をすればいいのかがはっきりします」という内容なので、「どのように（how）」が最も自然です。N3受験者は疑問詞の使い分けをよく練習しましょう。</em></p>
<p><strong>Q: [2] 目的は、目標や手段と違います。の後に続く最も適切な接続詞はどれですか？</strong></p>
<ul>
<li><b>たとえば</b> <span style='color:green;'>(correct)</span></li>
<li>しかし</li>
<li>それとも</li>
<li>つまり</li>
</ul>
<p><em>Explanation: 「目的は、目標や手段と違います。」の後に例を挙げるので、「たとえば」が適切です。「しかし」「それとも」「つまり」は文脈に合いません。</em></p>
<p><strong>Q: [3] 途中で困難があっても、続ける力が生まれます。に入る最も適切な言葉はどれですか？</strong></p>
<ul>
<li>あきらめやすくなる</li>
<li><b>がんばり続けられる</b> <span style='color:green;'>(correct)</span></li>
<li>つかれてしまう</li>
<li>いやになる</li>
</ul>
<p><em>Explanation: 「続ける力が生まれます」とあるので、「がんばり続けられる」が一番合っています。あきらめる・つかれる・いやになるは逆の意味です。</em></p>
<p><strong>Q: [4] まず自分の目的を明確にすることが大切です。に入る最も適切な副詞はどれですか？</strong></p>
<ul>
<li>しっかり</li>
<li><b>はっきり</b> <span style='color:green;'>(correct)</span></li>
<li>きちんと</li>
<li>すぐに</li>
</ul>
<p><em>Explanation: 「目的を明確にすることが大切です」に合うのは「はっきり」です。「しっかり」「きちんと」も意味が近いですが、「明確にする」と一番自然につながるのは「はっきり」です。「すぐに」は意味が違います。</em></p>
<h1>第2部：読解</h1>
<h2>短文読解 (short_passage_read)</h2>
<p>4題。短い文章を読み、内容に関する問いに答える。</p>
<h3>店で価格を尋ねる</h3>
<p><strong>Q: このメールで、山田様は何を知ることができましたか。</strong></p>
<ul>
<li><b>冷蔵庫の価格と配送料について</b> <span style='color:green;'>(correct)</span></li>
<li>冷蔵庫の在庫状況について</li>
<li>冷蔵庫の色について</li>
<li>冷蔵庫のサイズについて</li>
</ul>
<p><em>Explanation: この問題では、ビジネスメールの内容から必要な情報を正確に読み取る力が求められます。N3レベルの学習者にとって、「価格」「配送料」「ご案内」などの語彙や敬語表現の理解がポイントです。選択肢には本文にない情報（在庫状況・色・サイズ）が含まれており、正しく内容を把握できているか判断できます。メール形式や敬称も自然で、指示文も加わっているため、受験者が迷わず解答できる構成になっています。</em></p>
<h3>レストランで食べ物を注文する</h3>
<p><strong>Q: 山本さんは、レストランでどのように食事を楽しみましたか。</strong></p>
<ul>
<li>自分の料理だけを食べた</li>
<li><b>田中さんの料理も少し食べた</b> <span style='color:green;'>(correct)</span></li>
<li>デザートをたくさん食べた</li>
<li>田中さんの注文を手伝った</li>
</ul>
<p><em>Explanation: この問題の難しさは、メール本文から「田中さんが選んだスープも少しもらえて、うれしかったです。」という部分を正しく理解し、「自分の注文以外も食べた」ことを読み取る点にあります。N3レベルでは「もらえて」「うれしかった」などの表現や、「自分の料理だけ」や「デザートをたくさん」などの選択肢の違いに注意することが必要です。選択肢を丁寧に比較し、本文と合致する内容を選ぶ力が問われます。</em></p>
<h3>道を尋ねる</h3>
<p><strong>Q: 次のメールを読んで、後の問いに答えなさい。これは、山田さんが友人のスミスさんに送ったメールです。スミスさんこんにちは。メールをありがとう。東京に来る日は決まりましたか。もしよかったら、東京駅まで迎えに行きます。東京駅はとても広いので、待ち合わせの場所を決めておきましょう。丸の内北口の改札の前でどうですか。駅で道に迷ったら、近くの駅員さんに「丸の内北口はどこですか」と聞いてください。当日は、私の携帯に電話してもいいですよ。会えるのを楽しみにしています。山田このメールで、山田さんはスミスさんにどんなアドバイスをしていますか。最も適切なものを一つ選びなさい。</strong></p>
<ul>
<li><b>東京駅で迷ったら、駅員に聞くように言っている。</b> <span style='color:green;'>(correct)</span></li>
<li>東京駅に着いたら、すぐに電話するように言っている。</li>
<li>東京駅の南口で待ち合わせるように言っている。</li>
<li>東京駅の近くのホテルに泊まるように言っている。</li>
</ul>
<p><em>Explanation: この問題は、メール本文の内容を正確に読み取り、「道を尋ねる」というN3レベルの重要表現を理解できているかを問うものです。本文中に「駅で道に迷ったら、近くの駅員さんに『丸の内北口はどこですか』と聞いてください。」と明記されています。したがって、1が正解です。他の選択肢は、本文に直接書かれていない内容や、本文と異なる情報です。2は「すぐに電話」とは書かれておらず、3は「南口」ではなく「北口」、4はホテルについての記述がありません。誤答も本文の内容に近い表現を使うことで、受験者の本文理解力をより深く測ることができます。この問題の主なチャレンジは、「迷ったとき」「駅員に聞く」「待ち合わせ場所」といった日常的な日本語のやりとりを正確に理解し、適切な選択肢を選ぶ力です。N3レベルの語彙と文法がしっかり使われており、実用的な内容となっています。</em></p>
<h3>週末の予定について話す</h3>
<p><strong>Q: 田中さんは、日曜日に何をする予定ですか。</strong></p>
<ul>
<li>家で本を読む</li>
<li>スーパーで買い物をする</li>
<li><b>友達と映画を見に行く</b> <span style='color:green;'>(correct)</span></li>
<li>家族とカフェに行く</li>
</ul>
<p><em>Explanation: この問題のポイントは、日付と予定を正確に読み取る力です。N3レベルでは、「〜つもりです」「〜予定です」など未来の表現や、日付ごとの行動を区別する読解力が求められます。「日曜日は、友達と映画を見に行く予定です。」という文を正しく理解できるかがカギです。選択肢の中には土曜日の予定や、本文にない内容が含まれているので、惑わされずに本文から直接答えを見つけることが大切です。</em></p>
<h2>中文章読解 (midsize_passage_read)</h2>
<p>2題。中程度の長さの文章を読み、主旨や詳細情報について答える。</p>
<h3>趣味について話す</h3>
<p><strong>Q: この文章を書いた人は、旅行についてどのように考えていますか。</strong></p>
<ul>
<li>旅行は計画が大変なので、あまり好きではない。</li>
<li><b>旅行を通じて新しい発見や経験ができるので楽しいと思っている。</b> <span style='color:green;'>(correct)</span></li>
<li>旅行は友達と行くのが一番大切だと思っている。</li>
<li>旅行は国内だけで十分だと思っている。</li>
</ul>
<p><em>Explanation: 本文では、「その土地の文化や食べ物を知ることができて、とても楽しいです」や、「旅行を通して、自分の世界が広がったと感じています」と書かれており、旅行を通じて新しい発見や経験ができることを楽しいと感じていることが読み取れます。選択肢1や4は本文と反対の内容であり、3も「友達と行くこともありますが、一人で…」とあるため、友達と行くことだけが大切とは考えていません。N3学習者にとっては、筆者の気持ちや考えを読み取る力、キーワード（「楽しい」「広がった」など）を正確に把握する力が問われる点が難しいポイントです。</em></p>
<h3>家族について話す</h3>
<p><strong>Q: 家族について書かれているこの文章から、正しいものを一つ選びなさい。</strong></p>
<ul>
<li>家族は四人で、姉が二人いる。</li>
<li>父は家で家事をしている。</li>
<li><b>家族は毎週日曜日に一緒に夕食を食べる。</b> <span style='color:green;'>(correct)</span></li>
<li>弟は高校生で、バスケが好きだ。</li>
</ul>
<p><em>Explanation: この問題の主なチャレンジは、家族構成や家族の活動、職業、趣味など、N3レベルの基礎的な語彙と文法を正しく読み取ることです。選択肢の中には、本文の内容と少し違う情報（例えば人数や職業、趣味）が混ざっているため、細かい情報を正確に把握する読解力が求められます。特に家族構成や日常の習慣に関する記述をしっかり確認することがポイントです。</em></p>
<h2>長文読解 (long_passage_read)</h2>
<p>1題。やや長い文章を読み、全体の内容や筆者の意図を理解する。</p>
<h3>旅行の計画について話す</h3>
【次の文章を読んで、質問に答えなさい】田中さんは、来月友達と大阪に旅行に行く予定です。先週、田中さんは旅行の計画を立てるために、友達と集まりました。まず、みんなでどこに行きたいかを話し合いました。大阪城やユニバーサル・スタジオ・ジャパン、道頓堀など、人気の観光地がたくさんあります。田中さんは、歴史が好きなので、大阪城に一番行きたいと言いました。友達の一人は、テーマパークが好きなので、ユニバーサル・スタジオ・ジャパンに行きたいと言いました。みんなで相談して、二日目は大阪城、三日目はユニバーサル・スタジオ・ジャパンに行くことに決めました。ホテルは駅の近くに予約しました。駅から観光地まで電車で行けるので、とても便利です。また、旅行中に美味しいたこ焼きを食べることも楽しみにしています。田中さんたちは、旅行の日が来るのを楽しみにしています。
<p><strong>Q: 田中さんは旅行の計画を立てるために、何をしましたか。</strong></p>
<ul>
<li><b>友達と集まって、旅行の計画を立てました。</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 主語の省略と「集まる」「計画を立てる」の文脈理解が必要。N3では複数人の行動や目的を正確に読み取ることが大切です。</em></p>
<p><strong>Q: 田中さんたちは大阪でどんな観光地に行くことにしましたか。</strong></p>
<ul>
<li><b>大阪城とユニバーサル・スタジオ・ジャパンに行くことにしました。</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 複数の観光地が出てくるため、情報を整理して答える必要があります。N3では「相談して決めた」などの文型もポイント。</em></p>
<p><strong>Q: 田中さんたちがホテルを予約した場所はどこですか。</strong></p>
<ul>
<li><b>駅の近くにホテルを予約しました。</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 「場所＋に予約する」「駅の近く」など、場所を表す表現に注意しましょう。</em></p>
<p><strong>Q: 田中さんはなぜ大阪城に行きたいと言いましたか。</strong></p>
<ul>
<li><b>歴史が好きだからです。</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 「ので」や「から」による理由説明を読み取る力が問われます。</em></p>
<p><strong>Q: みんなはどのようにして行く場所を決めましたか。</strong></p>
<ul>
<li><b>みんなで相談して決めました。</b> <span style='color:green;'>(correct)</span></li>
</ul>
<p><em>Explanation: 「相談して決める」という表現を正しく理解できるかが重要です。</em></p>
<p><strong>Q: 田中さんたちが旅行で楽しみにしている食べ物はどれですか。</strong></p>
<ul>
<li>お好み焼き</li>
<li><b>たこ焼き</b> <span style='color:green;'>(correct)</span></li>
<li>ラーメン</li>
<li>カレー</li>
</ul>
<p><em>Explanation: 旅行中の楽しみや具体的な名詞の読み取り。N3では本文中の具体的な情報を正確に選ぶ力が必要です。</em></p>
<h2>情報検索 (info_retrieval)</h2>
<p>1題。案内や掲示などの情報から必要な情報を探し出す。</p>
<h3>電車の切符を買う</h3>
<h3>電車の切符の買い方と案内</h3><p>次の表と説明を読んで、後の質問に答えなさい。</p><table border="1" cellspacing="0" cellpadding="4" style="border-collapse:collapse;width:100%;text-align:center;"><tr style="background-color:#f0f0f0;"><th>購入場所</th><th>利用可能時間</th><th>支払い方法</th><th>できること</th></tr><tr><td>自動券売機</td><td>5:00～23:30</td><td>現金・ICカード</td><td>きっぷ購入、ICカードチャージ</td></tr><tr><td>窓口</td><td>7:00～20:00</td><td>現金・クレジットカード</td><td>新幹線・特急券も購入可</td></tr><tr><td>インターネット予約</td><td>24時間</td><td>クレジットカード</td><td>会員登録が必要、きっぷは駅で受け取り</td></tr><tr><td>旅行会社</td><td>10:00～18:00</td><td>現金</td><td>JRきっぷのみ、手数料なし</td></tr></table><ul><li>自動券売機は駅の入り口近くにあります。画面の案内に従って、行き先や人数を選び、お金を入れるときっぷが出ます。</li><li>ICカード（Suica、PASMOなど）は券売機でチャージできます。改札でタッチして使います。</li><li>特急や新幹線に乗る場合は、乗車券のほかに特急券や指定席券が必要です。窓口や高機能券売機、インターネットで購入できます。</li><li>インターネットで予約したきっぷは、出発当日または7日前から駅の指定席券売機で受け取ります。</li><li>旅行会社では、JRきっぷを手数料なしで購入できますが、店舗によっては取り扱いがない場合もあります。</li></ul><hr><p><b>【指示】</b> 次の質問に最も合うものを一つずつ選びなさい。</p>
<p><strong>Q: 駅でICカードにお金を入れたいとき、どこでできますか。</strong></p>
<ul>
<li>ア　窓口</li>
<li><b>イ　自動券売機</b> <span style='color:green;'>(correct)</span></li>
<li>ウ　インターネット予約</li>
<li>エ　旅行会社</li>
</ul>
<p><em>Explanation: 自動券売機は駅の入り口近くにあり、ICカードのチャージもできます。</em></p>
<p><strong>Q: インターネットで切符を予約した場合、どのように受け取りますか。</strong></p>
<ul>
<li>ア　家に郵送してもらう</li>
<li><b>イ　駅の指定席券売機で受け取る</b> <span style='color:green;'>(correct)</span></li>
<li>ウ　改札で駅員から受け取る</li>
<li>エ　旅行会社のカウンターで受け取る</li>
</ul>
<p><em>Explanation: インターネットで予約したきっぷは、出発当日または7日前から駅の指定席券売機で受け取ります。</em></p>
<p><strong>Q: 窓口でできないことはどれですか。</strong></p>
<ul>
<li>ア　新幹線の切符を買う</li>
<li><b>イ　ICカードにチャージする</b> <span style='color:green;'>(correct)</span></li>
<li>ウ　現金で支払う</li>
<li>エ　クレジットカードで支払う</li>
</ul>
<p><em>Explanation: 窓口ではICカードへのチャージはできません。</em></p>
<p><strong>Q: 旅行会社で切符を買うときに、正しいことはどれですか。</strong></p>
<ul>
<li>ア　手数料がかかる</li>
<li><b>イ　現金で支払う</b> <span style='color:green;'>(correct)</span></li>
<li>ウ　ICカードで支払う</li>
<li>エ　クレジットカードで支払う</li>
</ul>
<p><em>Explanation: 旅行会社ではJRきっぷのみ手数料なしで購入でき、支払いは現金のみです。</em></p>
<p><strong>Q: 特急に乗りたいとき、必要なものは何ですか。</strong></p>
<ul>
<li>ア　乗車券だけ</li>
<li>イ　特急券だけ</li>
<li><b>ウ　乗車券と特急券</b> <span style='color:green;'>(correct)</span></li>
<li>エ　ICカードだけ</li>
</ul>
<p><em>Explanation: 特急に乗る場合は乗車券と特急券の両方が必要です。</em></p>
<h1>第3部：聴解</h1>
<h2>話題理解 (topic_understanding)</h2>
<p>6問。流れる会話や説明から話題を把握する。</p>
<h3>お気に入りのレストランについて話す</h3>
<p><strong>Q: この後、ユキさんは何をしたいと思っていますか。</strong></p>
<ul>
<li>新しいラーメン屋に行く</li>
<li><b>森のカフェでチョコレートケーキを食べる</b> <span style='color:green;'>(correct)</span></li>
<li>ケンさんの家に行く</li>
<li>フルーツタルトを作る</li>
</ul>
<p><em>Explanation: この問題の難しさは、「お気に入り」「静か」「ケーキ」などN3レベルの話題語彙を聞き分け、会話の流れからユキさんの次の希望行動（＝お気に入りのカフェでチョコレートケーキを食べたい）を推測する点です。選択肢には似た内容（ケーキやフルーツタルト）が含まれているため、会話の具体的な内容を正確に聞き取る力が問われます。</em></p>
<h3>健康とフィットネスについて話す</h3>
<p><strong>Q: 友人Aはこの後、何をするつもりですか。</strong></p>
<ul>
<li>毎朝ジョギングを始める</li>
<li>食事のバランスを考える</li>
<li><b>明日から運動をしてみる</b> <span style='color:green;'>(correct)</span></li>
<li>プロテインを毎日飲む</li>
</ul>
<p><em>Explanation: この問題は、会話の最後で「じゃあ、私も明日から少し運動してみようかな」と友人Aが自分の行動を決めている点を聞き取る必要があります。会話の流れを理解し、実際に何をするつもりかを選ぶ力が求められます。選択肢1や2も会話に出てきますが、「明日から運動」と具体的に述べているのが3番です。N3レベルの語彙と表現を使い、会話の要点を聞き取る力を測る良問です。</em></p>
<h3>最近の映画について話す</h3>
<p><strong>Q: この会話を読んで、男の人がこの後一番したいことに近いものを一つ選びなさい。</strong></p>
<ul>
<li>アクション映画を見る</li>
<li>音楽を聞きに行く</li>
<li><b>感動する映画を見に行く</b> <span style='color:green;'>(correct)</span></li>
<li>高校に行く</li>
</ul>
<p><em>Explanation: この問題のポイントは、男の人の発言「たまにはそういう映画も見てみたいな」と、女の人の「今度一緒に見に行かない？」という誘いに「ぜひ行こう！」と答えている部分です。つまり、男の人は感動する映画（「青い空の約束」）を見に行くことに興味を持っています。N3レベルの語彙（映画館、感動、約束、俳優、演技、アクションなど）や表現（〜してみたい、〜ばかり、もしよかったら、ぜひ〜しよう）を適切に使用しています。選択肢には内容に関連するものや紛らわしいもの（アクション映画、音楽など）を含め、正しく会話内容を理解できているかを問う形式です。</em></p>
<h3>支払い方法について話す</h3>
<p><strong>Q: この後、お客さんはまず何をしますか。最も適切なものを一つ選びなさい。</strong></p>
<ul>
<li>現金で支払う</li>
<li>クレジットカードを渡す</li>
<li><b>レシートとポイントカードを受け取る</b> <span style='color:green;'>(correct)</span></li>
<li>店員に支払い方法について質問する</li>
</ul>
<p><em>Explanation: この問題は、支払いの流れや店員とのやりとりを正確に理解する力が求められます。N3レベルの語彙（「ICカード」「タッチ」「ポイントカード」「お返しします」など）を使い、実際の日本の買い物場面を再現しています。選択肢には、会話の流れから外れるもの（現金で支払う、クレジットカードを渡す、店員に質問する）を含めることで、細部まで聞き取る力を測ります。正解を選ぶには、最後の店員の発言「レシートとポイントカードもお返しします。」に注目し、次にお客さんが何をするかを推測する必要があります。</em></p>
<h3>家事の分担について話す</h3>
<p><strong>Q: この後、夫はまず何をしますか。</strong></p>
<ul>
<li><b>洗濯をする日を決める</b> <span style='color:green;'>(correct)</span></li>
<li>掃除をする時間を決める</li>
<li>土日の夕食のメニューを考える</li>
<li>妻に掃除のやり方を聞く</li>
</ul>
<p><em>Explanation: この会話では、妻が洗濯を週に2回夫にお願いし、夫が「じゃあ、洗濯は僕がやるよ」と約束しています。直後に必要な行動は、いつ洗濯をするか日を決めることです。選択肢2と4も家事分担に関係しますが、掃除は「週末ならできる」と話しているだけで、すぐに時間を決める流れではありません。また、選択肢3の夕食メニューについては「考えておくね」と言っていますが、まず優先されるのは妻の要望である洗濯の分担です。受験者は会話の流れと約束された内容を正確に把握する必要があり、N3レベルの聞き取り力と実生活に基づく判断力が問われます。</em></p>
<h3>公共施設の利用方法について話す</h3>
<p><strong>Q: 女の人はこれから何をしますか。</strong></p>
<ul>
<li>会議室の掃除をする</li>
<li><b>利用申し込みの紙を記入する</b> <span style='color:green;'>(correct)</span></li>
<li>会議室の利用料金をはらう</li>
<li>会議室の場所を案内する</li>
</ul>
<p><em>Explanation: この問題は、会話の流れを正確に理解し、次に行うべき行動を特定するリスニング力を測るものです。女の人は「申し込みを書きます」と最後に言っており、これから申し込み用紙に記入することが明らかです。選択肢2が正解となります。N3レベルの語彙と表現を使い、公共施設の利用手順という日常的な場面を設定しているので、実用的な日本語運用能力が問われます。設問と選択肢の整合性も保たれており、混乱が生じにくい良問です。</em></p>
<h2>要点理解 (keypoint_understanding)</h2>
<p>6問。会話の中の重要な情報や要点を聞き取る。</p>
<h3>仕事のプロジェクトについて話す</h3>
<p><strong>Q: 女の人がまず最初にすることは何ですか。</strong></p>
<ul>
<li>新しいメンバーを探すこと</li>
<li><b>業務の問題点を整理すること</b> <span style='color:green;'>(correct)</span></li>
<li>改善案をまとめること</li>
<li>結果を報告すること</li>
</ul>
<p><em>Explanation: この問題はN3レベルの語彙と文法を用いています。会話の中で「まずは現状の問題点を整理してから、改善案を考える予定」と述べられている部分を正確に理解できるかがポイントです。他の選択肢も会話内で言及されていますが、時系列や優先順位を正しく把握する力が求められます。選択肢の内容も混同しやすいため、注意深く聞き取る必要があります。</em></p>
<h3>おすすめを尋ねる</h3>
<p><strong>Q: 男の人は女の人にどこをすすめられましたか。</strong></p>
<ul>
<li>京都</li>
<li><b>箱根</b> <span style='color:green;'>(correct)</span></li>
<li>沖縄</li>
<li>北海道</li>
</ul>
<p><em>Explanation: この問題のポイントは、「箱根はどう？」という女の人の発言が、男の人へのおすすめであることを聞き取ることです。会話の中で温泉や美味しいもの、景色などの希望条件が出てきますが、「それなら、箱根はどう？」という提案が正答の根拠です。また、「特急で行ける」「温泉」「黒たまご」などの情報もヒントになります。N3レベルでは、会話の流れからどの場所がすすめられたのかを正確に理解する力が求められるため、設問として適切です。固有名詞（箱根、黒たまごなど）は文脈で理解できるよう工夫しています。</em></p>
<h3>交通状況について話す</h3>
<p><strong>Q: 男の人は、なぜ会社に着くのが遅くなったと言っていますか。</strong></p>
<ul>
<li>交通事故があったから</li>
<li>電車が少し遅れたから</li>
<li><b>工事で道路が一方通行になっていたから</b> <span style='color:green;'>(correct)</span></li>
<li>朝寝坊したから</li>
</ul>
<p><em>Explanation: 正答の根拠となる会話文は「工事中の道路があって、そこが一方通行になっていたから、みんな迂回しなきゃいけなかったんだよ。」です。事故はなかったと明言されており、電車の遅れについては女の人の話です。学習者は、会話の流れから正しい理由を聞き分け、他の選択肢に惑わされないよう注意が必要です。N3レベルでは、話題に出た情報と実際の理由を区別する聴解力が問われます。</em></p>
<h3>ショッピング体験を説明する</h3>
<p><strong>Q: この会話からわかる「新しいショッピング体験」の特徴は何ですか。</strong></p>
<ul>
<li><b>商品をためしたり、店員と話しながら買い物ができること</b> <span style='color:green;'>(correct)</span></li>
<li>すべての商品がネットでしか買えないこと</li>
<li>お店にコーヒーを飲みながら買い物ができるスペースがあることだけが特徴であること</li>
<li>店員と話さずに買い物をすること</li>
</ul>
<p><em>Explanation: この問題は、会話全体から新しいショッピング体験の主な特徴を聞き取る力を問います。選択肢1が正解で、会話では「商品をためす」「店員が説明する」「安心して買い物ができる」など複数の特徴が述べられています。選択肢2はネットショッピングの特徴であり、会話内容と合いません。選択肢3はスペースについてのみを特徴としていますが、会話では他にも多くの特徴が語られているため不正解です。選択肢4は店員と話さない買い物ですが、会話では「店員と直接話すことの良さ」が強調されています。受験者は会話全体を聞き取り、要点をまとめて正しい選択肢を選ぶ必要があり、情報の取捨選択がポイントとなります。</em></p>
<h3>ペットについて話す</h3>
<p><strong>Q: 男の人はどうして猫を飼い始めましたか。</strong></p>
<ul>
<li>犬が好きだから</li>
<li><b>アパートで犬が飼えなかったから</b> <span style='color:green;'>(correct)</span></li>
<li>友だちにすすめられたから</li>
<li>猫を偶然見つけたから</li>
</ul>
<p><em>Explanation: この問題のポイントは、「犬を飼いたかったが、アパートが犬禁止で猫なら大丈夫だった」という理由を正確に聞き取ることです。N3レベルの語彙・文法で構成されていますが、「～なら大丈夫」「大家さんに言われて」などの表現に注意が必要です。選択肢には内容を混乱させやすいもの（1や4）が含まれており、正確なリスニング理解力が問われます。</em></p>
<h3>新しいスキルを学ぶ計画について話す</h3>
<p><strong>Q: 次の会話を聞いて、話の目的として最も適切なものを選びなさい。</strong></p>
<ul>
<li>新しいスキルを学ぶための目標を決めるため</li>
<li><b>学習の計画を見直すため</b> <span style='color:green;'>(correct)</span></li>
<li>友だちと一緒に勉強するため</li>
<li>新しい本を買うため</li>
</ul>
<p><em>Explanation: この問題では、会話の中で「計画を見直す」という目的が明確に語られるかを聞き取る必要があります。N3レベルの語彙や表現を正しく理解し、話し手の意図に注目することがポイントです。選択肢の中で似た内容があるため、計画の見直しが主題であることを正確に判断できるかが難しさとなります。</em></p>
<h2>概要理解 (summary_understanding)</h2>
<p>3問。聴いた内容の大意やまとめを問う。</p>
<h3>課題と解決策について話す</h3>
<p><strong>Q: このあと、社員はまず何をしますか。</strong></p>
<ul>
<li>他の会社の会議に参加する</li>
<li><b>部門ごとのミーティングの日程を決める</b> <span style='color:green;'>(correct)</span></li>
<li>営業部と開発部でランチ会を開く</li>
<li>部長に仕事の進み具合を報告する</li>
</ul>
<p><em>Explanation: この問題では、「課題」と「解決策」というビジネスシーンでよく使われるN3レベルの語彙が使われています。社員は「私が日程を調整して、みんなに連絡します」と話しているので、次にすることは「ミーティングの日程を決める」ことです。他の選択肢は会話の内容と合っていません。正答を選ぶには、会話の流れと要点を的確につかむ必要があります。</em></p>
<h3>地域のイベントについて話す</h3>
<p><strong>Q: このあと、男の人は何をするつもりですか。</strong></p>
<ul>
<li>伝統的な踊りに参加する</li>
<li><b>イベントで写真を撮る</b> <span style='color:green;'>(correct)</span></li>
<li>屋台で料理を作る</li>
<li>ワークショップに参加する</li>
</ul>
<p><em>Explanation: この問題では、聞き取りの中で「僕は写真を撮る係をやろうかな。イベントの写真をSNSにアップしたら、みんなにも見てもらえるし。」という男の人の発言がポイントです。N3レベルの語彙と表現（屋台、伝統的な踊り、ワークショップ、写真を撮るなど）を使いながら、会話の流れから「次に男の人がすること」を正確に把握する力が問われます。選択肢の中には似た内容も含まれているため、細かい聞き分けと要点の把握が主なチャレンジとなります。</em></p>
<h3>おすすめの旅行先について話す</h3>
<p><strong>Q: 女の人はこのあとどうしますか。</strong></p>
<ul>
<li>京都に行くことを決める</li>
<li>沖縄に行くことを決める</li>
<li><b>どちらに行くか、もう少し考える</b> <span style='color:green;'>(correct)</span></li>
<li>旅行に行くのをやめる</li>
</ul>
<p><em>Explanation: この会話では、女の人が京都と沖縄、どちらに行くか迷っていて、「もう少し考えて、どっちに行くか決めるね」と言っています。したがって、選択肢3が正解です。N3レベルの学習者にとって、「もう少し考えて、どっちに行くか決めるね」という表現が、すぐに決めない＝まだ決めていない、という意味を理解できるかどうかがポイントになります。また、選択肢1や2に引っ張られないように、会話の最後までしっかり聞く必要があります。</em></p>
<h2>積極的表現 (active_expression)</h2>
<p>4問。会話の中で自分の意見や希望を述べる場面を聞き取る。</p>
<h3>キャリア目標について話す</h3>
<p><strong>Q: What will the person pointed by the arrow (the young employee) say next?</strong></p>
<ul>
<li><b>まず、プロジェクトリーダーとしての経験を積みたいです。</b> <span style='color:green;'>(correct)</span></li>
<li>会社を辞めて、海外で働きたいです。</li>
<li>休みを増やして、趣味の時間を増やしたいです。</li>
</ul>
<p><em>Explanation: This question assesses students' ability to understand context and predict a logical, professional response in a career discussion. Options 2 and 3 do not align with the positive and proactive tone expected in a formal career goal setting within a company. Option 1 uses N3-level vocabulary and is the most appropriate and realistic answer in a Japanese workplace setting, reflecting ambition and willingness to contribute to the team. The challenge for students is to distinguish between personal wishes and suitable career goals in a business context.</em></p>
<h3>将来の抱負について話す</h3>
<p><strong>Q: このあと、矢印の女の子（佐藤さん）は何と言いますか。</strong></p>
<ul>
<li><b>私は日本の歴史に興味があるので、歴史の先生になりたいです。</b> <span style='color:green;'>(correct)</span></li>
<li>私はサッカーが好きなので、毎日サッカーの練習をしています。</li>
<li>私は料理が得意なので、今度みんなにお弁当を作ります。</li>
</ul>
<p><em>Explanation: この問題のポイントは、「将来の抱負」というテーマに合う答えを選ぶことです。1番は「歴史の先生になりたい」という明確な将来の目標を理由とともに述べており、N3レベルの文法や語彙が使われています。2番と3番は現在の趣味や特技について述べていますが、「将来どうなりたいか」という抱負にはなっていません。N3学習者にとって、意志や希望を表す「〜たいです」「〜ので」などの文型を理解し、将来の目標を表現できるかが問われています。また、誤答選択肢も自然な内容なので、文脈理解力と将来の抱負表現の区別が試されます。</em></p>
<h3>友人へのプレゼント選びについて話す</h3>
<p><strong>Q: このあと、矢印の人は何と言いますか。</strong></p>
<ul>
<li><b>じゃあ、コーヒーカップを見に行こうかな。</b> <span style='color:green;'>(correct)</span></li>
<li>やっぱり、去年と同じハンカチにしよう。</li>
<li>もうプレゼントは買ってあるよ。</li>
</ul>
<p><em>Explanation: The main challenge in this question is distinguishing between the options based on the context of the conversation. The student pointed by the arrow is indecisive about the gift and is considering new ideas. Option 1 shows a natural next step based on the friend's suggestion. Option 2 contradicts her desire to choose something special and different from last year. Option 3 is not logical, as she is still undecided and hasn't bought anything yet. This tests careful listening and understanding of nuance at the N3 level.</em></p>
<h3>特別な日の計画について話す</h3>
<p><strong>Q: What will the mother say next?</strong></p>
<ul>
<li>じゃあ、一緒にケーキを作りましょうか。</li>
<li><b>今日はあなたの好きなレストランに行く予定よ。</b> <span style='color:green;'>(correct)</span></li>
<li>今から公園で散歩しましょう。</li>
</ul>
<p><em>Explanation: The main challenge for students is to listen for key information about the special day and the planned activity. The dialogue context makes it clear that the family is celebrating the daughter’s birthday and the father has suggested going out for dinner. Option 2 matches this context, using the N3-level phrase 「行く予定」 to express a future plan. Option 1 and 3 mention other activities (making a cake, going for a walk) that are not mentioned in the dialogue, so students need to distinguish the correct future plan based on the listening.</em></p>
<h2>即時応答 (immediate_ack)</h2>
<p>9問。短い会話や問いかけに対して即座に適切な返答を選ぶ。</p>
<h3>バスの時刻表を尋ねる</h3>
<p><strong>Q: このあと、男の人はどうしますか。</strong></p>
<p><em>Explanation: この問題のポイントは、会話の流れから「次のバスは10時15分に出発しますよ」と言われた後、男の人がどうするかを考えることです。N3レベルの学習者にとって、「時刻表」「出発」「待つ」など基本的な語彙の理解が問われます。選択肢1は「すぐにバスに乗る」ですが、バスはまだ来ていません。選択肢2は「もう一度聞く」ですが、すでに答えをもらっています。したがって、正しい選択は「10時15分まで待ちます」となります。時間表現や会話の流れを正確に理解することが主なチャレンジポイントです。</em></p>
<h3>領収書を求める</h3>
<p><strong>Q: 会話を聞いて、次に男の人が何をするか、最もよいものを選びなさい。</strong></p>
<p><em>Explanation: この問題のポイントは、「領収書をいただけますか？」という依頼に対して、男の人が「今、レジで発行します」と答えているため、次に男の人がすることは領収書の発行です。N3レベルの学習者にとって、「発行する」「領収書」などの語彙の聞き取りと、会話の流れを正確に理解することが課題となります。選択肢の中には他の店員の業務（コーヒーを作る、サンドイッチを温める）も含まれているため、聞き取りの注意力が求められます。</em></p>
<h3>おすすめを尋ねる</h3>
<p><strong>Q: この後、男の人はどうしますか。</strong></p>
<p><em>Explanation: この問題のポイントは、「今から行ってみます」という男の人の発言です。これは、女の人がすすめた駅前のイタリアンレストランに行く意志を表しています。N3レベルの語彙と表現を使っていますが、「おすすめ」「行ってみます」など、聞き取りが苦手な学習者は話の流れを正確に理解できないことがあります。「おすすめを尋ねる」→「すすめられた場所に行く」という自然な流れを押さえましょう。</em></p>
<h3>引っ越しの準備について話す</h3>
<p><strong>Q: 男の人はこれから何をしますか。</strong></p>
<p><em>Explanation: この問題の主な難しさは、「これから何をするか」という未来の行動を聞き取る点です。会話では「今日はまず、いらないものを捨てようと思っています」という発言があり、意志表現「〜ようと思っています」に注目できるかがポイントです。N3レベルの語彙「いらないもの」「捨てる」も重要です。</em></p>
<h3>学校や職場での一日の流れについて話す</h3>
<p><strong>Q: 次の会話を読んで、最も適切なものを一つ選びなさい。</strong></p>
<p><em>Explanation: この会話では、田中さんが「これから教室の掃除をします」と言っています。選択肢1の「休憩」や選択肢3の「また仕事をします」といった内容は会話に出てきません。田中さんの次の行動は掃除をすることなので、2が正解です。会話の流れを正確に読み取り、登場人物の次の行動を理解することがポイントです。</em></p>
<h3>言語学習のコツについて話す</h3>
<p><strong>Q: 会話の内容から、女の人が次に取る行動として最も適切なものを一つ選びなさい。</strong></p>
<p><em>Explanation: この問題は、会話の流れを理解し、アドバイスをどう受け止めて行動を変えるかを問うものです。N3レベルの「日記を書く」「ドラマを見る」「楽しみながら勉強する」といった表現が理解できるかがポイントです。選択肢も具体的に違いが分かるように設定しており、正しい選択肢を選ぶためには、会話の意図を正確に読み取る必要があります。</em></p>
<h3>好きな季節について話す</h3>
<p><strong>Q: 今週末、男の人は何をしますか。</strong></p>
<p><em>Explanation: 男の人は「今週末も友だちと山に行くつもりです」と言っています。「紅葉を見に行く」という前の発言と合わせて、1が正解です。この問題のポイントは、「紅葉」「山に行く」「つもり」の語彙や表現を正しく理解できるかどうかです。秋の季節感や日本の文化的な行動を反映した内容で、N3レベルの学習者に適しています。選択肢も季節や会話内容に関連し、消去法だけでなく内容理解が必要となっています。</em></p>
<h3>地元の食べ物や特産品について話す</h3>
<p><strong>Q: 田中さん（女）はこの後どうしますか。</strong></p>
<p><em>Explanation: この問題のポイントは、会話の流れを正しく理解し、田中さん（女）が次に何をするかを推測することです。最後に田中さん（男）が「僕も買ってみようかな」と言っているので、田中さん（女）は「おすすめですよ」などとすすめる可能性が高いです。選択肢2や3は会話からは自然ではありません。N3レベルの語彙や表現を使い、ふりがなも付けて、より分かりやすくしました。</em></p>
<h3>健康診断や医者への訪問について話す</h3>
<p><strong>Q: このあと、男の人は何をしますか。</strong></p>
<p><em>Explanation: この問題のポイントは、「これからどうするんですか？」という質問に対して、男の人が「医者に相談しに行こうと思っています」と答えていることです。したがって、男の人が次にすることは「病院に行って医者に相談する」になります。N3レベルでは、「相談する」「健康診断」などの語彙の理解と、会話の流れから次の行動を推測する力が問われます。選択肢2や3にひきずられず、会話文の意図を正確につかむことが主なチャレンジです。</em></p>
</body></html>

In [13]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"../../JLPT_{timestamp}.html"

with open(filename, "w", encoding="utf-8") as file:
    file.write(html_output)